In [261]:
require(haven)
require(data.table)
source('model_functions.R')

In [262]:
dt_orig <- readRDS('../data/DB_perimetro.RDS')

#master_d4 <- readRDS('../data/master_drop4_v4.RDS')

In [263]:
table(dt_orig$STATO_PRATICA)


    10 
140108 

In [264]:
nrow(dt_orig)

[1] 140108

In [265]:
table(dt_orig[,`Colore - NUMERO INTERROGAZIONI`], useNA = 'always')


GIALLO  VERDE   <NA> 
   475  26929 112704 

In [266]:
dt_orig[,anno := substr(as.Date(DATA_DELIBERA_O_ANNULL_FIDO),1,4)]

In [267]:
head(dt_orig,1)

NUM_PRATICA,PROGRESS_PRATICA,STATO_PRATICA,CHIAVE_DB,ABI,ABI_PEF5,FILIALE_PEF5,ANNO_PEF5,PROGRESSIVO_PEF5,TIPO_PRATICA,⋯,I5-I6,I4,I3,I1-I2,NUOVI_VERDI,NUOVO_ESITO_FINALE,NUOVI_VERDI_SENZA_SOGLIA,NUOVO_ESITO_SENZA SOGLIA,target,anno
<dbl>,<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
102230026,1,10,1899-12-31 10:01:10,NA,NA,NA,NA,NA,3,⋯,n.a.,n.a.,n.a.,n.a.,0,ROSSO,0,ROSSO,OK,2020


In [268]:
round(100*sum(dt_orig[anno == 2017,]$target=="DEFAULT")/nrow(dt_orig[ anno == 2017,]),2)
round(100*sum(dt_orig$target=="DEFAULT")/nrow(dt_orig),2)

[1] 4.72

[1] 3.02

In [269]:
tab_td <- dt_orig[,list(td = round(100*sum(target=="DEFAULT")/.N,2)), by = c('anno','MacroSegmento_Attuale LC')][order(anno)]

tab_td <- dcast(tab_td, `MacroSegmento_Attuale LC` ~ anno, value.var = "td")



tab_td_appo <- dt_orig[,list(td = round(100*sum(target=="DEFAULT")/.N,2)), by = c('MacroSegmento_Attuale LC')]
tab_td <- merge(tab_td,tab_td_appo, by = 'MacroSegmento_Attuale LC' )

row_td <- dt_orig[,list(td = round(100*sum(target=="DEFAULT")/.N,2)), by = c('anno')][order(anno)]
row_td[,`MacroSegmento_Attuale LC` := "All"]
row_td<- dcast(row_td,`MacroSegmento_Attuale LC` ~ anno, value.var = "td")
row_td[,td := round(100*sum(dt_orig$target=="DEFAULT")/nrow(dt_orig),2)]


write.xlsx(rbind(tab_td,row_td), 
           'numeriche_db_completo.xlsx',
            sheetName = 'td',
           row.names = FALSE,  
           append = FALSE)

In [270]:
table(dt_orig[TIPO_PRODOTTO == 9,anno])


2017 2018 2019 2020 2021 
1312 1281  792  277  356 

In [271]:
tab_colore <- dt_orig[ ,list(n = .N), by = c('anno','ESITO_FINALE_VPC')][order(anno)]

tab_colore<- dcast(tab_colore, ESITO_FINALE_VPC ~ anno, value.var = "n")

write.xlsx(tab_colore, 
           'numeriche_varie.xlsx',
            sheetName = 'colore',
           row.names = FALSE,  
           append = FALSE)

In [272]:
tab_tipo_prod <- dt_orig[ ,list(n_t_p = .N), by = c('anno','TIPO_PRODOTTO')][order(anno)]
count_anno <- dt_orig[ ,list(n_anno = .N), by = c('anno')][order(anno)]
tab_tipo_prod_appo <- merge(tab_tipo_prod,
count_anno,
      by = 'anno')
tab_tipo_prod_appo[,perc:= round(100*n_t_p/n_anno,2)]
tab_tipo_prod_appo_wide<- dcast(tab_tipo_prod_appo, TIPO_PRODOTTO ~ anno, value.var = "perc")
tab_tipo_prod_appo_wide
write.xlsx(tab_tipo_prod_appo_wide, 
           'numeriche_varie.xlsx',
            sheetName = 'tipo_prodotto_perc',
           row.names = FALSE,  
           append = TRUE)
write.xlsx(tab_tipo_prod_appo, 
           'numeriche_varie.xlsx',
            sheetName = 'tipo_prodotto_num',
           row.names = FALSE,  
           append = TRUE)

TIPO_PRODOTTO,2015,2016,2017,2018,2019,2020,2021
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,100,100,94.2,97.25,97.71,98.53,NA
9,NA,NA,5.8,2.75,2.29,1.47,100


In [273]:
tab_tipo_prod_appo

anno,TIPO_PRODOTTO,n_t_p,n_anno,perc
<chr>,<dbl>,<int>,<int>,<dbl>
2015,1,29,29,100.00
2016,1,17091,17091,100.00
2017,1,21316,22628,94.20
2017,9,1312,22628,5.80
2018,1,45254,46535,97.25
2018,9,1281,46535,2.75
2019,1,33860,34652,97.71
2019,9,792,34652,2.29
2020,1,18540,18817,98.53


In [274]:
tab_numeriche <- dt_orig[ ,list(count = .N), by = c('anno','MacroSegmento_Attuale LC')][order(anno)]
tab_numeriche <- dcast(tab_numeriche, `MacroSegmento_Attuale LC` ~ anno, value.var = "count")
tab_numeriche[,Totale := rowSums(.SD,na.rm = TRUE), .SDcols = 2:8]
write.xlsx(tab_numeriche, 
           'numeriche_db_completo.xlsx',
            sheetName = 'numeriche',
           row.names = FALSE,  
           append = TRUE)

In [275]:
max(as.Date(dt_orig[  TIPO_PRODOTTO== '1',]$DATA_DELIBERA_O_ANNULL_FIDO))

[1] "2020-10-30"

In [276]:
write.xlsx(dt_orig[,list(count = .N), by = c('anno','TIPO_PRODOTTO')][order(anno,TIPO_PRODOTTO)], 
           'pratiche_per_anno.xlsx',
            sheetName = 'tipo_prodotto',
           row.names = FALSE,  
           append = FALSE)

In [277]:
write.xlsx(dt_orig[,list(count = .N), by = c('anno','ESITO_FINALE_VPC')][order(anno,ESITO_FINALE_VPC)],
           'pratiche_per_anno.xlsx',
            sheetName = 'colore_vpc',
           row.names = FALSE,  
           append = TRUE)

In [278]:
master_orig <- readRDS('../data/master_drop4_v4.RDS')

In [279]:
100*nrow(dt_orig[`MacroSegmento_Attuale LC` != "Sme Retail" & is.na(COD_COLORE_CLASSE_T0),])/nrow(dt_orig[`MacroSegmento_Attuale LC` != "Sme Retail",])

[1] 35.08489

In [280]:
sort(unique(substr(as.Date(dt_orig[`MacroSegmento_Attuale LC` != "Sme Retail" & !is.na(COD_COLORE_CLASSE_T0),DATA_DELIBERA_O_ANNULL_FIDO]),1,7)))

[1] "2015-12" "2016-01" "2016-02" "2016-03" "2016-04" "2016-05" "2016-06"
 [8] "2016-07" "2016-08" "2016-09" "2016-10" "2016-11" "2016-12" "2017-01"
[15] "2017-02" "2017-03" "2017-04" "2017-05" "2017-06" "2017-07" "2017-08"
[22] "2017-09" "2017-10" "2017-11" "2017-12" "2018-01" "2018-02" "2018-03"
[29] "2018-04" "2018-05" "2018-06" "2018-07" "2018-08" "2018-09" "2018-10"
[36] "2018-11" "2018-12" "2019-01" "2019-02" "2019-03" "2019-04" "2019-05"
[43] "2019-06" "2019-07" "2019-08" "2019-09" "2019-10" "2019-11" "2019-12"
[50] "2020-01" "2020-02" "2020-03" "2020-04" "2020-05" "2020-06" "2020-07"
[57] "2020-08" "2020-09" "2020-10" "2020-11" "2020-12" "2021-01" "2021-02"
[64] "2021-03" "2021-04" "2021-05" "2021-06"

## RATING 

In [993]:
dt_orig_rat <- copy(dt_orig)

In [994]:
dt_rating_factoring <- readRDS('../data/RDS/tab_ratin_calibrazione.RDS')

###banking scaricato con escl_fact_pcr_pro_sol_sndg = 0.
dt_rating_bank <- readRDS('../data/RDS/rating_banking_ok.RDS')

In [995]:
# dt_rat_t0_missing <- dt_orig[`MacroSegmento_Attuale LC` == 'Sme Retail' & (is.na(Rating_Inizio_Periodo) | Rating_Inizio_Periodo== "UR"),]

In [996]:
# nrow(dt_rat_t0_missing)/nrow(dt_orig[`MacroSegmento_Attuale LC` == 'Sme Retail' ,])

In [997]:
nrow(dt_rating_bank)
nrow(dt_rating_factoring)

[1] 159431

[1] 60966

In [998]:
sort(unique(dt_rating_factoring$DAT_FINE_PERIO))
sort(unique(dt_rating_bank$DAT_FINE_PERIO))

[1] "2013-12-31" "2014-12-31" "2015-12-31" "2016-12-31" "2017-12-31"
[6] "2018-12-31" "2019-12-31"

[1] "2013-12-31" "2014-12-31" "2015-12-31" "2016-12-31" "2017-12-31"
[6] "2018-12-31" "2019-12-31" "2020-12-31"

In [999]:
sort(unique(dt_rating_factoring$rating_finale_post_notch))
sort(unique(dt_rating_bank$rating_post_cal_Naff_finale))

[1] "I1" "I2" "I3" "I4" "I5" "I6" "M1" "M2" "M3" "M4" "R1" "R2" "R3" "R4" "R5"

[1] "I1" "I2" "I3" "I4" "I5" "I6" "M1" "M2" "M3" "M4" "R1" "R2" "R3" "R4" "R5"

In [1000]:
dt_rating <- merge(
    dt_rating_factoring,
    dt_rating_bank, 
    by=c("COD_SNDG",'DAT_FINE_PERIO'),
    all=TRUE
)

In [1001]:
table(dt_rating$DAT_FINE_PERIO)


2013-12-31 2014-12-31 2015-12-31 2016-12-31 2017-12-31 2018-12-31 2019-12-31 
     15695      18136      17447      21398      31415      44437      45576 
2020-12-31 
     26293 

In [1002]:
dt_rating[,classe_rating := ifelse(!is.na(rating_post_cal_Naff_finale),rating_post_cal_Naff_finale,rating_finale_post_notch)]

In [1003]:
table(dt_rating$classe_rating,useNA = 'ifany')


   I1    I2    I3    I4    I5    I6    M1    M2    M3    M4    R1    R2    R3 
12977 11613 19458 18412 14843 22673 33828 30999 24334 14912  6322  5141  2371 
   R4    R5 
 1845   669 

In [1004]:
sub_dati_rat <- dt_orig_rat[,
                    c('NUM_PRATICA','PROGRESS_PRATICA','SNDG_CEDUTO','DATA_DELIBERA_O_ANNULL_FIDO',
                      'MacroSegmento_Attuale LC','Rating_Inizio_Periodo','COD_COLORE_CLASSE_T0')]

In [1005]:
sub_dati_rat[,data_rating := as.Date(paste0(year(DATA_DELIBERA_O_ANNULL_FIDO)-1,"-12-31")) ]

In [1006]:
data_tot_rat <-merge(
    sub_dati_rat,
    dt_rating,
    by.x=c("SNDG_CEDUTO"),
    by.y=c("COD_SNDG"),
    all.x=TRUE
)

In [1007]:
rating_max<-data_tot_rat[DAT_FINE_PERIO<=data_rating,
                      .(d_fineperiodo_max=max(
                          DAT_FINE_PERIO,na.rm=TRUE)),
                         by=c("SNDG_CEDUTO",'DATA_DELIBERA_O_ANNULL_FIDO')]

In [1008]:
data_merge_2_rat<-unique(merge(
    sub_dati_rat,
    rating_max,
    by.x=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    by.y=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    all.x=TRUE
))

In [1009]:
table(data_merge_2_rat$d_fineperiodo_max, useNA = 'ifany')


2013-12-31 2014-12-31 2015-12-31 2016-12-31 2017-12-31 2018-12-31 2019-12-31 
      1323       3537       5629       6344      11878       8833       5809 
2020-12-31       <NA> 
        27      96728 

In [1010]:
colnames(dt_rating)

[1] "COD_SNDG"                    "DAT_FINE_PERIO"             
 [3] "FORMA_GIURIDICA"             "AREA_GEO"                   
 [5] "FAMIGLIA_ANAG"               "flag_esclusivo_cf"          
 [7] "SIZE"                        "FLAG_HR"                    
 [9] "DEFAULT"                     "DEFAULT_S100"               
[11] "SamplingWeight"              "score_cr0"                  
[13] "score_crsys"                 "score_anag"                 
[15] "score_bil_integ"             "score_bil_integ_w"          
[17] "score_anag_w"                "score_crsys_w"              
[19] "score_cr0_w"                 "score_integ"                
[21] "FLAG_DENOTCHING"             "rating_pdc"                 
[23] "rating_finale"               "offset"                     
[25] "prob"                        "score_integr_offset"        
[27] "prob_offset"                 "rating_finale_post_notch"   
[29] "pd_finale_post_notch"        "pd_post_lim"                
[31] "rating_post_cal_Naff_finale" "classe_rating"

In [1011]:
colnames(dt_rating)[!colnames(dt_rating)%in% c('COD_SNDG','DAT_FINE_PERIO')] <- paste0(
    "RAT_",
    colnames(dt_rating)[!colnames(dt_rating)%in% c('COD_SNDG','DAT_FINE_PERIO')])

In [1012]:
master_rating <- unique(merge(
    data_merge_2_rat,
    dt_rating,
    by.x=c("SNDG_CEDUTO","d_fineperiodo_max"),
    by.y=c("COD_SNDG","DAT_FINE_PERIO"),
    all.x=TRUE
))
#master_rating[,d_fineperiodo_max := NULL]
master_rating[,data_rating := NULL]
master_rating[,RAT_delta_t := round(as.numeric(as.Date(DATA_DELIBERA_O_ANNULL_FIDO) - d_fineperiodo_max)/356,1)]
cols_master <- colnames(master_rating)

In [1013]:
master_rating[`MacroSegmento_Attuale LC` !='Sme Retail',RAT_classe_rating := Rating_Inizio_Periodo]
master_rating[`MacroSegmento_Attuale LC` !='Sme Retail',RAT_delta_t := NA]
master_rating <- master_rating[,..cols_master]

In [1014]:
master_rating[,RAT_classe_rating_filt:= RAT_classe_rating]
master_rating[RAT_delta_t >1 & `MacroSegmento_Attuale LC` =='Sme Retail',RAT_classe_rating_filt:= NA]

In [1015]:
table(master_rating$RAT_classe_rating_filt, useNA = 'always')


   I1  I1.A  I1.C  I1.D  I1.E  I1.F    I2    I3    I4    I5    I6    M1    M2 
 3151     1    24    88    10    19  3680  6095  6998  6785  7482  7563  7318 
   M3    M4   MT1   MT2   MT3   MT4   MT5   MT6   MT8   MT9    R1    R2    R3 
 6596  5107     1     1     3     4     3     2     1     2  2141  1296   665 
   R4    R5  RE07  RE08  RE11  RT01  RT02  RT03  RT04  RT05  RT06  RT07  RT08 
  465   918     3     2     2     4     9     6     7    18    15    16    15 
 RT09  RT10  RT11  RT12  RT13  RT14    UR  <NA> 
   17     5     2     4     1     2  1407 72154 

In [1016]:
master_rating[grepl("RT",RAT_classe_rating_filt), RAT_classe_rating_filt := "RT" ]
master_rating[grepl("MT",RAT_classe_rating_filt), RAT_classe_rating_filt := "MT" ]
master_rating[grepl("RE",RAT_classe_rating_filt), RAT_classe_rating_filt := "RE" ]
master_rating[grepl("I1",RAT_classe_rating_filt), RAT_classe_rating_filt := "I1" ]

In [1017]:
str(dt_rating)

Classes ‘data.table’ and 'data.frame':	220397 obs. of  32 variables:
 $ COD_SNDG                       : chr  "0000000000007435" "0000000000007435" "0000000000008626" "0000000000009554" ...
  ..- attr(*, "label")= chr "Super NDG"
  ..- attr(*, "format.sas")= chr "$16"
 $ DAT_FINE_PERIO                 : Date, format: "2019-12-31" "2020-12-31" ...
 $ RAT_FORMA_GIURIDICA            : chr  NA NA NA NA ...
  ..- attr(*, "format.sas")= chr "$5"
 $ RAT_AREA_GEO                   : chr  NA NA NA NA ...
  ..- attr(*, "format.sas")= chr "$15"
 $ RAT_FAMIGLIA_ANAG              : chr  NA NA NA NA ...
  ..- attr(*, "format.sas")= chr "$100"
 $ RAT_flag_esclusivo_cf          : num  NA NA NA NA NA NA NA NA NA NA ...
  ..- attr(*, "format.sas")= chr "BEST12"
 $ RAT_SIZE                       : num  NA NA NA NA NA NA NA NA NA NA ...
 $ RAT_FLAG_HR                    : num  NA NA NA NA NA NA NA NA NA NA ...
 $ RAT_DEFAULT                    : num  NA NA NA NA NA NA NA NA NA NA ...
  ..- attr(*, "format

In [1018]:
dt_rating_trend <- readRDS('../data/RDS/rating_trend.RDS')


In [1019]:
master_rating <- merge(master_rating,
                      dt_rating_trend,
                      by = c('NUM_PRATICA','PROGRESS_PRATICA'),
                      all.x = TRUE)

### CRA

In [1020]:
master_rating[, CRA_ADJ := COD_COLORE_CLASSE_T0]

master_rating[`MacroSegmento_Attuale LC` == "Sme Retail", 
              CRA_ADJ_sme_num := as.numeric(factor(RAT_classe_rating, 
                                               levels = c("I1", "I2", "I3", "I4", "I5", "I6", 
                                                          "M1", "M2","M3", "M4","R1", "R2", "R3", "R4", "R5")))]


# cra_verde <- c("I1", "I2", "I3", "I4", "I5", "I6", "M1", "M2")
# cra_giallo <- c("M3", "M4")
# cra_rosso <- c("R1", "R2", "R3", "R4", "R5")
# master_rating[CRA_ADJ %in% cra_verde, CRA_ADJ := "00"]
# master_rating[CRA_ADJ %in% cra_giallo, CRA_ADJ := "01"]
# master_rating[CRA_ADJ %in% cra_rosso, CRA_ADJ := "02"]

In [1021]:
dt_notch_cra <- readRDS('../data/RDS/dt_notch_CRA.RDS')

In [1022]:
master_rating <- merge(master_rating,
                      dt_notch_cra,
                    by = c('NUM_PRATICA','PROGRESS_PRATICA'),
                      all.x = TRUE)

In [1023]:
colnames(master_rating)

[1] "NUM_PRATICA"                     "PROGRESS_PRATICA"               
 [3] "SNDG_CEDUTO"                     "d_fineperiodo_max"              
 [5] "DATA_DELIBERA_O_ANNULL_FIDO"     "MacroSegmento_Attuale LC"       
 [7] "Rating_Inizio_Periodo"           "COD_COLORE_CLASSE_T0"           
 [9] "RAT_FORMA_GIURIDICA"             "RAT_AREA_GEO"                   
[11] "RAT_FAMIGLIA_ANAG"               "RAT_flag_esclusivo_cf"          
[13] "RAT_SIZE"                        "RAT_FLAG_HR"                    
[15] "RAT_DEFAULT"                     "RAT_DEFAULT_S100"               
[17] "RAT_SamplingWeight"              "RAT_score_cr0"                  
[19] "RAT_score_crsys"                 "RAT_score_anag"                 
[21] "RAT_score_bil_integ"             "RAT_score_bil_integ_w"          
[23] "RAT_score_anag_w"                "RAT_score_crsys_w"              
[25] "RAT_score_cr0_w"                 "RAT_score_integ"                
[27] "RAT_FLAG_DENOTCHING"             "RAT_rating_pdc"                 
[29] "RAT_rating_finale"               "RAT_offset"                     
[31] "RAT_prob"                        "RAT_score_integr_offset"        
[33] "RAT_prob_offset"                 "RAT_rating_finale_post_notch"   
[35] "RAT_pd_finale_post_notch"        "RAT_pd_post_lim"                
[37] "RAT_rating_post_cal_Naff_finale" "RAT_classe_rating"              
[39] "RAT_delta_t"                     "RAT_classe_rating_filt"         
[41] "RAT_andamento"                   "CRA_ADJ"                        
[43] "CRA_ADJ_sme_num"                 "notch"

In [1024]:
master_rating[,CRA_ADJ_sme_notch := ifelse(!is.na(notch),CRA_ADJ_sme_num + notch, CRA_ADJ_sme_num)]

In [1025]:
master_rating[!is.na(CRA_ADJ_sme_notch),CRA_ADJ_sme_col := "GIALLO"]
master_rating[CRA_ADJ_sme_notch <= 8,CRA_ADJ_sme_col := "VERDE"]
master_rating[CRA_ADJ_sme_notch >10  ,CRA_ADJ_sme_col := "ROSSO"]

In [1026]:
master_rating[`MacroSegmento_Attuale LC` == "Sme Retail", CRA_ADJ := CRA_ADJ_sme_col]

In [1028]:
table(master_rating[`MacroSegmento_Attuale LC` == "Sme Retail",]$CRA_ADJ, useNA = "always")


GIALLO  ROSSO  VERDE   <NA> 
  8164   3365  25013  52088 

## FORMA GIURIDICA

In [1029]:
dt_f_giu <- setDT(readRDS('../data/RDS/dt_formagiuridica.RDS'))

In [1030]:
colnames(dt_f_giu)

[1] "NUM_PRATICA"      "PROGRESS_PRATICA" "SNDG_CEDUTO"      "COD_TP_CLI"

In [1031]:
master_rating <- merge(master_rating,
                       dt_f_giu[,.(NUM_PRATICA,PROGRESS_PRATICA,COD_TP_CLI)],
                       by = c('NUM_PRATICA','PROGRESS_PRATICA'),
                       all.x = TRUE)

In [1032]:
table(master_rating[is.na(RAT_FORMA_GIURIDICA),COD_TP_CLI], useNA = 'always')


         AG  ASNR ASRIC  COND  CONS COOSC    DI  ENTE  EREL FALLC FALNC FILIT 
   19     1     5     3   252    46    37 38073    28     2     2     1     3 
  FON  MIGR    PF PSCRL   SAA   SAS    SC  SCRI  SCRL SCRLA   SDF    SE   SNC 
    1     1    88     1     2  5994     1    13  1297     1    12     1  8282 
  SPA   SRL    SS   STU  <NA> 
  118 22566   122     6 55159 

In [1033]:
master_rating[`MacroSegmento_Attuale LC` != "Sme Retail" & is.na(COD_TP_CLI),]

NUM_PRATICA,PROGRESS_PRATICA,SNDG_CEDUTO,d_fineperiodo_max,DATA_DELIBERA_O_ANNULL_FIDO,MacroSegmento_Attuale LC,Rating_Inizio_Periodo,COD_COLORE_CLASSE_T0,RAT_FORMA_GIURIDICA,RAT_AREA_GEO,⋯,RAT_classe_rating,RAT_delta_t,RAT_classe_rating_filt,RAT_andamento,CRA_ADJ,CRA_ADJ_sme_num,notch,CRA_ADJ_sme_notch,CRA_ADJ_sme_col,COD_TP_CLI
<dbl>,<int>,<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>,<chr>
102230030,1,0000000017851888,NA,2020-01-02-23.04.55.267865,Sme Corporate,M3,01,NA,NA,⋯,M3,NA,M3,Stabile,01,NA,NA,NA,NA,NA
102230031,1,0000000067790030,NA,2020-01-02-23.04.55.267865,Sme Corporate,I5,00,NA,NA,⋯,I5,NA,I5,Peggiorato,00,NA,NA,NA,NA,NA
102230032,1,0000000017591710,2018-12-31,2020-01-02-23.04.55.267865,Sme Corporate,I6,00,NA,NA,⋯,I6,NA,I6,Stabile,00,NA,NA,NA,NA,NA
102230039,1,0000000017779860,NA,2020-01-03-16.30.33.281674,Large corporate,I6,00,NA,NA,⋯,I6,NA,I6,Stabile,00,NA,NA,NA,NA,NA
102230040,1,0000000027621470,NA,2020-01-03-17.00.33.418378,Corporate,I5,NA,NA,NA,⋯,I5,NA,I5,Stabile,NA,NA,NA,NA,NA,NA
103230023,1,0000000017844258,NA,2020-01-03-23.25.52.535120,Corporate,I4,00,NA,NA,⋯,I4,NA,I4,Peggiorato,00,NA,NA,NA,NA,NA
103230030,2,0000000090309384,2019-12-31,2020-02-07-13.01.28.250683,Sme Corporate,I4,00,NA,NA,⋯,I4,NA,I4,Stabile,00,NA,NA,NA,NA,NA
103230032,1,0000000017797806,NA,2020-01-03-23.25.52.535120,Sme Corporate,I4,00,NA,NA,⋯,I4,NA,I4,Non presente rating ultimo trimestre,00,NA,NA,NA,NA,NA
103230035,1,0000000089539627,2019-12-31,2020-02-13-15.00.31.167214,Sme Corporate,I6,NA,SRL,4 - SUD ISOLE,⋯,I6,NA,I6,Non presente rating ultimo trimestre,NA,NA,NA,NA,NA,NA


In [1034]:
master_rating[, FORMA_GIURIDICA := ifelse(!is.na(COD_TP_CLI),COD_TP_CLI,RAT_FORMA_GIURIDICA)]

In [1035]:
cols_2keep_rat <- c('NUM_PRATICA','PROGRESS_PRATICA','CRA_ADJ','RAT_classe_rating_filt','RAT_classe_rating','FORMA_GIURIDICA','RAT_delta_t','RAT_andamento')

In [1036]:
master_rating_sme <- master_rating[`MacroSegmento_Attuale LC` == "Sme Retail",..cols_2keep_rat]
master_rating_nonsme <- master_rating[`MacroSegmento_Attuale LC` != "Sme Retail" & `MacroSegmento_Attuale LC` != "Altro",..cols_2keep_rat]
saveRDS(master_rating_sme,'../data/RDS/master/sme/master_rating.RDS')
saveRDS(master_rating_nonsme,'../data/RDS/master/non_sme/master_rating.RDS')
saveRDS(master_rating,'../data/RDS/master/master_rating.RDS')

In [317]:
write.csv(master_rating[,.(SNDG_CEDUTO,DATA_DELIBERA_O_ANNULL_FIDO,NUM_PRATICA,PROGRESS_PRATICA,`MacroSegmento_Attuale LC`,RAT_classe_rating_filt)],
           'dati_rating_modello.csv',
           row.names = FALSE)

In [318]:

cols_small <- c('SNDG_CEDUTO','DATA_DELIBERA_O_ANNULL_FIDO',
                'NUM_PRATICA','PROGRESS_PRATICA','MacroSegmento_Attuale LC','Rating_Inizio_Periodo',
               'RAT_rating_post_cal_Naff_finale','RAT_rating_finale_post_notch','RAT_classe_rating','RAT_classe_rating_filt')
master_rat_small <- master_rating[,..cols_small]
str(master_rat_small)

Classes ‘data.table’ and 'data.frame':	140108 obs. of  10 variables:
 $ SNDG_CEDUTO                    : chr  "0000000091391367" "0000000088606666" "0000000017851888" "0000000067790030" ...
 $ DATA_DELIBERA_O_ANNULL_FIDO    : chr  "2020-01-02-23.04.55.267865" "2020-01-02-23.04.55.267865" "2020-01-02-23.04.55.267865" "2020-01-02-23.04.55.267865" ...
 $ NUM_PRATICA                    : num  1.02e+08 1.02e+08 1.02e+08 1.02e+08 1.02e+08 ...
 $ PROGRESS_PRATICA               : int  1 2 1 1 1 1 1 1 1 1 ...
 $ MacroSegmento_Attuale LC       : chr  "Sme Retail" "Sme Retail" "Sme Corporate" "Sme Corporate" ...
 $ Rating_Inizio_Periodo          : chr  "I3" NA "M3" "I5" ...
 $ RAT_rating_post_cal_Naff_finale: chr  "I3" NA NA NA ...
 $ RAT_rating_finale_post_notch   : chr  NA NA NA NA ...
 $ RAT_classe_rating              : chr  "I3" NA "M3" "I5" ...
 $ RAT_classe_rating_filt         : chr  "I3" NA "M3" "I5" ...
 - attr(*, ".internal.selfref")=<externalptr> 
 - attr(*, "sorted")= chr [1:2] "NUM_PR

In [319]:
nrow(master_rating[!is.na(RAT_rating_finale_post_notch) & !is.na(RAT_rating_post_cal_Naff_finale),])
nrow(master_rating[!is.na(RAT_rating_finale_post_notch) & is.na(RAT_rating_post_cal_Naff_finale),])
nrow(master_rating[is.na(RAT_rating_finale_post_notch) & !is.na(RAT_rating_post_cal_Naff_finale),])
nrow(master_rating[RAT_rating_finale_post_notch == RAT_rating_post_cal_Naff_finale,])
master_rating[!is.na(RAT_rating_finale_post_notch) & !is.na(RAT_rating_post_cal_Naff_finale),c('RAT_rating_finale_post_notch','RAT_rating_post_cal_Naff_finale')]

[1] 0

[1] 7971

[1] 35409

[1] 0

RAT_rating_finale_post_notch,RAT_rating_post_cal_Naff_finale
<chr>,<chr>


In [320]:
# table(master_orig[  macrosegmento_attuale_lc =='Sme Retail' ,classe_rating],useNA = 'ifany')

# rbindlist(lapply(unique(master_orig$macrosegmento_attuale_lc), function(seg){
#     missing_val <- sum(is.na(master_orig[macrosegmento_attuale_lc==seg]$classe_rating))
#     all_val <- nrow(master_orig[macrosegmento_attuale_lc==seg])
#     perc <- 100*missing_val/all_val
#     return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
# }))

In [321]:
if(!file.exists('config/config_RATING_cols.R')){
    write.table(paste0("cols_rating <- c('",paste(colnames(master_rating),collapse = "',\n'"),"')"),
            "config/config_RATING_cols.R", sep=";",col.names = FALSE, row.names = FALSE,quote = FALSE)
}

source('config/config_RATING_cols.R')

In [322]:
rbindlist(lapply(unique(master_rating$`MacroSegmento_Attuale LC`), function(seg){
    missing_val <- sum(is.na(master_rating[`MacroSegmento_Attuale LC`==seg]$RAT_classe_rating))
    all_val <- nrow(master_rating[`MacroSegmento_Attuale LC`==seg])
    perc <- 100*missing_val/all_val
    
    missing_val_filt <- sum(is.na(master_rating[`MacroSegmento_Attuale LC`==seg]$RAT_classe_rating_filt))
    all_val_filt <- nrow(master_rating[`MacroSegmento_Attuale LC`==seg])
    perc_filt <- 100*missing_val_filt/all_val_filt
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2), percentuale_missing_filt = round(perc_filt,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing,percentuale_missing_filt
<chr>,<int>,<int>,<dbl>,<dbl>
Sme Retail,52088,88630,58.77,65.63
Sme Corporate,4044,31229,12.95,12.95
Large corporate,55,3010,1.83,1.83
Corporate,182,6765,2.69,2.69
Altro,9709,10474,92.70,92.70


In [323]:

# table(master_rating$RAT_classe_rating,useNA = 'ifany')
# table(master_rating$RAT_rating_post_cal_Naff_finale,useNA = 'ifany')
# table(master_rating$RAT_rating_finale_post_notch,useNA = 'ifany')

In [324]:
table(master_rating$COD_COLORE_CLASSE_T0, useNA = 'ifany')
table(master_rating[,CRA_ADJ], useNA = 'ifany')


   00    01    02    03    04  <NA> 
38284  8158  4714  1523  9741 77688 


   00    01    02    03    04  <NA> 
48346 13665  5597   499  1852 70149 

In [325]:
rbindlist(lapply(unique(master_rating$`MacroSegmento_Attuale LC`), function(seg){
    missing_val <- sum(is.na(master_rating[`MacroSegmento_Attuale LC`==seg]$CRA_ADJ))
    all_val <- nrow(master_rating[`MacroSegmento_Attuale LC`==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,52088,88630,58.77
Sme Corporate,6755,31229,21.63
Large corporate,338,3010,11.23
Corporate,677,6765,10.01
Altro,10291,10474,98.25


### analisi dati DSI

In [326]:
cols_rating <- c('NUM_PRATICA','PROGRESS_PRATICA',setdiff(cols_rating,colnames(dt_orig)))

dt_orig_rat <- merge(
    dt_orig,
    master_rating[,..cols_rating],
    by = c('NUM_PRATICA','PROGRESS_PRATICA'),
    all.x=TRUE
)

In [327]:
table(dt_orig_rat[`MacroSegmento_Attuale LC`!= 'Sme Retail' ,]$Rating_Inizio_Periodo,useNA = "always")


   I1  I1.A  I1.C  I1.D  I1.E  I1.F    I2    I3    I4    I5    I6    M1    M2 
 1006     1    24    88    10    19  1999  3137  4591  4793  4763  3220  3433 
   M3    M4   MT1   MT2   MT3   MT4   MT5   MT6   MT8   MT9    R1    R2    R3 
 3135  2638     1     1     3     4     3     2     1     2  1152   545   366 
   R4    R5  RE07  RE08  RE11  RT01  RT02  RT03  RT04  RT05  RT06  RT07  RT08 
  176   840     3     2     2     4     9     6     7    18    15    16    15 
 RT09  RT10  RT11  RT12  RT13  RT14    UR  <NA> 
   17     5     2     4     1     2  1407 13990 

In [330]:
# table(master_d4[macrosegmento_attuale_lc != 'Sme Retail',]$classe_rating,useNA = "always")

In [331]:
# master_dt_orig <- merge(
#     dt_orig_rat,
#     master_d4,
#     by.x = c('NUM_PRATICA','PROGRESS_PRATICA'),
#     by.y = c('num_pratica','progress_pratica'),
#     all.x=TRUE
# )

## CR

In [967]:
dt_cr <- readRDS('../data/RDS/dt_cr.RDS')

In [968]:
setDT(dt_cr)

In [741]:
colnames(dt_cr)

[1] "NUM_PRATICA"                 "PROGRESS_PRATICA"           
  [3] "STATO_PRATICA"               "SNDG_CEDUTO"                
  [5] "Sndg_ceduto_new"             "SNDG_CEDUTO_old"            
  [7] "Sndg_ceduto_old_adj"         "DATA_DELIBERA_O_ANNULL_FIDO"
  [9] "CR_Anno"                     "CR_Mese"                    
 [11] "CR_Giorno"                   "DATA_ISTRUTTORIA"           
 [13] "CR_Individuazione_Data"      "CR_DAT_FINE_PERIO"          
 [15] "CR_Data_CR_testo"            "CR_Giorno_CR"               
 [17] "CR_Mese_CR"                  "CR_Anno_CR"                 
 [19] "CR_COD_SNDG"                 "CR_COD_NSG"                 
 [21] "CR_Accordato_200_005"        "CR_Accordato_oper_200_005"  
 [23] "CR_Utilizzato_200_005"       "CR_IMP_GARANZ_200_005"      
 [25] "CR_IMP_GARANT_200_005"       "CR_IMP_SCONF_200_005"       
 [27] "CR_IMP_MARG_UTIL_200_005"    "CR_IMP_VALINTR_200_005"     
 [29] "CR_IMP_SALDO_MED_200_005"    "CR_IMP_ALTRI_200_005"       
 [31] "CR_Accordato_200_018"        "CR_Accordato_oper_200_018"  
 [33] "CR_Utilizzato_200_018"       "CR_IMP_GARANZ_200_018"      
 [35] "CR_IMP_GARANT_200_018"       "CR_IMP_SCONF_200_018"       
 [37] "CR_IMP_MARG_UTIL_200_018"    "CR_IMP_VALINTR_200_018"     
 [39] "CR_IMP_SALDO_MED_200_018"    "CR_IMP_ALTRI_200_018"       
 [41] "CR_Accordato_400_005"        "CR_Accordato__oper_400_005" 
 [43] "CR_Utilizzato_400_005"       "CR_IMP_GARANZ_400_005"      
 [45] "CR_IMP_GARANT_400_005"       "CR_IMP_SCONF_400_005"       
 [47] "CR_IMP_MARG_UTIL_400_005"    "CR_IMP_VALINTR_400_005"     
 [49] "CR_IMP_SALDO_MED_400_005"    "CR_IMP_ALTRI_400_005"       
 [51] "CR_Accordato_400_018"        "CR_Accordato_oper_400_018"  
 [53] "CR_Utilizzato_400_018"       "CR_IMP_GARANZ_400_018"      
 [55] "CR_IMP_GARANT_400_018"       "CR_IMP_SCONF_400_018"       
 [57] "CR_IMP_MARG_UTIL_400_018"    "CR_IMP_VALINTR_400_018"     
 [59] "CR_IMP_SALDO_MED_400_018"    "CR_IMP_ALTRI_400_018"       
 [61] "CR_Accordato_600_000"        "CR_Accordato_oper_600_000"  
 [63] "CR_Utilizzato_600_000"       "CR_IMP_GARANZ_600_000"      
 [65] "CR_IMP_GARANT_600_000"       "CR_IMP_SCONF_600_000"       
 [67] "CR_IMP_MARG_UTIL_600_000"    "CR_IMP_VALINTR_600_000"     
 [69] "CR_IMP_SALDO_MED_600_000"    "CR_IMP_ALTRI_600_000"       
 [71] "CR_Accordato_800_000"        "CR_Accordato_oper_800_000"  
 [73] "CR_Utilizzato_800_000"       "CR_IMP_GARANZ_800_000"      
 [75] "CR_IMP_GARANT_800_000"       "CR_IMP_SCONF_800_000"       
 [77] "CR_IMP_MARG_UTIL_800_000"    "CR_IMP_VALINTR_800_000"     
 [79] "CR_IMP_SALDO_MED_800_000"    "CR_IMP_ALTRI_800_000"       
 [81] "CR_Accordato_1000_000"       "CR_Accordato_oper_1000_000" 
 [83] "CR_Utilizzato_1000_000"      "CR_IMP_GARANZ_1000_000"     
 [85] "CR_IMP_GARANT_1000_000"      "CR_IMP_SCONF_1000_000"      
 [87] "CR_IMP_MARG_UTIL_1000_000"   "CR_IMP_VALINTR_1000_000"    
 [89] "CR_IMP_SALDO_MED_1000_000"   "CR_IMP_ALTRI_1000_000"      
 [91] "CR_Accordato_2200_000"       "CR_Accordato_oper_2200_000" 
 [93] "CR_Utilizzato_2200_000"      "CR_IMP_GARANZ_2200_000"     
 [95] "CR_IMP_GARANT_2200_000"      "CR_IMP_SCONF_2200_000"      
 [97] "CR_IMP_MARG_UTIL_2200_000"   "CR_IMP_VALINTR_2200_000"    
 [99] "CR_IMP_SALDO_MED_2200_000"   "CR_IMP_ALTRI_2200_000"      
[101] "CR_Accordato_2400_000"       "CR_Accordato_oper_2400_000" 
[103] "CR_Utilizzato_2400_000"      "CR_IMP_GARANZ_2400_000"     
[105] "CR_IMP_GARANT_2400_000"      "CR_IMP_SCONF_2400_000"      
[107] "CR_IMP_MARG_UTIL_2400_000"   "CR_IMP_VALINTR,_2400_000"   
[109] "CR_IMP_SALDO_MED_2400_000"   "CR_IMP_ALTRI_2400_000"      
[111] "CR_Accordato_4900_003"       "CR_Accordato_oper_4900_003" 
[113] "CR_Utilizzato_4900_003"      "CR_IMP_GARANZ_4900_003"     
[115] "CR_IMP_GARANT_4900_003"      "CR_IMP_SCONF_4900_003"      
[117] "CR_IMP_MARG_UTIL_4900_003"   "CR_IMP_VALINTR_4900_003"    
[119] "CR_IMP_SALDO_MED_4900_003"   "CR_IMP_ALTRI_4900_003"      
[121] "CR_Accordato_4900_005"       "CR_Ac

In [334]:

dt_cr[,CR_data := as.Date(paste0(CR_Anno,"-",CR_Mese,"-",CR_Giorno))]
dt_cr[,CR_delta_t := round(as.numeric(as.Date(DATA_DELIBERA_O_ANNULL_FIDO)-CR_data)/365,1)]

In [335]:
#table(as.Date(dt_cr$DATA_ISTRUTTORIA))
table(dt_cr$CR_delta_t)


    0 
74430 

In [336]:
sort(unique(paste0(dt_cr$CR_Anno,"-",dt_cr$CR_Mese)))

[1] "2015-12" "2016-1"  "2016-10" "2016-11" "2016-12" "2016-2"  "2016-3" 
 [8] "2016-4"  "2016-5"  "2016-6"  "2016-7"  "2016-8"  "2016-9"  "2017-1" 
[15] "2017-10" "2017-11" "2017-12" "2017-2"  "2017-3"  "2017-4"  "2017-5" 
[22] "2017-6"  "2017-7"  "2017-8"  "2017-9"  "2018-1"  "2018-10" "2018-11"
[29] "2018-12" "2018-2"  "2018-3"  "2018-4"  "2018-5"  "2018-6"  "2018-7" 
[36] "2018-8"  "2018-9"  "2019-1"  "2019-10" "2019-11" "2019-12" "2019-2" 
[43] "2019-3"  "2019-4"  "2019-5"  "2019-6"  "2019-7"  "2019-8"  "2019-9" 
[50] "2020-1"  "2020-10" "2020-11" "2020-12" "2020-2"  "2020-3"  "2020-4" 
[57] "2020-5"  "2020-6"  "2020-7"  "2020-8"  "2020-9"  "2021-1"  "2021-2" 
[64] "2021-3"  "2021-4"  "2021-5"  "2021-6"

In [337]:
if(!file.exists('config/config_CR_cols.R')){
    write.table(paste0("cols_cr <- c('",paste(colnames(dt_cr),collapse = "',\n'"),"')"),
            "config/config_CR_cols.R", sep=";",col.names = FALSE, row.names = FALSE,quote = FALSE)
}

source('config/config_CR_cols.R')

In [338]:
# write.xlsx(cols_cr,
#            'variabili_cr.xlsx',
#            row.names = FALSE)

In [339]:
master_cr_raw <- merge(
    dt_orig[,c('NUM_PRATICA','PROGRESS_PRATICA',
                      'MacroSegmento_Attuale LC')],
    dt_cr,
    by.x = c('NUM_PRATICA','PROGRESS_PRATICA'),
    by.y = c('NUM_PRATICA','PROGRESS_PRATICA'),
    all.x=TRUE
)

In [340]:
#saveRDS(master_cr,'../data/RDS/master_cr.RDS')

In [341]:
rbindlist(lapply(unique(master_cr_raw$`MacroSegmento_Attuale LC`), function(seg){
    missing_val <- sum(is.na(master_cr_raw[`MacroSegmento_Attuale LC`==seg]$CR_DAT_FINE_PERIO))
    all_val <- nrow(master_cr_raw[`MacroSegmento_Attuale LC`==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,69598,88630,78.53
Sme Corporate,8439,31229,27.02
Large corporate,266,3010,8.84
Corporate,1016,6765,15.02
Altro,9374,10474,89.50


In [342]:
# dt_orig_cr <- merge(
#     dt_orig,
#     master_cr[,..cols_cr],
#     by = c('NUM_PRATICA','PROGRESS_PRATICA'),
#     all.x=TRUE
# )

### Creazione indicatori

In [737]:
source('config/config_CR_cols.R')

In [738]:
cols_cr <- c('MacroSegmento_Attuale LC',cols_cr)

In [345]:
master_cr <- master_cr_raw[,..cols_cr]

In [346]:
cols_cr

[1] "MacroSegmento_Attuale LC"    "NUM_PRATICA"                
 [3] "PROGRESS_PRATICA"            "STATO_PRATICA"              
 [5] "SNDG_CEDUTO"                 "DATA_DELIBERA_O_ANNULL_FIDO"
 [7] "DATA_ISTRUTTORIA"            "CR_Accordato_200_005"       
 [9] "CR_Utilizzato_200_005"       "CR_IMP_SCONF_200_005"       
[11] "CR_Accordato_200_018"        "CR_Utilizzato_200_018"      
[13] "CR_IMP_SCONF_200_018"        "CR_Accordato_400_005"       
[15] "CR_Utilizzato_400_005"       "CR_IMP_SCONF_400_005"       
[17] "CR_Accordato_400_018"        "CR_Utilizzato_400_018"      
[19] "CR_IMP_SCONF_400_018"        "CR_Accordato_600_000"       
[21] "CR_Utilizzato_600_000"       "CR_IMP_SCONF_600_000"       
[23] "CR_Accordato_800_000"        "CR_Utilizzato_800_000"      
[25] "CR_IMP_SCONF_800_000"        "CR_Accordato_1000_000"      
[27] "CR_Utilizzato_1000_000"      "CR_IMP_SCONF_1000_000"      
[29] "CR_Accordato_2200_000"       "CR_Utilizzato_2200_000"     
[31] "CR_IMP_SCONF_2200_000"       "CR_Accordato_2400_000"      
[33] "CR_Utilizzato_2400_000"      "CR_IMP_SCONF_2400_000"      
[35] "CR_Accordato_4900_003"       "CR_Utilizzato_4900_003"     
[37] "CR_IMP_SCONF_4900_003"       "CR_Accordato_4900_005"      
[39] "CR_Utilizzato_4900_005"      "CR_IMP_SCONF_4900_005"      
[41] "CR_Accordato_4900_018"       "CR_Utilizzato_4900_018"     
[43] "CR_IMP_SCONF_4900_018"       "CR_Accordato_4901_003"      
[45] "CR_Utilizzato_4901_003"      "CR_IMP_SCONF_4901_003"      
[47] "CR_Accordato_4901_005"       "CR_Utilizzato_4901_005"     
[49] "CR_IMP_SCONF_4901_005"       "CR_Accordato_4901_018"      
[51] "CR_Utilizzato_4901_018"      "CR_IMP_SCONF_4901_018"      
[53] "CR_Accordato_4902_003"       "CR_Utilizzato_4902_003"     
[55] "CR_IMP_SCONF_4902_003"       "CR_Accordato_4902_005"      
[57] "CR_Utilizzato_4902_005"      "CR_IMP_SCONF_4902_005"      
[59] "CR_Accordato_4902_018"       "CR_Utilizzato_4902_018"     
[61] "CR_IMP_SCONF_4902_018"

In [347]:

master_cr[,accordato := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_acc]

master_cr[,sconfino := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_sconf]
master_cr[,utilizzato := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_uti]

master_cr[,accordato_revoca := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_acc_revoca]
master_cr[,sconfino_revoca := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_sconf_revoca]
master_cr[,utilizzato_revoca := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_uti_revoca]

master_cr[,accordato_autoliq := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_acc_autoliq]
master_cr[,sconfino_autoliq := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_sconf_autoliq]
master_cr[,utilizzato_autoliq := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_uti_autoliq]

master_cr[,accordato_scadenza := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_acc_scadenza]
master_cr[,sconfino_scadenza := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_sconf_scadenza]
master_cr[,utilizzato_scadenza := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_uti_scadenza]

master_cr[,accordato_bt := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_acc_bt]
master_cr[,sconfino_bt := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_sconf_bt]
master_cr[,utilizzato_bt := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_uti_bt]

master_cr[,accordato_mlt := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_acc_mlt]
master_cr[,sconfino_mlt := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_sconf_mlt]
master_cr[,utilizzato_mlt := ifelse(is.finite(do.call(pmin, c(.SD, na.rm=TRUE))),rowSums(.SD, na.rm=T),NA), .SD = cols_uti_mlt]



master_cr[, sconf_acc := sconfino/accordato]
master_cr[, sconf_acc_bt := sconfino_bt/accordato_bt]
master_cr[, sconf_acc_mlt := sconfino_mlt/accordato_mlt]

master_cr[, uti_acc := utilizzato/accordato]
master_cr[, uti_acc_bt := utilizzato_bt/accordato_bt]
master_cr[, uti_acc_mlt := utilizzato_mlt/accordato_mlt]

master_cr[, uti_acc_revoca := utilizzato_revoca/accordato_revoca]
master_cr[, uti_acc_autoliq := utilizzato_autoliq/accordato_autoliq]
master_cr[, uti_acc_scadenza := utilizzato_scadenza/accordato_scadenza]

In [348]:
range(master_cr$accordato, na.rm = TRUE)

[1]           0 18673481737

In [349]:
cols_2keep_cr <- c('NUM_PRATICA','PROGRESS_PRATICA',setdiff(colnames(master_cr),cols_cr))

cols_2keep_cr

[1] "NUM_PRATICA"         "PROGRESS_PRATICA"    "accordato"          
 [4] "sconfino"            "utilizzato"          "accordato_revoca"   
 [7] "sconfino_revoca"     "utilizzato_revoca"   "accordato_autoliq"  
[10] "sconfino_autoliq"    "utilizzato_autoliq"  "accordato_scadenza" 
[13] "sconfino_scadenza"   "utilizzato_scadenza" "accordato_bt"       
[16] "sconfino_bt"         "utilizzato_bt"       "accordato_mlt"      
[19] "sconfino_mlt"        "utilizzato_mlt"      "sconf_acc"          
[22] "sconf_acc_bt"        "sconf_acc_mlt"       "uti_acc"            
[25] "uti_acc_bt"          "uti_acc_mlt"         "uti_acc_revoca"     
[28] "uti_acc_autoliq"     "uti_acc_scadenza"

In [350]:
master_cr_sme <- master_cr[`MacroSegmento_Attuale LC` == "Sme Retail",..cols_2keep_cr]
colnames(master_cr_sme)<-c('NUM_PRATICA','PROGRESS_PRATICA',
                           paste0("CR_",setdiff(colnames(master_cr),cols_cr)))

master_cr_nonsme <- master_cr[`MacroSegmento_Attuale LC` != "Sme Retail"& `MacroSegmento_Attuale LC` != "Altro",..cols_2keep_cr]
colnames(master_cr_nonsme)<-c('NUM_PRATICA','PROGRESS_PRATICA',
                                           paste0("CR_",setdiff(colnames(master_cr),cols_cr)))

saveRDS(master_cr_sme,'../data/RDS/master/sme/master_cr.RDS')
saveRDS(master_cr_nonsme,'../data/RDS/master/non_sme/master_cr.RDS')

In [975]:



options(scipen=999)
master_cr[NUM_PRATICA == '81112083108',sconf_acc]
master_cr[NUM_PRATICA == '90308230159',uti_acc_revoca]




[1] 0.00005148138

[1] 3.2926

## ATECO

In [744]:
mapping_ateco <- read.xlsx('../mapping ateco.xlsx',sheetName = 'Mapping ATECO')

In [792]:
saveRDS(mapping_ateco,'../data/RDS/mapping_ateco.RDS')

In [791]:
sort(unique(mapping_ateco$MICRO.PROMETEIA))

[1] "AGRI"   "ESTR1"  "ESTR2"  "ESTR3"  "N1013"  "N101A"  "N102"   "N103"  
  [9] "N104"   "N105"   "N106"   "N1073"  "N107A"  "N1082"  "N1083"  "N108R" 
 [17] "N109"   "N1107"  "N110A"  "N110B"  "N110C"  "N120"   "N133"   "N139"  
 [25] "N13A"   "N141"   "N142"   "N1431"  "N1439"  "N1511"  "N1512"  "N152"  
 [33] "N161"   "N162"   "N171"   "N172"   "N181"   "N182"   "N191"   "N192"  
 [41] "N201"   "N202"   "N203"   "N204"   "N205"   "N206"   "N211"   "N212"  
 [49] "N2211"  "N2219"  "N222"   "N2314"  "N231A"  "N232"   "N2331"  "N2332" 
 [57] "N234"   "N2351"  "N2352"  "N236"   "N237"   "N239"   "N241"   "N242"  
 [65] "N243"   "N244"   "N245"   "N251"   "N252"   "N253"   "N254"   "N255"  
 [73] "N256"   "N257"   "N259"   "N261"   "N262"   "N263"   "N264"   "N265"  
 [81] "N266"   "N267"   "N268"   "N271"   "N272"   "N273"   "N274"   "N2751C"
 [89] "N2752"  "N279"   "N2811"  "N2814"  "N2815"  "N281A"  "N2822"  "N282A" 
 [97] "N283"   "N2841"  "N2849"  "N2891"  "N2892"  "N2893"  "N2894"  "N2895" 
[105] "N2896"  "N2899"  "N2912"  "N292"   "N293"   "N301"   "N302"   "N303"  
[113] "N304"   "N3091"  "N309A"  "N3101"  "N3102"  "N31091" "N310A"  "N321"  
[121] "N322"   "N323"   "N324"   "N325"   "N329"   "N330"   "N351"   "N35X"  
[129] "N38X"   "N410"   "N420"   "N430"   "N450"   "N461"   "N462"   "N463"  
[137] "N4646"  "N464A"  "N464B"  "N464C"  "N467"   "N469"   "N46X"   "N4711" 
[145] "N4719"  "N472A"  "N473"   "N4759"  "N475A"  "N476"   "N477A"  "N477B" 
[153] "N477C"  "N478"   "N479"   "N4799"  "N47A"   "N493"   "N494"   "N49A"  
[161] "N521"   "N522"   "N531"   "N532"   "N550"   "N560"   "N581A"  "N581B" 
[169] "N582"   "N590"   "N600"   "N610"   "N620"   "N683"   "N68A"   "N730"  
[177] "N790"   "NATR"   "NHOL"   "NSER1"  "NSER2"  "NSER5"  "NSER6"  "NSER8" 
[185] "NSFIN"  "NSRR"

In [766]:
table(dt_ateco$DAT_COSTITUZ)


1600-01-01 1799-12-30 1800-01-01 1856-03-13 1856-04-26 1861-01-01 1861-07-30 
      6482          2         35          2          1          1          1 
1865-02-23 1870-04-27 1877-03-07 1877-03-30 1880-04-07 1889-01-03 1889-05-27 
         1          2          3          2          4          1          2 
1889-05-30 1890-01-26 1891-11-01 1892-10-26 1892-11-05 1893-01-01 1893-07-11 
         1          2          4          1          1          1          1 
1893-08-10 1893-11-30 1895-08-23 1896-10-31 1897-01-01 1898-02-28 1898-11-10 
         1          2          3          3          1          2          1 
1898-12-14 1899-01-01 1899-06-04 1899-11-23 1900-01-01 1900-01-02 1900-01-10 
         1          1          2          3        315         68          1 
1900-02-01 1900-02-05 1900-03-17 1900-11-15 1901-01-01 1901-02-02 1901-04-06 
         4          3          7          1         36          1          7 
1901-05-31 1901-07-14 1902-02-22 1902-10-30 1904-07-08 1904-10-

In [855]:
dt_ateco <- readRDS('../data/RDS/dt_ateco.RDS')

In [856]:
colnames((mapping_ateco))

[1] "progressivo"                                       
 [2] "ATECO.fittizio"                                    
 [3] "ATECO"                                             
 [4] "ATECO.6.cifre"                                     
 [5] "Ateco_NUM"                                         
 [6] "Descrizione"                                       
 [7] "SETTORE_ISTAT"                                     
 [8] "Settore...ENG"                                     
 [9] "Microsettore...ENG"                                
[10] "Settore...ITA"                                     
[11] "Microsettore...ITA"                                
[12] "Outlook.Studi"                                     
[13] "Applicazione.settore.marketing.CIB"                
[14] "MICRO.PROMETEIA"                                   
[15] "EW_CRA"                                            
[16] "Settore.sensibile.ESG"                             
[17] "Settore.climate.sensitive"                         
[18] "Attrattività.BdT.Imprese...Strategie.Creditizie"   
[19] "Attrattività.BdT.Az..Retail...Strategie.Creditizie"
[20] "Data.Attrattività"                                 
[21] "Nace.liv2"                                         
[22] "Nace.2.cifre"                                      
[23] "Livello.rischio..Estero..LUGLIO.2021"              
[24] "Codice.settore"                                    
[25] "Codice.microsettore"

In [857]:
dt_ateco<-merge(dt_ateco,
                    mapping_ateco[,c('ATECO','Codice.settore','Codice.microsettore','Settore...ITA','Microsettore...ITA','MICRO.PROMETEIA')],
                    by.x = 'COD_ATECO',
                    by.y = 'ATECO',
                    all.x = TRUE
               )

In [858]:
dt_ateco[, SNDG_CEDENTE := as.numeric(COD_SNDG)]

In [859]:
nrow(dt_ateco)

[1] 109763

In [860]:
colnames(dt_ateco)

[1] "COD_ATECO"           "COD_SNDG"            "DAT_COSTITUZ"       
 [4] "FLG_PREV_ADG"        "Codice.settore"      "Codice.microsettore"
 [7] "Settore...ITA"       "Microsettore...ITA"  "MICRO.PROMETEIA"    
[10] "SNDG_CEDENTE"

In [861]:
setnames(dt_ateco,
         c('Codice.settore','Codice.microsettore','Settore...ITA','Microsettore...ITA'),
         c('AT_cod_sett','AT_cod_microsett','AT_sett_desc','AT_microsett_desc'))

In [862]:
table(dt_ateco$AT_cod_sett)


ABCON00 AEROS00 AGRIC00 ALIMD00 ALIMP00 APMED00 ASERV00 ASSIC00 ATRIC00 AUTOD00 
    204       6    1335    3370    2465     217    6448     505     458    3205 
AUTOP00 CHIMI00 COMPO00 COSTR00 DETTA00 DIFES00 DURED00 DUREP00 ESTRA00 ESTRC00 
    338    2454     580   12166    5431      29    1460     815      69       1 
FARMD00 FARMP00 FINSE00   GDO00 GIOAZ00 HARDW00 HOLDI00 IMMOB00 INFRA00 INGRO00 
   2874     108     438    1008     883     372     216     965     628    5469 
ISTRU00 LEGNO00 MACCH00 MAIMG00 MAIMS00 MATCO00 MEDIA00 METAL00 OILGA00 POWER00 
    415    1031    1648    1211     765     710     553     706    3216     122 
PRODM00 PUBAM00 RACTR00 SERVA00 SILVI00 SOFTW00 TABAC00 TELEC00 TESSD00 TESSP00 
   3711      92     779     642      89    1447   10021     172    1060    1959 
TRASP00 TRASS00 TURAL00 TURRI00 UTILI00 
    101   11383     636   10388     188 

In [863]:
dt_ateco_cedente <- unique(merge(dt_orig[,c('NUM_PRATICA','PROGRESS_PRATICA','SNDG_CEDENTE')],
                 dt_ateco,
                 by.x = 'SNDG_CEDENTE',
                 by.y = 'SNDG_CEDENTE',
                 all.x = TRUE))

In [864]:
dt_ateco <- unique(merge(dt_orig[,c('SNDG_CEDUTO','DATA_DELIBERA_O_ANNULL_FIDO',
                'NUM_PRATICA','PROGRESS_PRATICA','MacroSegmento_Attuale LC')],
                 dt_ateco,
                 by.x = 'SNDG_CEDUTO',
                 by.y = 'COD_SNDG',
                 all.x = TRUE))

In [865]:
colnames(dt_ateco)

[1] "SNDG_CEDUTO"                 "DATA_DELIBERA_O_ANNULL_FIDO"
 [3] "NUM_PRATICA"                 "PROGRESS_PRATICA"           
 [5] "MacroSegmento_Attuale LC"    "COD_ATECO"                  
 [7] "DAT_COSTITUZ"                "FLG_PREV_ADG"               
 [9] "AT_cod_sett"                 "AT_cod_microsett"           
[11] "AT_sett_desc"                "AT_microsett_desc"          
[13] "MICRO.PROMETEIA"             "SNDG_CEDENTE"

In [866]:
setnames(dt_ateco_cedente,
         c('AT_cod_sett','AT_cod_microsett','AT_sett_desc','AT_microsett_desc'),
         paste0(c('AT_cod_sett','AT_cod_microsett','AT_sett_desc','AT_microsett_desc'),"_cedente"))
dt_ateco_cedente

SNDG_CEDENTE,NUM_PRATICA,PROGRESS_PRATICA,COD_ATECO,COD_SNDG,DAT_COSTITUZ,FLG_PREV_ADG,AT_cod_sett_cedente,AT_cod_microsett_cedente,AT_sett_desc_cedente,AT_microsett_desc_cedente,MICRO.PROMETEIA
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
125816,80328230315,1,NA,NA,NA,NA,NA,NA,NA,NA,NA
125816,80328230316,1,NA,NA,NA,NA,NA,NA,NA,NA,NA
125816,80328230317,1,NA,NA,NA,NA,NA,NA,NA,NA,NA
125816,80329231432,1,NA,NA,NA,NA,NA,NA,NA,NA,NA
125816,80329231433,1,NA,NA,NA,NA,NA,NA,NA,NA,NA
125816,80329231434,1,NA,NA,NA,NA,NA,NA,NA,NA,NA
125816,80403230274,1,NA,NA,NA,NA,NA,NA,NA,NA,NA
125816,80404230237,2,NA,NA,NA,NA,NA,NA,NA,NA,NA
125816,80406230182,1,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [867]:
dt_ateco <- merge(dt_ateco,
                    dt_ateco_cedente[,
                    c('NUM_PRATICA','PROGRESS_PRATICA','AT_cod_sett_cedente','AT_cod_microsett_cedente',
                     'AT_sett_desc_cedente','AT_microsett_desc_cedente')],
                    by = c('NUM_PRATICA','PROGRESS_PRATICA'),
                    all.x = TRUE)

In [868]:
colnames(dt_ateco)
nrow(unique(dt_ateco))

[1] "NUM_PRATICA"                 "PROGRESS_PRATICA"           
 [3] "SNDG_CEDUTO"                 "DATA_DELIBERA_O_ANNULL_FIDO"
 [5] "MacroSegmento_Attuale LC"    "COD_ATECO"                  
 [7] "DAT_COSTITUZ"                "FLG_PREV_ADG"               
 [9] "AT_cod_sett"                 "AT_cod_microsett"           
[11] "AT_sett_desc"                "AT_microsett_desc"          
[13] "MICRO.PROMETEIA"             "SNDG_CEDENTE"               
[15] "AT_cod_sett_cedente"         "AT_cod_microsett_cedente"   
[17] "AT_sett_desc_cedente"        "AT_microsett_desc_cedente"

[1] 140108

In [869]:
dt_ateco[,AT_eta_soc := round(as.numeric(difftime(as.Date(DATA_DELIBERA_O_ANNULL_FIDO),as.Date(DAT_COSTITUZ),units = 'days')/365),2)]


In [870]:
saveRDS(dt_ateco,'../data/RDS/master/master_ateco.RDS' )

In [837]:
#setnames(dt_ateco,'COD_ATECO','AT_COD_ATECO')
cols_2keep_ateco <- c('NUM_PRATICA','PROGRESS_PRATICA','AT_eta_soc','AT_cod_sett','AT_cod_microsett','AT_cod_sett_cedente','AT_cod_microsett_cedente','AT_sett_desc','AT_microsett_desc','AT_sett_desc_cedente','AT_microsett_desc_cedente','MICRO.PROMETEIA')

In [763]:
master_ateco_sme <- dt_ateco[`MacroSegmento_Attuale LC` == "Sme Retail", ..cols_2keep_ateco]
master_ateco_nonsme <- dt_ateco[`MacroSegmento_Attuale LC` != "Sme Retail" & `MacroSegmento_Attuale LC` != "Altro" , ..cols_2keep_ateco]

In [764]:
saveRDS(master_ateco_sme,'../data/RDS/master/sme/master_ateco.RDS')
saveRDS(master_ateco_nonsme,'../data/RDS/master/non_sme/master_ateco.RDS')

## BILANCIO

### Aggancio 

In [372]:
dt_bil_orig <- readRDS('../data/RDS/bilancio_completo.RDS')

In [373]:
tab_cod_cartipo <- dt_bil_orig[,list(COD_CARTIPO_DEF = max(COD_CARTIPO),COD_CARTIPO=COD_CARTIPO), by= c("COD_SNDG","DAT_BIL")]

In [374]:
cols_bil_2keep <- colnames(dt_bil_orig)

cols_bil_val <- c('COD_SNDG','DAT_BIL','DAT_ULTRICL',"COD_CARTIPO",'COD_STRUTTBIL',cols_bil_2keep[grepl("VAL_VOCE",cols_bil_2keep)])
#cols_bil_val

In [375]:
dt_bil <- merge(dt_bil_orig,tab_cod_cartipo, all.x = TRUE, by= c("COD_SNDG","DAT_BIL",'COD_CARTIPO'))

dt_bil <- unique(dt_bil[COD_CARTIPO_DEF == COD_CARTIPO,])

In [376]:
tab_DAT_ULTRICL <- dt_bil[,list(DAT_ULTRICL_DEF = max(DAT_ULTRICL),DAT_ULTRICL=DAT_ULTRICL), by= c("COD_SNDG","DAT_BIL")]

dt_bil <- merge(dt_bil,tab_DAT_ULTRICL, all.x = TRUE, by= c("COD_SNDG","DAT_BIL",'DAT_ULTRICL'))

In [377]:
dt_bil <- unique(dt_bil[DAT_ULTRICL_DEF == DAT_ULTRICL,])
sndg_2rm <- dt_bil[,list(num = .N), by = c("COD_SNDG","DAT_BIL")][num>1,]$COD_SNDG

In [378]:
options(
    #repr.matrix.max.rows=20, 
    repr.matrix.max.cols=20
)

In [379]:

dt_check <- merge(dt_bil,
                  dt_bil[,list(num = .N), by = c("COD_SNDG","DAT_BIL")],
                  by = c("COD_SNDG","DAT_BIL"))
unique(dt_check[num>1,colnames(dt_check)[!colnames(dt_check) %in% c('VAL_VOCE_838','VAL_VOCE_837','DAT_FINE_PERIO')],with = FALSE])

COD_SNDG,DAT_BIL,DAT_ULTRICL,COD_CARTIPO,COD_STRUTTBIL,VAL_DURLEGBIL,COD_NATBIL,COD_STATOBIL,COD_MODRIL,COD_TP_SCHEMA,⋯,VAL_VOCE_925,VAL_VOCE_926,VAL_VOCE_927,VAL_VOCE_928,VAL_VOCE_929,VAL_VOCE_930,VAL_VOCE_931,COD_CARTIPO_DEF,DAT_ULTRICL_DEF,num
<chr>,<date>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<date>,<int>
0000000000056353,2017-02-28,2018-10-16,100,02,12,0,33,E,8,⋯,0,0,0,0,0,34,0,100,2018-10-16,2
0000000000056353,2018-02-28,2018-10-16,100,02,12,0,33,E,8,⋯,0,0,0,0,0,-6,0,100,2018-10-16,2
0000000000609077,2017-12-31,2018-08-10,100,01,12,7,22,A,8,⋯,0,0,0,0,0,0,0,100,2018-08-10,2
0000000000810204,2017-12-31,2018-10-03,100,02,12,0,22,E,6,⋯,1,0,3,0,18,0,0,100,2018-10-03,2
0000000000928090,2017-12-31,2018-12-20,100,04,12,0,22,E,6,⋯,1,3,0,0,5,0,0,100,2018-12-20,2
0000000000940504,2017-12-31,2018-11-16,100,01,12,0,22,E,6,⋯,0,0,0,0,0,0,0,100,2018-11-16,2
0000000000967128,2017-12-31,2018-11-28,100,01,12,0,22,E,6,⋯,0,0,0,0,64,0,0,100,2018-11-28,2
0000000000981972,2017-12-31,2018-09-05,100,01,12,0,33,E,7,⋯,0,0,0,0,0,0,0,100,2018-09-05,2
0000000001025288,2017-12-31,2018-06-21,100,02,12,0,22,E,6,⋯,0,0,0,0,0,0,0,100,2018-06-21,2


In [380]:
table(dt_check[num>1,]$VAL_VOCE_838,useNA = "always")
table(dt_check[num>1,]$VAL_VOCE_837,useNA = "always")


    0  <NA> 
30348 30348 


    0  <NA> 
30348 30348 

In [381]:
#### Considero solo l'ultimo dato disponibile
tab_DAT_FINE_PERIO <- dt_bil[,list(DAT_FINE_PERIO_DEF = max(DAT_FINE_PERIO),DAT_FINE_PERIO=DAT_FINE_PERIO), by= c("COD_SNDG","DAT_BIL")]

dt_bil <- merge(dt_bil,tab_DAT_FINE_PERIO, all.x = TRUE, by= c("COD_SNDG","DAT_BIL",'DAT_FINE_PERIO'))

In [382]:
dt_bil <- unique(dt_bil[DAT_FINE_PERIO_DEF == DAT_FINE_PERIO,])
dt_bil<- unique(dt_bil[,..cols_bil_val])

In [383]:
head(dt_bil,1)

COD_SNDG,DAT_BIL,DAT_ULTRICL,COD_CARTIPO,COD_STRUTTBIL,VAL_VOCE_301,VAL_VOCE_302,VAL_VOCE_303,VAL_VOCE_304,VAL_VOCE_305,⋯,VAL_VOCE_924,VAL_VOCE_925,VAL_VOCE_926,VAL_VOCE_927,VAL_VOCE_928,VAL_VOCE_929,VAL_VOCE_930,VAL_VOCE_931,VAL_VOCE_837,VAL_VOCE_838
<chr>,<date>,<date>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0000000000012113,2015-12-31,2016-03-31,100,04,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,NA,NA


In [384]:
sub_dati_bil <- dt_orig[,
                    c('NUM_PRATICA','PROGRESS_PRATICA','SNDG_CEDUTO','DATA_DELIBERA_O_ANNULL_FIDO',
                      'MacroSegmento_Attuale LC')]

sub_dati_bil[month(DATA_DELIBERA_O_ANNULL_FIDO)<=8,
             data_bil := as.Date(paste0(year(DATA_DELIBERA_O_ANNULL_FIDO)-2,"-12-31")) ]
sub_dati_bil[month(DATA_DELIBERA_O_ANNULL_FIDO)>8,
             data_bil := as.Date(paste0(year(DATA_DELIBERA_O_ANNULL_FIDO)-1,"-12-31")) ]



In [385]:
data_tot_bil <-merge(
    sub_dati_bil,
    dt_bil,
    by.x=c("SNDG_CEDUTO"),
    by.y=c("COD_SNDG"),
    allow.cartesian = TRUE,
    all.x=TRUE
)

In [386]:
bil_max<-data_tot_bil[DAT_BIL<=data_bil,
                      .(dat_bil_max=max(
                          DAT_BIL,na.rm=TRUE)),
                         by=c("SNDG_CEDUTO",'DATA_DELIBERA_O_ANNULL_FIDO')]

data_merge_2_bil<-unique(merge(
    sub_dati_bil,
    bil_max,
    by.x=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    by.y=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    all.x=TRUE
))

In [387]:
nrow(dt_bil[is.na(DAT_ULTRICL),])
#master_bil_raw

[1] 0

In [388]:
master_bil_raw <- unique(merge(
    data_merge_2_bil,
    dt_bil,
    by.x=c("SNDG_CEDUTO","dat_bil_max"),
    by.y=c("COD_SNDG","DAT_BIL"),
    all.x=TRUE
))
master_bil_raw[,BIL_delta_t := round(as.numeric(as.Date(DATA_DELIBERA_O_ANNULL_FIDO) -dat_bil_max )/365,0) ]

In [389]:
table(master_bil_raw$BIL_delta_t,useNA = 'always')
nrow(master_bil_raw[is.na(DAT_ULTRICL)])


    1     2     3     4     5     6  <NA> 
53882 11900   726   281    82    10 73227 

[1] 73227

In [390]:
master_bil_raw[,flag_t:= BIL_delta_t]
master_bil_raw[flag_t >3,flag_t:= NA]
table(master_bil_raw$flag_t,useNA = 'always')


    1     2     3  <NA> 
53882 11900   726 73600 

In [391]:
colnames(master_bil_raw)

[1] "SNDG_CEDUTO"                 "dat_bil_max"                
  [3] "DATA_DELIBERA_O_ANNULL_FIDO" "NUM_PRATICA"                
  [5] "PROGRESS_PRATICA"            "MacroSegmento_Attuale LC"   
  [7] "data_bil"                    "DAT_ULTRICL"                
  [9] "COD_CARTIPO"                 "COD_STRUTTBIL"              
 [11] "VAL_VOCE_301"                "VAL_VOCE_302"               
 [13] "VAL_VOCE_303"                "VAL_VOCE_304"               
 [15] "VAL_VOCE_305"                "VAL_VOCE_306"               
 [17] "VAL_VOCE_307"                "VAL_VOCE_308"               
 [19] "VAL_VOCE_309"                "VAL_VOCE_310"               
 [21] "VAL_VOCE_311"                "VAL_VOCE_312"               
 [23] "VAL_VOCE_313"                "VAL_VOCE_314"               
 [25] "VAL_VOCE_315"                "VAL_VOCE_316"               
 [27] "VAL_VOCE_317"                "VAL_VOCE_318"               
 [29] "VAL_VOCE_319"                "VAL_VOCE_320"               
 [31] "VAL_VOCE_321"                "VAL_VOCE_322"               
 [33] "VAL_VOCE_323"                "VAL_VOCE_324"               
 [35] "VAL_VOCE_325"                "VAL_VOCE_326"               
 [37] "VAL_VOCE_327"                "VAL_VOCE_328"               
 [39] "VAL_VOCE_329"                "VAL_VOCE_330"               
 [41] "VAL_VOCE_331"                "VAL_VOCE_332"               
 [43] "VAL_VOCE_333"                "VAL_VOCE_334"               
 [45] "VAL_VOCE_335"                "VAL_VOCE_336"               
 [47] "VAL_VOCE_337"                "VAL_VOCE_338"               
 [49] "VAL_VOCE_401"                "VAL_VOCE_402"               
 [51] "VAL_VOCE_403"                "VAL_VOCE_404"               
 [53] "VAL_VOCE_405"                "VAL_VOCE_406"               
 [55] "VAL_VOCE_407"                "VAL_VOCE_408"               
 [57] "VAL_VOCE_409"                "VAL_VOCE_410"               
 [59] "VAL_VOCE_411"                "VAL_VOCE_412"               
 [61] "VAL_VOCE_413"                "VAL_VOCE_414"               
 [63] "VAL_VOCE_415"                "VAL_VOCE_416"               
 [65] "VAL_VOCE_417"                "VAL_VOCE_418"               
 [67] "VAL_VOCE_419"                "VAL_VOCE_420"               
 [69] "VAL_VOCE_421"                "VAL_VOCE_422"               
 [71] "VAL_VOCE_423"                "VAL_VOCE_424"               
 [73] "VAL_VOCE_425"                "VAL_VOCE_426"               
 [75] "VAL_VOCE_427"                "VAL_VOCE_428"               
 [77] "VAL_VOCE_429"                "VAL_VOCE_430"               
 [79] "VAL_VOCE_431"                "VAL_VOCE_432"               
 [81] "VAL_VOCE_433"                "VAL_VOCE_434"               
 [83] "VAL_VOCE_435"                "VAL_VOCE_436"               
 [85] "VAL_VOCE_437"                "VAL_VOCE_438"               
 [87] "VAL_VOCE_439"                "VAL_VOCE_440"               
 [89] "VAL_VOCE_441"                "VAL_VOCE_442"               
 [91] "VAL_VOCE_501"                "VAL_VOCE_502"               
 [93] "VAL_VOCE_503"                "VAL_VOCE_504"               
 [95] "VAL_VOCE_505"                "VAL_VOCE_506"               
 [97] "VAL_VOCE_507"                "VAL_VOCE_508"               
 [99] "VAL_VOCE_509"                "VAL_VOCE_510"               
[101] "VAL_VOCE_511"                "VAL_VOCE_512"               
[103] "VAL_VOCE_513"                "VAL_VOCE_514"               
[105] "VAL_VOCE_515"                "VAL_VOCE_516"               
[107] "VAL_VOCE_517"                "VAL_VOCE_518"               
[109] "VAL_VOCE_519"                "VAL_VOCE_520"               
[111] "VAL_VOCE_521"                "VAL_VOCE_522"               
[113] "VAL_VOCE_523"                "VAL_VOCE_524"               
[115] "VAL_VOCE_525"                "VAL_VOCE_526"               
[117] "VAL_VOCE_527"                "VAL_VOCE_528"               
[119] "VAL_VOCE_529"                "VAL_VOCE_530"               
[121] "VAL_VOCE_531"                "VAL_V

In [392]:
table(master_bil_raw$COD_STRUTTBIL)


   01    02    03    04    05    06    07    08    09 
22790 18832  4921 19464   604    46     7   206    11 

In [393]:
#dt_bil[COD_SNDG =='0000000000069542',]

In [394]:
#master_bil_raw[is.na(DAT_ULTRICL),]#[SNDG_CEDUTO =='0000000000056353',]

In [395]:
#data_merge_2_bil

rbindlist(lapply(unique(master_bil_raw$`MacroSegmento_Attuale LC`), function(seg){
    missing_val <- sum(is.na(master_bil_raw[`MacroSegmento_Attuale LC`==seg]$DAT_ULTRICL))
    all_val <- nrow(master_bil_raw[`MacroSegmento_Attuale LC`==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,61989,88630,69.94
Sme Corporate,1922,31229,6.15
Altro,8732,10474,83.37
Corporate,448,6765,6.62
Large corporate,136,3010,4.52


In [396]:
#data_merge_2_bil

rbindlist(lapply(unique(master_bil_raw$`MacroSegmento_Attuale LC`), function(seg){
    missing_val <- sum(is.na(master_bil_raw[`MacroSegmento_Attuale LC`==seg]$flag_t))
    all_val <- nrow(master_bil_raw[`MacroSegmento_Attuale LC`==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,62319,88630,70.31
Sme Corporate,1940,31229,6.21
Altro,8738,10474,83.43
Corporate,465,6765,6.87
Large corporate,138,3010,4.58


In [397]:
saveRDS(master_bil_raw,'../data/RDS/master_raw_bil.RDS')

### creazione indicatori

In [398]:
#mapping_nomi <- setDT(read.xlsx('../mapping_voci_cebi.xlsx',sheetName = 'nuovi_indicatori'))

In [399]:
master_bil <- copy(master_bil_raw)
colnames(master_bil)[1:12]


[1] "SNDG_CEDUTO"                 "dat_bil_max"                
 [3] "DATA_DELIBERA_O_ANNULL_FIDO" "NUM_PRATICA"                
 [5] "PROGRESS_PRATICA"            "MacroSegmento_Attuale LC"   
 [7] "data_bil"                    "DAT_ULTRICL"                
 [9] "COD_CARTIPO"                 "COD_STRUTTBIL"              
[11] "VAL_VOCE_301"                "VAL_VOCE_302"

In [400]:
# mapping_nomi_100 <- mapping_nomi[cod_cartipo == 100,]
# mapping_nomi_104 <- mapping_nomi[cod_cartipo == 104,]

In [401]:

master_bil[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 100, 
           onerifin_ebitda := VAL_VOCE_623/VAL_VOCE_644]
master_bil[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 104, 
           onerifin_ebitda := VAL_VOCE_624/VAL_VOCE_645]

master_bil[COD_STRUTTBIL %in% c(paste0('0',5)) & COD_CARTIPO == 100, 
           onerifin_ebitda := VAL_VOCE_621/VAL_VOCE_641]
master_bil[COD_STRUTTBIL %in% c(paste0('0',5)) & COD_CARTIPO == 104, 
           onerifin_ebitda := VAL_VOCE_621/VAL_VOCE_642]

master_bil[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 100, 
           fatturato := max(VAL_VOCE_605,VAL_VOCE_601,na.rm = TRUE)]

master_bil[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 104, 
           fatturato := max(VAL_VOCE_605,VAL_VOCE_601,na.rm = TRUE)]

In [402]:



# master_bil[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 100, 
#            fatturato := do.call(pmax, .SD), .SDcols = c('VAL_VOCE_605','VAL_VOCE_601')]

# master_bil[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 100, 
#            fatturato := max(VAL_VOCE_605,VAL_VOCE_601,na.omit=TRUE)]
# master_bil[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 104, 
#            fatturato := do.call(pmax, .SD), .SDcols = c('VAL_VOCE_605','VAL_VOCE_601')]

master_bil[COD_STRUTTBIL %in% c(paste0('0',5)) & COD_CARTIPO == 100, 
           fatturato := VAL_VOCE_601]
master_bil[COD_STRUTTBIL %in% c(paste0('0',5)) & COD_CARTIPO == 104, 
           fatturato := VAL_VOCE_601]

master_bil[COD_STRUTTBIL %in% c(paste0('0',1:5)) & COD_CARTIPO == 100, 
              pat_netto := VAL_VOCE_415]
master_bil[COD_STRUTTBIL %in% c(paste0('0',1:5)) & COD_CARTIPO == 104, 
          pat_netto := VAL_VOCE_413]

master_bil[COD_STRUTTBIL %in% c(paste0('0',6:8)) & COD_CARTIPO == 100, 
         pat_netto := VAL_VOCE_431]
master_bil[COD_STRUTTBIL %in% c(paste0('0',6:8)) & COD_CARTIPO == 104, 
         pat_netto := VAL_VOCE_432]

master_bil[COD_STRUTTBIL %in% c(paste0('0',9)) , 
        pat_netto := VAL_VOCE_432]

In [403]:
master_bil[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 100, 
           PFN_Ebitda := VAL_VOCE_832/VAL_VOCE_644]

master_bil[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 104, 
           PFN_Ebitda := VAL_VOCE_836/VAL_VOCE_645]


master_bil[COD_STRUTTBIL %in% c(paste0('0',5)) & COD_CARTIPO == 100, 
           PFN_Ebitda := VAL_VOCE_832/VAL_VOCE_641]
master_bil[COD_STRUTTBIL %in% c(paste0('0',5)) & COD_CARTIPO == 104, 
           PFN_Ebitda := VAL_VOCE_836/VAL_VOCE_642]

In [404]:
str(master_bil$PFN_Ebitda)

 num [1:140108] NA NA NA NA NA NA NA 0 NA NA ...
 - attr(*, "label")= chr "Strutture Patrimoniali Di Sintesi"


In [405]:
sub_dati_bil_diff <- master_bil[, c('SNDG_CEDUTO','dat_bil_max','DATA_DELIBERA_O_ANNULL_FIDO','NUM_PRATICA','PROGRESS_PRATICA')]

In [406]:
sub_dati_bil_diff[,data_diff := ifelse(!is.na(dat_bil_max),paste0(year(as.Date(dat_bil_max))-1,"-12-31"),NA)]
sub_dati_bil_diff[,data_diff := as.Date(data_diff)]


In [407]:
data_tot_bil_diff <-merge(
    sub_dati_bil_diff,
    dt_bil,
    by.x=c("SNDG_CEDUTO"),
    by.y=c("COD_SNDG"),
    allow.cartesian = TRUE,
    all.x=TRUE
)

In [408]:
bil_diff<-data_tot_bil_diff[DAT_BIL<=data_diff,
                      .(dat_bil_diff=max(
                          DAT_BIL,na.rm=TRUE)),
                         by=c("SNDG_CEDUTO",'DATA_DELIBERA_O_ANNULL_FIDO')]

In [409]:
data_merge_2_bil_diff<-unique(merge(
    sub_dati_bil_diff,
    bil_diff,
    by.x=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    by.y=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    all.x=TRUE
))

In [410]:
dati_bil_diff <- unique(merge(
    data_merge_2_bil_diff,
    dt_bil,
    by.x=c("SNDG_CEDUTO","dat_bil_diff"),
    by.y=c("COD_SNDG","DAT_BIL"),
    all.x=TRUE
))

In [411]:


dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 100, 
           last_onerifin_ebitda := VAL_VOCE_623/VAL_VOCE_644]
dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 104, 
           last_onerifin_ebitda := VAL_VOCE_624/VAL_VOCE_645]

dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',5)) & COD_CARTIPO == 100, 
           last_onerifin_ebitda := VAL_VOCE_621/VAL_VOCE_641]
dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',5)) & COD_CARTIPO == 104, 
           last_onerifin_ebitda := VAL_VOCE_621/VAL_VOCE_642]


dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 100, 
           last_fatturato :=max(VAL_VOCE_605,VAL_VOCE_601,na.rm = TRUE)]
dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 104, 
           last_fatturato :=max(VAL_VOCE_605,VAL_VOCE_601,na.rm = TRUE)]

dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',5)) & COD_CARTIPO == 100, 
           last_fatturato := VAL_VOCE_601]
dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',5)) & COD_CARTIPO == 104, 
           last_fatturato := VAL_VOCE_601]

dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',1:5)) & COD_CARTIPO == 100, 
              last_pat_netto := VAL_VOCE_415]
dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',1:5)) & COD_CARTIPO == 104, 
          last_pat_netto := VAL_VOCE_413]

dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',6:8)) & COD_CARTIPO == 100, 
         last_pat_netto := VAL_VOCE_431]
dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',6:8)) & COD_CARTIPO == 104, 
         last_pat_netto := VAL_VOCE_432]

dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',9)) , 
        last_pat_netto := VAL_VOCE_432]

In [412]:
dati_bil_diff

SNDG_CEDUTO,dat_bil_diff,DATA_DELIBERA_O_ANNULL_FIDO,dat_bil_max,NUM_PRATICA,PROGRESS_PRATICA,data_diff,DAT_ULTRICL,COD_CARTIPO,COD_STRUTTBIL,⋯,VAL_VOCE_927,VAL_VOCE_928,VAL_VOCE_929,VAL_VOCE_930,VAL_VOCE_931,VAL_VOCE_837,VAL_VOCE_838,last_onerifin_ebitda,last_fatturato,last_pat_netto
<chr>,<date>,<chr>,<date>,<dbl>,<dbl>,<date>,<date>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0000000000007435,NA,2017-12-13-23.12.13.369581,NA,71213230934,19,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000008626,NA,2020-03-17-23.05.24.295914,NA,317230521,1,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000009554,NA,2018-06-26-23.17.27.178500,NA,80626230257,18,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000009972,NA,2016-12-09-22.37.50.845653,NA,61209223117,6,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000010803,NA,2020-07-07-00.08.36.167718,NA,706230185,6,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000011090,NA,2017-05-16-22.34.46.658290,NA,70516223122,1,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000011301,NA,2018-08-07-23.26.20.879021,NA,80807230438,1,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000012113,NA,2019-05-06-23.27.19.168761,2015-12-31,90506230835,3,2014-12-31,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000015807,NA,2017-12-13-00.53.21.585212,NA,71212231459,1,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [413]:
master_bil <- merge(master_bil,
                    dati_bil_diff[,c('NUM_PRATICA','PROGRESS_PRATICA',
                                     'last_onerifin_ebitda','last_fatturato','last_pat_netto')],
                    by = c('NUM_PRATICA','PROGRESS_PRATICA'),
                    all.x = TRUE
                    )

In [414]:
master_bil[,delta_fatturato := fatturato-last_fatturato]
master_bil[,delta_pat_netto := pat_netto-last_pat_netto]

In [415]:
colnames(master_bil)

[1] "NUM_PRATICA"                 "PROGRESS_PRATICA"           
  [3] "SNDG_CEDUTO"                 "dat_bil_max"                
  [5] "DATA_DELIBERA_O_ANNULL_FIDO" "MacroSegmento_Attuale LC"   
  [7] "data_bil"                    "DAT_ULTRICL"                
  [9] "COD_CARTIPO"                 "COD_STRUTTBIL"              
 [11] "VAL_VOCE_301"                "VAL_VOCE_302"               
 [13] "VAL_VOCE_303"                "VAL_VOCE_304"               
 [15] "VAL_VOCE_305"                "VAL_VOCE_306"               
 [17] "VAL_VOCE_307"                "VAL_VOCE_308"               
 [19] "VAL_VOCE_309"                "VAL_VOCE_310"               
 [21] "VAL_VOCE_311"                "VAL_VOCE_312"               
 [23] "VAL_VOCE_313"                "VAL_VOCE_314"               
 [25] "VAL_VOCE_315"                "VAL_VOCE_316"               
 [27] "VAL_VOCE_317"                "VAL_VOCE_318"               
 [29] "VAL_VOCE_319"                "VAL_VOCE_320"               
 [31] "VAL_VOCE_321"                "VAL_VOCE_322"               
 [33] "VAL_VOCE_323"                "VAL_VOCE_324"               
 [35] "VAL_VOCE_325"                "VAL_VOCE_326"               
 [37] "VAL_VOCE_327"                "VAL_VOCE_328"               
 [39] "VAL_VOCE_329"                "VAL_VOCE_330"               
 [41] "VAL_VOCE_331"                "VAL_VOCE_332"               
 [43] "VAL_VOCE_333"                "VAL_VOCE_334"               
 [45] "VAL_VOCE_335"                "VAL_VOCE_336"               
 [47] "VAL_VOCE_337"                "VAL_VOCE_338"               
 [49] "VAL_VOCE_401"                "VAL_VOCE_402"               
 [51] "VAL_VOCE_403"                "VAL_VOCE_404"               
 [53] "VAL_VOCE_405"                "VAL_VOCE_406"               
 [55] "VAL_VOCE_407"                "VAL_VOCE_408"               
 [57] "VAL_VOCE_409"                "VAL_VOCE_410"               
 [59] "VAL_VOCE_411"                "VAL_VOCE_412"               
 [61] "VAL_VOCE_413"                "VAL_VOCE_414"               
 [63] "VAL_VOCE_415"                "VAL_VOCE_416"               
 [65] "VAL_VOCE_417"                "VAL_VOCE_418"               
 [67] "VAL_VOCE_419"                "VAL_VOCE_420"               
 [69] "VAL_VOCE_421"                "VAL_VOCE_422"               
 [71] "VAL_VOCE_423"                "VAL_VOCE_424"               
 [73] "VAL_VOCE_425"                "VAL_VOCE_426"               
 [75] "VAL_VOCE_427"                "VAL_VOCE_428"               
 [77] "VAL_VOCE_429"                "VAL_VOCE_430"               
 [79] "VAL_VOCE_431"                "VAL_VOCE_432"               
 [81] "VAL_VOCE_433"                "VAL_VOCE_434"               
 [83] "VAL_VOCE_435"                "VAL_VOCE_436"               
 [85] "VAL_VOCE_437"                "VAL_VOCE_438"               
 [87] "VAL_VOCE_439"                "VAL_VOCE_440"               
 [89] "VAL_VOCE_441"                "VAL_VOCE_442"               
 [91] "VAL_VOCE_501"                "VAL_VOCE_502"               
 [93] "VAL_VOCE_503"                "VAL_VOCE_504"               
 [95] "VAL_VOCE_505"                "VAL_VOCE_506"               
 [97] "VAL_VOCE_507"                "VAL_VOCE_508"               
 [99] "VAL_VOCE_509"                "VAL_VOCE_510"               
[101] "VAL_VOCE_511"                "VAL_VOCE_512"               
[103] "VAL_VOCE_513"                "VAL_VOCE_514"               
[105] "VAL_VOCE_515"                "VAL_VOCE_516"               
[107] "VAL_VOCE_517"                "VAL_VOCE_518"               
[109] "VAL_VOCE_519"                "VAL_VOCE_520"               
[111] "VAL_VOCE_521"                "VAL_VOCE_522"               
[113] "VAL_VOCE_523"                "VAL_VOCE_524"               
[115] "VAL_VOCE_525"                "VAL_VOCE_526"               
[117] "VAL_VOCE_527"                "VAL_VOCE_528"               
[119] "VAL_VOCE_529"                "VAL_VOCE_530"               
[121] "VAL_VOCE_531"                "VAL_V

In [ ]:
colnames()

In [416]:
master_bil[BIL_delta_t>3, c('fatturato','pat_netto','onerifin_ebitda','PFN_Ebitda',
                            'delta_fatturato','delta_pat_netto')] <- NA
colnames(master_bil)[colnames(master_bil) %in% c('fatturato','pat_netto','onerifin_ebitda',
                                        'PFN_Ebitda','delta_fatturato','delta_pat_netto')] <- paste0("BIL_",c('fatturato','pat_netto','onerifin_ebitda','PFN_Ebitda','delta_fatturato','delta_pat_netto'))
master_bil


NUM_PRATICA,PROGRESS_PRATICA,SNDG_CEDUTO,dat_bil_max,DATA_DELIBERA_O_ANNULL_FIDO,MacroSegmento_Attuale LC,data_bil,DAT_ULTRICL,COD_CARTIPO,COD_STRUTTBIL,⋯,flag_t,BIL_fatturato,BIL_pat_netto,BIL_onerifin_ebitda,BIL_PFN_Ebitda,last_onerifin_ebitda,last_fatturato,last_pat_netto,BIL_delta_fatturato,BIL_delta_pat_netto
<dbl>,<dbl>,<chr>,<date>,<chr>,<chr>,<date>,<date>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
102230026,1,0000000091391367,2018-12-31,2020-01-02-23.04.55.267865,Sme Retail,2018-12-31,2019-09-09,100,04,⋯,1,-0.005128205,28888251,1106,1.87179487,0.004629630,28888251,577,0,529
102230029,2,0000000088606666,NA,2020-01-02-23.04.55.267865,Sme Retail,2018-12-31,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
102230030,1,0000000017851888,2018-12-31,2020-01-02-23.04.55.267865,Sme Corporate,2018-12-31,2019-12-12,100,04,⋯,1,0.343283582,28888251,194,8.75124378,0.120401338,28888251,327,0,-133
102230031,1,0000000067790030,2018-12-31,2020-01-02-23.04.55.267865,Sme Corporate,2018-12-31,2019-09-10,100,03,⋯,1,0.043478261,28888251,4124,-1.04840033,0.042207792,28888251,3318,0,806
102230032,1,0000000017591710,2018-12-31,2020-01-02-23.04.55.267865,Sme Corporate,2018-12-31,2019-08-02,100,01,⋯,1,0.111111111,28888251,503,5.12865497,0.056122449,28888251,486,0,17
102230034,1,0000000017605272,2018-12-31,2020-01-02-23.04.55.267865,Sme Retail,2018-12-31,2020-06-11,100,03,⋯,1,0.014492754,28888251,582,-0.42753623,0.062500000,28888251,515,0,67
102230035,1,0000000017605835,2018-12-31,2020-01-02-23.04.55.267865,Sme Retail,2018-12-31,2019-11-06,100,04,⋯,1,0.000000000,28888251,143,-0.39053254,0.000000000,28888251,53,0,90
102230039,1,0000000017779860,2018-12-31,2020-01-03-16.30.33.281674,Large corporate,2018-12-31,2019-07-29,104,02,⋯,1,0.054368732,76156000,1621105,0.62993504,0.072610087,67347000,1604328,8809000,16777
102230040,1,0000000027621470,2018-12-31,2020-01-03-17.00.33.418378,Corporate,2018-12-31,2019-08-12,100,04,⋯,1,0.301542777,28888251,28203,-2.37353904,0.149606299,28888251,32046,0,-3843


In [417]:
cols_2keep_bil <- c('NUM_PRATICA','PROGRESS_PRATICA','BIL_fatturato','BIL_pat_netto','BIL_PFN_Ebitda','BIL_onerifin_ebitda','BIL_delta_fatturato','BIL_delta_pat_netto')
                
cols_2keep_bil

[1] "NUM_PRATICA"         "PROGRESS_PRATICA"    "BIL_fatturato"      
[4] "BIL_pat_netto"       "BIL_PFN_Ebitda"      "BIL_onerifin_ebitda"
[7] "BIL_delta_fatturato" "BIL_delta_pat_netto"

In [418]:
master_bil_sme <- master_bil[`MacroSegmento_Attuale LC` == 'Sme Retail', ..cols_2keep_bil]
master_bil_nonsme <- master_bil[`MacroSegmento_Attuale LC` != 'Sme Retail' & `MacroSegmento_Attuale LC` != "Altro", ..cols_2keep_bil ]

saveRDS(master_bil_sme,'../data/RDS/master/sme/master_bil.RDS')
saveRDS(master_bil_nonsme,'../data/RDS/master/non_sme/master_bil.RDS')

In [419]:
str((master_bil_sme$BIL_PFN_Ebitda))

 num [1:88630] 1.872 NA -0.428 -0.391 -1.735 ...
 - attr(*, "label")= chr "Strutture Patrimoniali Di Sintesi"


In [420]:
#data_merge_2_bil

rbindlist(lapply(unique(master_bil$`MacroSegmento_Attuale LC`), function(seg){
    missing_val <- sum(is.na(master_bil[`MacroSegmento_Attuale LC`==seg]$BIL_pat_netto))
    all_val <- nrow(master_bil[`MacroSegmento_Attuale LC`==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,62337,88630,70.33
Sme Corporate,2000,31229,6.40
Large corporate,187,3010,6.21
Corporate,529,6765,7.82
Altro,8814,10474,84.15


In [421]:
range(master_bil$BIL_fatturato,na.rm = TRUE)

[1] -82 Inf

In [979]:
dati_bil_diff[COD_STRUTTBIL %in% c(paste0('0',1:4)) & COD_CARTIPO == 104 & VAL_VOCE_624/VAL_VOCE_645 > 10, 
           .(VAL_VOCE_624,VAL_VOCE_645)]

VAL_VOCE_624,VAL_VOCE_645
<dbl>,<dbl>
53631,3923
53631,3923
53631,3923


In [981]:
master_appo_bil <- copy(master_bil)

In [982]:
master_appo_bil[, oneri_ebitda := VAL_VOCE_624,VAL_VOCE_645]

In [992]:
nrow(master_appo_bil[oneri_ebitda/1000 <50 & oneri_ebitda/1000 >-50,])/nrow(master_appo_bil)

[1] 0.4741057

## MODELLI FISCALI

In [422]:
dt_mod_fisc <- readRDS('../data/RDS/dt_modfisc.RDS')

In [423]:
# sort(colnames(dt_mod_fisc_orig))

# write.xlsx(colnames(dt_mod_fisc_orig),
#            'variabili_modfisc.xlsx',
#            row.names = FALSE)

In [424]:
dt_mod_fisc

COD_CARTIPO,DAT_BIL,DAT_RICLAS,DAT_COST,COD_MODELLO,COD_TIPO_SOGG,COD_FORMGIUR,COD_TIPO_CONT,COD_STATO_BIL,COD_STRUT_BIL,⋯,IMP_FOND_RISC_ONE,IMP_TRA_FINE_RAP,IMP_ALTR_DEB,IMP_RAT_RIS_PAS,IMP_FOND_AMM_IMM,IMP_TOT_PASS,IMP_REDD_IMP_FORF,VAL_TOTDIPE,COD_SNDG,NUM_DURLEG
<chr>,<date>,<date>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
120,2018-12-31,2020-07-03,2000-01-31,22,2,DI,S,22,02,⋯,0,0,0,0,0,0,0,-9999999,0000000000008626,12
120,2019-12-31,2021-12-17,2000-01-31,22,2,DI,S,22,04,⋯,0,0,0,0,0,0,0,-9999999,0000000000008626,12
120,2020-12-31,2021-12-17,2000-01-31,22,2,DI,S,22,04,⋯,0,0,0,0,0,0,0,-9999999,0000000000008626,12
120,2017-12-31,2018-10-25,2001-08-02,22,2,DI,S,22,04,⋯,0,0,0,0,0,0,0,-9999999,0000000000009554,12
120,2018-12-31,2020-05-19,2001-08-02,22,2,DI,S,22,04,⋯,0,0,0,0,0,0,0,-9999999,0000000000009554,12
120,2017-12-31,2018-12-21,2001-03-21,22,2,DI,S,22,02,⋯,0,0,0,0,0,0,0,-9999999,0000000000011090,12
120,2018-12-31,2019-10-01,2001-03-21,22,2,DI,S,22,02,⋯,0,0,0,0,0,0,0,-9999999,0000000000011090,12
120,2017-12-31,2019-05-09,1990-12-18,22,2,DI,S,22,02,⋯,0,0,0,0,0,0,0,-9999999,0000000000011301,12
120,2018-12-31,2020-11-24,1990-12-18,22,2,DI,S,22,02,⋯,0,0,0,0,0,0,0,-9999999,0000000000011301,12


In [425]:
tab_DAT_RICLAS <- dt_mod_fisc[,list(DAT_RICLAS_DEF = max(DAT_RICLAS),DAT_RICLAS=DAT_RICLAS), by= c("COD_SNDG","DAT_BIL")]

dt_mod_fisc <- merge(dt_mod_fisc,tab_DAT_RICLAS, all.x = TRUE, by= c("COD_SNDG","DAT_BIL",'DAT_RICLAS'))

dt_mod_fisc <- unique(dt_mod_fisc[DAT_RICLAS_DEF == DAT_RICLAS,])
#sndg_2rm <- dt_bil[,list(num = .N), by = c("COD_SNDG","DAT_BIL")][num>1,]$COD_SNDG

In [426]:
tab_DAT_COST <- dt_mod_fisc[,list(DAT_COST_DEF = max(DAT_COST),DAT_COST=DAT_COST), by= c("COD_SNDG","DAT_BIL")]

In [427]:
dt_mod_fisc <- merge(dt_mod_fisc,tab_DAT_COST, all.x = TRUE, by= c("COD_SNDG","DAT_BIL",'DAT_COST'))

dt_mod_fisc <- unique(dt_mod_fisc[DAT_COST_DEF == DAT_COST,])
#sndg_2rm <- dt_bil[,list(num = .N), by = c("COD_SNDG","DAT_BIL")][num>1,]$COD_SNDG

In [428]:
colnames(dt_mod_fisc)

[1] "COD_SNDG"           "DAT_BIL"            "DAT_COST"          
 [4] "DAT_RICLAS"         "COD_CARTIPO"        "COD_MODELLO"       
 [7] "COD_TIPO_SOGG"      "COD_FORMGIUR"       "COD_TIPO_CONT"     
[10] "COD_STATO_BIL"      "COD_STRUT_BIL"      "COD_DIPE"          
[13] "NUM_DUROPE"         "COD_ATECO007"       "IMP_RCV_VEND_PREST"
[16] "IMP_AL_RCV_PROVEN"  "IMP_TOT_COMP_POS"   "IMP_TOT_AMM"       
[19] "IMP_ON_DIVERS_GEST" "IMP_INT_PASSON_ASS" "IMP_TOT_COMP_NEG"  
[22] "VAL_PROD_NETTA"     "IMP_DISP_LIQ_TOT"   "IMP_DEB_V_FORNITOR"
[25] "IMP_DEB_V_BAN_ENES" "IMP_DEB_V_BAN_OLES" "IMP_CRE_V_CLIENTI" 
[28] "IMP_IMM_MAT_S_FIN"  "IMP_IMM_IMM_S_FIN"  "IMP_TOT_ATTIVITA"  
[31] "IMP_PATTIM_NETTO"   "IMP_UTILE_PERD_UN"  "IMP_IMM_IMM"       
[34] "IMP_IMM_MAT_NET"    "IMP_FOND_AMM_B_MAT" "IMP_IMM_FINANZ"    
[37] "IMP_RIM_MAGAZINE"   "IMP_ALTR_CRED_ATT"  "IMP_ATT_FIN_NO_IMM"
[40] "IMP_RAT_RIS_ATT"    "IMP_FOND_RISC_ONE"  "IMP_TRA_FINE_RAP"  
[43] "IMP_ALTR_DEB"       "IMP_RAT_RIS_PAS"    "IMP_FOND_AMM_IMM"  
[46] "IMP_TOT_PASS"       "IMP_REDD_IMP_FORF"  "VAL_TOTDIPE"       
[49] "NUM_DURLEG"         "DAT_RICLAS_DEF"     "DAT_COST_DEF"

Rimuovo l'SNDG 0000000083265125 per l'anno 2018-12-31 perché ha due dati discordanti

In [429]:
dt_mod_fisc <- dt_mod_fisc[!(COD_SNDG =='0000000083265125' &DAT_BIL == '2018-12-31') ,]#[order(COD_SNDG,DAT_BIL)])

In [430]:
sndg_dupl <- dt_mod_fisc[,.(n = .N), by = c('COD_SNDG','DAT_BIL')][n>1]$COD_SNDG

In [431]:
dt_mod_fisc[COD_SNDG %in% sndg_dupl,][order(COD_SNDG,DAT_BIL)]

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


COD_SNDG,DAT_BIL,DAT_COST,DAT_RICLAS,COD_CARTIPO,COD_MODELLO,COD_TIPO_SOGG,COD_FORMGIUR,COD_TIPO_CONT,COD_STATO_BIL,⋯,IMP_TRA_FINE_RAP,IMP_ALTR_DEB,IMP_RAT_RIS_PAS,IMP_FOND_AMM_IMM,IMP_TOT_PASS,IMP_REDD_IMP_FORF,VAL_TOTDIPE,NUM_DURLEG,DAT_RICLAS_DEF,DAT_COST_DEF
<chr>,<date>,<date>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<date>,<date>


In [432]:
table(dt_mod_fisc$COD_MODELLO, useNA = 'ifany')


   10    11    12    13    14    15    21    22    23    24    25    29 
   53 16920  9155    35   180     9  9062 30878    35   220   296     2 

In [433]:
sub_dati_mod_fisc <- dt_orig[,
                    c('NUM_PRATICA','PROGRESS_PRATICA','SNDG_CEDUTO','DATA_DELIBERA_O_ANNULL_FIDO',
                      'MacroSegmento_Attuale LC')]

sub_dati_mod_fisc[month(DATA_DELIBERA_O_ANNULL_FIDO)<=8,
             data_mod_fisc := as.Date(paste0(year(DATA_DELIBERA_O_ANNULL_FIDO)-2,"-12-31")) ]
sub_dati_mod_fisc[month(DATA_DELIBERA_O_ANNULL_FIDO)>8,
             data_mod_fisc := as.Date(paste0(year(DATA_DELIBERA_O_ANNULL_FIDO)-1,"-12-31")) ]

In [434]:
data_tot_mod_fisc <-merge(
    sub_dati_mod_fisc,
    dt_mod_fisc,
    by.x=c("SNDG_CEDUTO"),
    by.y=c("COD_SNDG"),
    allow.cartesian = TRUE,
    all.x=TRUE
)

#data_tot_mod_fisc

In [435]:
mod_fisc_max<-data_tot_mod_fisc[DAT_BIL<=data_mod_fisc,
                      .(dat_mod_fisc_max=max(
                          DAT_BIL,na.rm=TRUE)),
                         by=c("SNDG_CEDUTO",'DATA_DELIBERA_O_ANNULL_FIDO')]

In [436]:
data_merge_2_mod_fisc<-unique(merge(
    sub_dati_mod_fisc,
    mod_fisc_max,
    by.x=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    by.y=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    all.x=TRUE
))

In [437]:
master_mod_fisc <- unique(merge(
    data_merge_2_mod_fisc,
    dt_mod_fisc,
    by.x=c("SNDG_CEDUTO","dat_mod_fisc_max"),
    by.y=c("COD_SNDG","DAT_BIL"),
    all.x=TRUE
))

master_mod_fisc[,MOD_FISC_delta_t := round(as.numeric(as.Date(DATA_DELIBERA_O_ANNULL_FIDO) -dat_mod_fisc_max )/365,0) ]

In [438]:
master_mod_fisc[, fatturato := IMP_RCV_VEND_PREST]
master_mod_fisc[, pat_netto := IMP_PATTIM_NETTO]

In [439]:
table(master_mod_fisc$MOD_FISC_delta_t)


    1     2     3     4     5     6 
11719  4631  1249   479   160    25 

In [440]:

rbindlist(lapply(unique(master_mod_fisc$`MacroSegmento_Attuale LC`), function(seg){
    missing_val <- sum(is.na(master_mod_fisc[`MacroSegmento_Attuale LC`==seg]$COD_MODELLO))
    all_val <- nrow(master_mod_fisc[`MacroSegmento_Attuale LC`==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,72556,88630,81.86
Sme Corporate,29247,31229,93.65
Altro,10306,10474,98.40
Corporate,6728,6765,99.45
Large corporate,3008,3010,99.93


### Creazione indicatori

In [441]:
table(dt_mod_fisc[,DAT_BIL])


2014-12-31 2015-12-31 2016-12-31 2017-12-31 2018-12-31 2019-12-31 2020-12-31 
      8923       9919      10696      11532      11841       9295       4639 

In [442]:
sub_dati_mod_fisc_diff <- master_mod_fisc[, c('SNDG_CEDUTO','dat_mod_fisc_max','DATA_DELIBERA_O_ANNULL_FIDO','NUM_PRATICA','PROGRESS_PRATICA')]

sub_dati_mod_fisc_diff[,data_diff := ifelse(!is.na(dat_mod_fisc_max),paste0(year(as.Date(dat_mod_fisc_max))-1,"-12-31"),NA)]
sub_dati_mod_fisc_diff[,data_diff := as.Date(data_diff)]

In [443]:
data_tot_mod_fisc_diff <-merge(
    sub_dati_mod_fisc_diff,
    dt_mod_fisc,
    by.x=c("SNDG_CEDUTO"),
    by.y=c("COD_SNDG"),
    allow.cartesian = TRUE,
    all.x=TRUE
)

In [444]:
mod_fisc_diff<-data_tot_mod_fisc_diff[DAT_BIL<=data_diff,
                      .(dat_mod_fisc_diff=max(
                          DAT_BIL,na.rm=TRUE)),
                         by=c("SNDG_CEDUTO",'DATA_DELIBERA_O_ANNULL_FIDO')]

data_merge_2_mod_fisc_diff<-unique(merge(
    sub_dati_mod_fisc_diff,
    mod_fisc_diff,
    by.x=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    by.y=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    all.x=TRUE
))

dati_mod_fisc_diff <- unique(merge(
    data_merge_2_mod_fisc_diff,
    dt_mod_fisc,
    by.x=c("SNDG_CEDUTO","dat_mod_fisc_diff"),
    by.y=c("COD_SNDG","DAT_BIL"),
    all.x=TRUE
))

In [445]:
dati_mod_fisc_diff[,last_fatturato := IMP_RCV_VEND_PREST]
dati_mod_fisc_diff[,last_pat_netto := IMP_PATTIM_NETTO]

In [446]:
master_mod_fisc <- merge(master_mod_fisc,
                    dati_mod_fisc_diff[,c('NUM_PRATICA','PROGRESS_PRATICA',
                                     'last_fatturato','last_pat_netto')],
                    by = c('NUM_PRATICA','PROGRESS_PRATICA'),
                    all.x = TRUE
                    )

In [447]:
colnames(master_mod_fisc)

[1] "NUM_PRATICA"                 "PROGRESS_PRATICA"           
 [3] "SNDG_CEDUTO"                 "dat_mod_fisc_max"           
 [5] "DATA_DELIBERA_O_ANNULL_FIDO" "MacroSegmento_Attuale LC"   
 [7] "data_mod_fisc"               "DAT_COST"                   
 [9] "DAT_RICLAS"                  "COD_CARTIPO"                
[11] "COD_MODELLO"                 "COD_TIPO_SOGG"              
[13] "COD_FORMGIUR"                "COD_TIPO_CONT"              
[15] "COD_STATO_BIL"               "COD_STRUT_BIL"              
[17] "COD_DIPE"                    "NUM_DUROPE"                 
[19] "COD_ATECO007"                "IMP_RCV_VEND_PREST"         
[21] "IMP_AL_RCV_PROVEN"           "IMP_TOT_COMP_POS"           
[23] "IMP_TOT_AMM"                 "IMP_ON_DIVERS_GEST"         
[25] "IMP_INT_PASSON_ASS"          "IMP_TOT_COMP_NEG"           
[27] "VAL_PROD_NETTA"              "IMP_DISP_LIQ_TOT"           
[29] "IMP_DEB_V_FORNITOR"          "IMP_DEB_V_BAN_ENES"         
[31] "IMP_DEB_V_BAN_OLES"          "IMP_CRE_V_CLIENTI"          
[33] "IMP_IMM_MAT_S_FIN"           "IMP_IMM_IMM_S_FIN"          
[35] "IMP_TOT_ATTIVITA"            "IMP_PATTIM_NETTO"           
[37] "IMP_UTILE_PERD_UN"           "IMP_IMM_IMM"                
[39] "IMP_IMM_MAT_NET"             "IMP_FOND_AMM_B_MAT"         
[41] "IMP_IMM_FINANZ"              "IMP_RIM_MAGAZINE"           
[43] "IMP_ALTR_CRED_ATT"           "IMP_ATT_FIN_NO_IMM"         
[45] "IMP_RAT_RIS_ATT"             "IMP_FOND_RISC_ONE"          
[47] "IMP_TRA_FINE_RAP"            "IMP_ALTR_DEB"               
[49] "IMP_RAT_RIS_PAS"             "IMP_FOND_AMM_IMM"           
[51] "IMP_TOT_PASS"                "IMP_REDD_IMP_FORF"          
[53] "VAL_TOTDIPE"                 "NUM_DURLEG"                 
[55] "DAT_RICLAS_DEF"              "DAT_COST_DEF"               
[57] "MOD_FISC_delta_t"            "fatturato"                  
[59] "pat_netto"                   "last_fatturato"             
[61] "last_pat_netto"

In [448]:
master_mod_fisc[,delta_fatturato := fatturato - last_fatturato ]
master_mod_fisc[,delta_pat_netto := pat_netto - last_pat_netto ]

In [449]:
cols_2keep_modfisc <- c('NUM_PRATICA','PROGRESS_PRATICA','fatturato','pat_netto','delta_fatturato','delta_pat_netto')

In [450]:
master_mod_fisc[MOD_FISC_delta_t >3, c('fatturato','pat_netto','delta_fatturato','delta_pat_netto')]<- NA

In [451]:
master_mod_fisc <- master_mod_fisc[,..cols_2keep_modfisc]
colnames(master_mod_fisc)<- c('NUM_PRATICA','PROGRESS_PRATICA','MOD_FISC_fatturato','MOD_FISC_pat_netto','MOD_FISC_delta_fatturato','MOD_FISC_delta_pat_netto')

In [452]:
master_bil_mfisc <- merge(master_bil[,c(cols_2keep_bil,'MacroSegmento_Attuale LC'), with = FALSE],
                          master_mod_fisc[,c('NUM_PRATICA','PROGRESS_PRATICA','MOD_FISC_fatturato','MOD_FISC_pat_netto','MOD_FISC_delta_fatturato','MOD_FISC_delta_pat_netto')],
                          by = c('NUM_PRATICA','PROGRESS_PRATICA'),
                          all.x = TRUE
                         )

In [453]:
colnames(master_bil_mfisc)

[1] "NUM_PRATICA"              "PROGRESS_PRATICA"        
 [3] "BIL_fatturato"            "BIL_pat_netto"           
 [5] "BIL_PFN_Ebitda"           "BIL_onerifin_ebitda"     
 [7] "BIL_delta_fatturato"      "BIL_delta_pat_netto"     
 [9] "MacroSegmento_Attuale LC" "MOD_FISC_fatturato"      
[11] "MOD_FISC_pat_netto"       "MOD_FISC_delta_fatturato"
[13] "MOD_FISC_delta_pat_netto"

In [454]:
master_bil_mfisc[,BIL_MFISC_fatturato := ifelse(!is.na(BIL_fatturato),BIL_fatturato,MOD_FISC_fatturato)]
master_bil_mfisc[,BIL_MFISC_pat_netto := ifelse(!is.na(BIL_pat_netto),BIL_pat_netto,MOD_FISC_pat_netto)]

master_bil_mfisc[,BIL_MFISC_delta_fatturato := ifelse(!is.na(BIL_delta_fatturato),
                                                      BIL_delta_fatturato,MOD_FISC_delta_fatturato)]
master_bil_mfisc[,BIL_MFISC_delta_pat_netto := ifelse(!is.na(BIL_delta_pat_netto),
                                                      BIL_delta_pat_netto,MOD_FISC_delta_pat_netto)]

In [455]:
cols_2keep_bil_mf <- c('NUM_PRATICA','PROGRESS_PRATICA','BIL_MFISC_fatturato',
                       'BIL_MFISC_pat_netto','BIL_MFISC_delta_fatturato','BIL_MFISC_delta_pat_netto')

In [456]:
master_bil_mf_sme <- master_bil_mfisc[`MacroSegmento_Attuale LC` == 'Sme Retail', ..cols_2keep_bil_mf]
master_bil_mf_nonsme <- master_bil_mfisc[`MacroSegmento_Attuale LC` != 'Sme Retail' & `MacroSegmento_Attuale LC` != "Altro", ..cols_2keep_bil_mf ]

saveRDS(master_bil_mf_sme,'../data/RDS/master/sme/master_bil_mf.RDS')
saveRDS(master_bil_mf_nonsme,'../data/RDS/master/non_sme/master_bil_mf.RDS')

## XRA

In [457]:
dt_xra_orig <- readRDS('../data/RDS/dt_xra_ok.RDS')

In [458]:
colnames(dt_xra_orig)

[1] "DAT_FINE_PERIO"   "COD_SOC"          "COD_NSG"          "COD_SNDG"        
 [5] "DAT_INI_SCF"      "NUM_GG_SCONF"     "IMP_ACR_DEL"      "IMP_ACR_OP"      
 [9] "IMP_UTIL"         "IMP_UTIL_TOT"     "IMP_UTIL_CUM"     "IMP_SCONFINO"    
[13] "IMP_SCONFINO_CUM" "COD_NDG_LEG"      "COD_ST_SEGN"      "NUM_GG_SCONF_OLD"
[17] "COD_DIP_GRP"      "COD_SEGM"         "DAT_SEGNAL"       "IMP_UTIL_FIRMA"  
[21] "FLG_CPGRP"        "FLG_LEG"          "COD_NSG_PF"

In [459]:
table(dt_xra_orig[DAT_SEGNAL < "1950-01-01",]$DAT_SEGNAL)


1600-01-01 
   1492918 

In [460]:
colnames(dt_xra_orig)
old_cols <- colnames(dt_xra_orig)[grepl("OLD",colnames(dt_xra_orig))]
cum_cols <- colnames(dt_xra_orig)[grepl("CUM",colnames(dt_xra_orig))]
cum_cols

[1] "DAT_FINE_PERIO"   "COD_SOC"          "COD_NSG"          "COD_SNDG"        
 [5] "DAT_INI_SCF"      "NUM_GG_SCONF"     "IMP_ACR_DEL"      "IMP_ACR_OP"      
 [9] "IMP_UTIL"         "IMP_UTIL_TOT"     "IMP_UTIL_CUM"     "IMP_SCONFINO"    
[13] "IMP_SCONFINO_CUM" "COD_NDG_LEG"      "COD_ST_SEGN"      "NUM_GG_SCONF_OLD"
[17] "COD_DIP_GRP"      "COD_SEGM"         "DAT_SEGNAL"       "IMP_UTIL_FIRMA"  
[21] "FLG_CPGRP"        "FLG_LEG"          "COD_NSG_PF"

[1] "IMP_UTIL_CUM"     "IMP_SCONFINO_CUM"

In [461]:
cols_xra <- setdiff(colnames(dt_xra_orig),
            c('FLG_CPGRP','FLG_LEG','FLG_PD_TECN_1','FLG_PD_TECN_2','FLG_PD_TECN_3',#'DAT_SEGNAL',
              'FLG_PD_TECN_4','FLG_QUINTO_CASO','FLG_SOSPENSIONE',
              'FLG_AZZERAMENTO','FLG_ENTE_PUBBLICO','COD_NSG_PF',old_cols))#,cum_cols,old_cols


In [462]:
dt_xra<- unique(dt_xra_orig[,..cols_xra])

In [463]:
sub_dati_xra <- dt_orig[,
                    c('NUM_PRATICA','PROGRESS_PRATICA','SNDG_CEDUTO','DATA_DELIBERA_O_ANNULL_FIDO',
                      'MacroSegmento_Attuale LC')]


In [464]:
sub_dati_xra[,data_xra := as.Date(DATA_DELIBERA_O_ANNULL_FIDO)]
data_tot_xra <-merge(
    sub_dati_xra,
    dt_xra,
    by.x=c("SNDG_CEDUTO"),
    by.y=c("COD_SNDG"),
    allow.cartesian = TRUE,
    all.x=TRUE
)

In [465]:
 xra_max<-data_tot_xra[DAT_FINE_PERIO<=data_xra,
                                            .(dat_xra_max=max(
                                                DAT_FINE_PERIO,na.rm=TRUE)),
                                               by=c("SNDG_CEDUTO",'DATA_DELIBERA_O_ANNULL_FIDO')]

In [466]:
data_merge_2_xra<-unique(merge(
   sub_dati_xra,
   xra_max,
   by.x=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
   by.y=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
   all.x=TRUE
))

In [467]:
data_merge_2_xra

SNDG_CEDUTO,DATA_DELIBERA_O_ANNULL_FIDO,NUM_PRATICA,PROGRESS_PRATICA,MacroSegmento_Attuale LC,data_xra,dat_xra_max
<chr>,<chr>,<dbl>,<dbl>,<chr>,<date>,<date>
0000000000007435,2017-12-13-23.12.13.369581,71213230934,19,Sme Retail,2017-12-13,NA
0000000000008626,2020-03-17-23.05.24.295914,317230521,1,Sme Retail,2020-03-17,NA
0000000000009554,2018-06-26-23.17.27.178500,80626230257,18,Sme Retail,2018-06-26,NA
0000000000009972,2016-12-09-22.37.50.845653,61209223117,6,Sme Retail,2016-12-09,2016-10-26
0000000000010803,2020-07-07-00.08.36.167718,706230185,6,Sme Retail,2020-07-07,2020-07-03
0000000000011090,2017-05-16-22.34.46.658290,70516223122,1,Sme Retail,2017-05-16,2017-05-16
0000000000011301,2018-08-07-23.26.20.879021,80807230438,1,Sme Retail,2018-08-07,NA
0000000000012113,2019-05-06-23.27.19.168761,90506230835,3,Sme Retail,2019-05-06,NA
0000000000015807,2017-12-13-00.53.21.585212,71212231459,1,Sme Retail,2017-12-13,NA


In [468]:
master_xra_raw <- merge(
    data_merge_2_xra,
    dt_xra,
    by.x=c("SNDG_CEDUTO","dat_xra_max"),
    by.y=c("COD_SNDG","DAT_FINE_PERIO"),
    all.x=TRUE
)

In [469]:
master_xra_raw

SNDG_CEDUTO,dat_xra_max,DATA_DELIBERA_O_ANNULL_FIDO,NUM_PRATICA,PROGRESS_PRATICA,MacroSegmento_Attuale LC,data_xra,COD_SOC,COD_NSG,DAT_INI_SCF,⋯,IMP_UTIL_TOT,IMP_UTIL_CUM,IMP_SCONFINO,IMP_SCONFINO_CUM,COD_NDG_LEG,COD_ST_SEGN,COD_DIP_GRP,COD_SEGM,DAT_SEGNAL,IMP_UTIL_FIRMA
<chr>,<date>,<chr>,<dbl>,<dbl>,<chr>,<date>,<chr>,<chr>,<date>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
0000000000007435,NA,2017-12-13-23.12.13.369581,71213230934,19,Sme Retail,2017-12-13,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000008626,NA,2020-03-17-23.05.24.295914,317230521,1,Sme Retail,2020-03-17,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000009554,NA,2018-06-26-23.17.27.178500,80626230257,18,Sme Retail,2018-06-26,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000009972,2016-10-26,2016-12-09-22.37.50.845653,61209223117,6,Sme Retail,2016-12-09,,,2016-10-26,⋯,-396,-396,-396,-396,1953493,S,,SRMI,2016-10-26,0
0000000000010803,2020-07-03,2020-07-07-00.08.36.167718,706230185,6,Sme Retail,2020-07-07,,,2020-07-03,⋯,-109,-109,-109,-109,4418931,S,,RE,2020-07-03,0
0000000000011090,2017-05-16,2017-05-16-22.34.46.658290,70516223122,1,Sme Retail,2017-05-16,,,2017-05-16,⋯,-39382,-39382,-443,-443,4102821,,,SR,1600-01-01,0
0000000000011301,NA,2018-08-07-23.26.20.879021,80807230438,1,Sme Retail,2018-08-07,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000012113,NA,2019-05-06-23.27.19.168761,90506230835,3,Sme Retail,2019-05-06,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000015807,NA,2017-12-13-00.53.21.585212,71212231459,1,Sme Retail,2017-12-13,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [470]:
rbindlist(lapply(unique(master_xra_raw$`MacroSegmento_Attuale LC`), function(seg){
    missing_val <- sum(is.na(master_xra_raw[`MacroSegmento_Attuale LC`==seg]$IMP_SCONFINO))
    all_val <- nrow(master_xra_raw[`MacroSegmento_Attuale LC`==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,71641,88630,80.83
Sme Corporate,21882,31229,70.07
Altro,9835,10474,93.90
Corporate,4253,6765,62.87
Large corporate,1481,3010,49.20


In [471]:
dt_xra[COD_SNDG == '0000000000012113',]

DAT_FINE_PERIO,COD_SOC,COD_NSG,COD_SNDG,DAT_INI_SCF,NUM_GG_SCONF,IMP_ACR_DEL,IMP_ACR_OP,IMP_UTIL,IMP_UTIL_TOT,IMP_UTIL_CUM,IMP_SCONFINO,IMP_SCONFINO_CUM,COD_NDG_LEG,COD_ST_SEGN,COD_DIP_GRP,COD_SEGM,DAT_SEGNAL,IMP_UTIL_FIRMA
<date>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
2020-03-03,,,0000000000012113,2020-03-03,1,13132,11182,-6215,-6215,-6215,-33,-33,4456293,,,SRMI,1600-01-01,0


In [472]:
nrow(dt_orig[SNDG_CEDUTO =='NESSUN_NDG',])

[1] 5

In [473]:
saveRDS(master_xra_raw, '../data/RDS/master_raw_xra.RDS')

### Creazione indicatori

In [474]:
sub_dati_xra_30gg <- master_xra_raw[, c('SNDG_CEDUTO','DATA_DELIBERA_O_ANNULL_FIDO','NUM_PRATICA','PROGRESS_PRATICA')]

In [475]:
sub_dati_xra_30gg[,data_30gg := as.Date(DATA_DELIBERA_O_ANNULL_FIDO)-30]



In [476]:
data_tot_30gg <-merge(
    sub_dati_xra_30gg,
    dt_xra,
    by.x=c("SNDG_CEDUTO"),
    by.y=c("COD_SNDG"),
    allow.cartesian = TRUE,
    all.x=TRUE
)

In [477]:
data_tot_30gg

SNDG_CEDUTO,DATA_DELIBERA_O_ANNULL_FIDO,NUM_PRATICA,PROGRESS_PRATICA,data_30gg,DAT_FINE_PERIO,COD_SOC,COD_NSG,DAT_INI_SCF,NUM_GG_SCONF,⋯,IMP_UTIL_TOT,IMP_UTIL_CUM,IMP_SCONFINO,IMP_SCONFINO_CUM,COD_NDG_LEG,COD_ST_SEGN,COD_DIP_GRP,COD_SEGM,DAT_SEGNAL,IMP_UTIL_FIRMA
<chr>,<chr>,<dbl>,<dbl>,<date>,<date>,<chr>,<chr>,<date>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
0000000000007435,2017-12-13-23.12.13.369581,71213230934,19,2017-11-13,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000008626,2020-03-17-23.05.24.295914,317230521,1,2020-02-16,2021-04-01,,,2021-04-01,1,⋯,-25005,-25005,-5,-5,5035863,,,RE,1600-01-01,0
0000000000008626,2020-03-17-23.05.24.295914,317230521,1,2020-02-16,2021-04-02,,,2021-04-01,2,⋯,-25005,-25005,-5,-5,5035863,,,RE,1600-01-01,0
0000000000008626,2020-03-17-23.05.24.295914,317230521,1,2020-02-16,2021-04-06,,,2021-04-01,6,⋯,-25005,-25005,-5,-5,5035863,,,RE,1600-01-01,0
0000000000008626,2020-03-17-23.05.24.295914,317230521,1,2020-02-16,2021-04-07,,,2021-04-01,7,⋯,-25005,-25005,-5,-5,5035863,,,RE,1600-01-01,0
0000000000008626,2020-03-17-23.05.24.295914,317230521,1,2020-02-16,2021-04-08,,,2021-04-01,8,⋯,-25005,-25005,-5,-5,5035863,,,RE,1600-01-01,0
0000000000008626,2020-03-17-23.05.24.295914,317230521,1,2020-02-16,2021-04-09,,,2021-04-01,9,⋯,-25005,-25005,-5,-5,5035863,,,RE,1600-01-01,0
0000000000008626,2020-03-17-23.05.24.295914,317230521,1,2020-02-16,2021-12-14,,,2021-12-14,1,⋯,-25220,-25220,-220,-220,5035863,,,SR,1600-01-01,0
0000000000009554,2018-06-26-23.17.27.178500,80626230257,18,2018-05-27,2021-08-30,,,2021-08-30,1,⋯,-11698,-11698,-198,-198,5152840,S,,SR,2021-08-30,0


In [478]:
xra_30gg<-data_tot_30gg[DAT_FINE_PERIO>= data_30gg & DAT_FINE_PERIO <DATA_DELIBERA_O_ANNULL_FIDO,
                      .(media_sconfino_30gg=mean(
                          IMP_SCONFINO,na.rm=TRUE)),
                         by=c("SNDG_CEDUTO",'DATA_DELIBERA_O_ANNULL_FIDO')]

In [479]:
dt_xra[COD_SNDG == '0000000000055687' &DAT_FINE_PERIO < as.Date('2019-03-05') & 
       DAT_FINE_PERIO >= as.Date('2019-02-05'),]

DAT_FINE_PERIO,COD_SOC,COD_NSG,COD_SNDG,DAT_INI_SCF,NUM_GG_SCONF,IMP_ACR_DEL,IMP_ACR_OP,IMP_UTIL,IMP_UTIL_TOT,IMP_UTIL_CUM,IMP_SCONFINO,IMP_SCONFINO_CUM,COD_NDG_LEG,COD_ST_SEGN,COD_DIP_GRP,COD_SEGM,DAT_SEGNAL,IMP_UTIL_FIRMA
<date>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
2019-03-04,,,0000000000055687,2019-03-04,1,136263,136263,-136906,-136906,-136906,-643,-643,14175830,,,SR,1600-01-01,0


In [480]:
data_merge_2_xra_30gg<-unique(merge(
    sub_dati_xra_30gg,
    xra_30gg,
    by.x=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    by.y=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    all.x=TRUE
))

In [481]:
dati_xra_30gg <- unique(merge(
    data_merge_2_xra_30gg,
    dt_xra,
    by.x=c("SNDG_CEDUTO","data_30gg"),
    by.y=c("COD_SNDG","DAT_FINE_PERIO"),
    all.x=TRUE
))

In [482]:
dati_xra_30gg

SNDG_CEDUTO,data_30gg,DATA_DELIBERA_O_ANNULL_FIDO,NUM_PRATICA,PROGRESS_PRATICA,media_sconfino_30gg,COD_SOC,COD_NSG,DAT_INI_SCF,NUM_GG_SCONF,⋯,IMP_UTIL_TOT,IMP_UTIL_CUM,IMP_SCONFINO,IMP_SCONFINO_CUM,COD_NDG_LEG,COD_ST_SEGN,COD_DIP_GRP,COD_SEGM,DAT_SEGNAL,IMP_UTIL_FIRMA
<chr>,<date>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<date>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
0000000000007435,2017-11-13,2017-12-13-23.12.13.369581,71213230934,19,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000008626,2020-02-16,2020-03-17-23.05.24.295914,317230521,1,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000009554,2018-05-27,2018-06-26-23.17.27.178500,80626230257,18,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000009972,2016-11-09,2016-12-09-22.37.50.845653,61209223117,6,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000010803,2020-06-07,2020-07-07-00.08.36.167718,706230185,6,-109.00,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000011090,2017-04-16,2017-05-16-22.34.46.658290,70516223122,1,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000011301,2018-07-08,2018-08-07-23.26.20.879021,80807230438,1,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000012113,2019-04-06,2019-05-06-23.27.19.168761,90506230835,3,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000015807,2017-11-13,2017-12-13-00.53.21.585212,71212231459,1,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [483]:
master_appo <- merge(master_xra_raw,
                     dati_xra_30gg[,c('NUM_PRATICA','PROGRESS_PRATICA','media_sconfino_30gg')],
                by.x=c('NUM_PRATICA','PROGRESS_PRATICA'),
                by.y=c('NUM_PRATICA','PROGRESS_PRATICA'),
                     all.x = TRUE
                     
)

In [484]:
master_appo[,delta_t := round(as.numeric(as.Date(DATA_DELIBERA_O_ANNULL_FIDO) - as.Date(dat_xra_max))/365)]
master_appo

NUM_PRATICA,PROGRESS_PRATICA,SNDG_CEDUTO,dat_xra_max,DATA_DELIBERA_O_ANNULL_FIDO,MacroSegmento_Attuale LC,data_xra,COD_SOC,COD_NSG,DAT_INI_SCF,⋯,IMP_SCONFINO,IMP_SCONFINO_CUM,COD_NDG_LEG,COD_ST_SEGN,COD_DIP_GRP,COD_SEGM,DAT_SEGNAL,IMP_UTIL_FIRMA,media_sconfino_30gg,delta_t
<dbl>,<dbl>,<chr>,<date>,<chr>,<chr>,<date>,<chr>,<chr>,<date>,⋯,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>
102230026,1,0000000091391367,NA,2020-01-02-23.04.55.267865,Sme Retail,2020-01-02,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
102230029,2,0000000088606666,NA,2020-01-02-23.04.55.267865,Sme Retail,2020-01-02,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
102230030,1,0000000017851888,2018-03-21,2020-01-02-23.04.55.267865,Sme Corporate,2020-01-02,,,2018-03-02,⋯,-411319,-6653179,0,S,,SC,2018-03-02,0,NA,2
102230031,1,0000000067790030,NA,2020-01-02-23.04.55.267865,Sme Corporate,2020-01-02,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
102230032,1,0000000017591710,NA,2020-01-02-23.04.55.267865,Sme Corporate,2020-01-02,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
102230034,1,0000000017605272,NA,2020-01-02-23.04.55.267865,Sme Retail,2020-01-02,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
102230035,1,0000000017605835,NA,2020-01-02-23.04.55.267865,Sme Retail,2020-01-02,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
102230039,1,0000000017779860,2019-06-12,2020-01-03-16.30.33.281674,Large corporate,2020-01-03,,,2019-04-01,⋯,-157563,-3194131,0,S,,CO,2019-05-31,0,NA,1
102230040,1,0000000027621470,2019-03-15,2020-01-03-17.00.33.418378,Corporate,2020-01-03,,,2018-12-18,⋯,-8413,-739214,0,S,,CO,2018-12-18,0,NA,1


In [485]:
cols_xra <-c('media_sconfino_30gg',
             'IMP_SCONFINO','IMP_UTIL_TOT','NUM_GG_SCONF','delta_t')
master_xra <- master_appo[,c('NUM_PRATICA','PROGRESS_PRATICA','MacroSegmento_Attuale LC',cols_xra),with = FALSE]#


In [486]:
master_xra[delta_t>1,c('media_sconfino_30gg',
             'IMP_SCONFINO','IMP_UTIL_TOT','NUM_GG_SCONF')]<- NA

In [487]:

setnames(master_xra,cols_xra,paste0("XRA_",cols_xra))
master_xra[,XRA_delta_t:= NULL]

In [488]:
colnames(master_xra)

[1] "NUM_PRATICA"              "PROGRESS_PRATICA"        
[3] "MacroSegmento_Attuale LC" "XRA_media_sconfino_30gg" 
[5] "XRA_IMP_SCONFINO"         "XRA_IMP_UTIL_TOT"        
[7] "XRA_NUM_GG_SCONF"

In [489]:
cols_2keep_xra  <- setdiff(colnames(master_xra),'MacroSegmento_Attuale LC')

In [490]:
master_xra_sme <- master_xra[`MacroSegmento_Attuale LC` == 'Sme Retail', ..cols_2keep_xra]
master_xra_nonsme <- master_xra[`MacroSegmento_Attuale LC` != 'Sme Retail' & `MacroSegmento_Attuale LC` != "Altro", ..cols_2keep_xra ]

saveRDS(master_xra_sme,'../data/RDS/master/sme/master_xra.RDS')
saveRDS(master_xra_nonsme,'../data/RDS/master/non_sme/master_xra.RDS')

In [491]:
master_xra_nonsme

NUM_PRATICA,PROGRESS_PRATICA,XRA_media_sconfino_30gg,XRA_IMP_SCONFINO,XRA_IMP_UTIL_TOT,XRA_NUM_GG_SCONF
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
102230030,1,NA,NA,NA,NA
102230031,1,NA,NA,NA,NA
102230032,1,NA,NA,NA,NA
102230039,1,NA,-157563,-1501488,73
102230040,1,NA,-8413,-36144,88
103230023,1,NA,NA,NA,NA
103230030,2,NA,NA,NA,NA
103230032,1,NA,NA,NA,NA
103230035,1,NA,NA,NA,NA


In [492]:
#saveRDS(master_xra,'../data//RDS/master_xra.RDS')

In [493]:
table(master_xra$XRA_delta_t)

< table of extent 0 >

In [494]:
dt_xra[COD_SNDG == '0000000000070713' & DAT_FINE_PERIO < '2020-10-02' & DAT_FINE_PERIO >= '2020-09-02',]

DAT_FINE_PERIO,COD_SOC,COD_NSG,COD_SNDG,DAT_INI_SCF,NUM_GG_SCONF,IMP_ACR_DEL,IMP_ACR_OP,IMP_UTIL,IMP_UTIL_TOT,IMP_UTIL_CUM,IMP_SCONFINO,IMP_SCONFINO_CUM,COD_NDG_LEG,COD_ST_SEGN,COD_DIP_GRP,COD_SEGM,DAT_SEGNAL,IMP_UTIL_FIRMA
<date>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
2020-09-29,,,0000000000070713,2020-09-29,1,2406546,2406546,-2398015,-2398015,-2398015,-1006,-1006,0,,,SC,1600-01-01,0
2020-09-30,,,0000000000070713,2020-09-29,2,2406546,2406546,-2398267,-2398267,-2398267,-1258,-1258,0,,,SC,1600-01-01,0
2020-10-01,,,0000000000070713,2020-09-29,3,2406546,2406546,-2398267,-2398267,-2398267,-1258,-1258,0,,,SC,1600-01-01,0


### analisi duplicati 

In [495]:
dt_xra_da2019 <- readRDS('../data/RDS/dt_xra_da2019.RDS')
dt_xra_fino2019 <- readRDS('../data/RDS/dt_xra_fino2019.RDS')

In [496]:
max(dt_xra_fino2019$DAT_FINE_PERIO)
min_date <- min(dt_xra_da2019$DAT_FINE_PERIO)
min_date
dt_xra_fino2019[DAT_FINE_PERIO < min_date, ]

[1] "2021-05-24"

[1] "2019-11-05"

DAT_FINE_PERIO,COD_SOC,COD_NSG,COD_SNDG,DAT_INI_SCF,NUM_GG_SCONF,IMP_ACR_DEL,IMP_ACR_OP,IMP_UTIL,IMP_UTIL_TOT,⋯,COD_ST_SEGN,NUM_GG_SCONF_OLD,COD_DEFAULT_SET,COD_DIP_GRP,COD_SEGM,DAT_SEGNAL,IMP_UTIL_FIRMA,FLG_CPGRP,FLG_LEG,COD_NSG_PF
<date>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<dbl>,<chr>,<chr>,<chr>,<date>,<dbl>,<chr>,<chr>,<chr>
2015-12-31,,,0000000000011090,2015-12-31,1,38353,38353,-41047,-41047,⋯,S,0,IB,,SR,2015-12-31,0,N,N,0
2015-12-31,,,0000000000016527,2015-12-31,1,45000,10828,-12184,-12184,⋯,S,0,IB,,SR,2015-12-31,0,N,N,0
2015-12-31,,,0000000000149004,2015-06-01,214,0,0,-29056,-29056,⋯,C,0,IB,,SRMI,2015-11-30,0,N,N,0
2015-12-31,,,0000000000348562,2015-12-31,1,0,0,-226,-226,⋯,S,0,IB,,SRMI,2015-12-31,0,N,N,0
2015-12-31,,,0000000000428152,2015-12-31,1,293,293,-615,-615,⋯,S,0,IB,,SRMI,2015-12-31,0,N,N,0
2015-12-31,,,0000000000599423,2015-12-31,1,0,0,-13,-13,⋯,S,0,IB,,SRMI,2015-12-31,0,N,N,0
2015-12-31,,,0000000000664550,2015-12-31,1,30685,26584,-27074,-27074,⋯,,0,IB,,SRMI,1600-01-01,0,N,N,0
2015-12-31,,,0000000001059506,2015-12-31,1,245000,210047,-201394,-201394,⋯,,0,IB,,SR,1600-01-01,0,N,N,0
2015-12-31,,,0000000001063972,2015-12-31,1,15000,15000,-15319,-15319,⋯,,0,IB,,SRMI,1600-01-01,0,N,N,0


In [497]:
dt_xra_orig <- readRDS('../data/RDS/dt_xra.RDS')

In [498]:
colnames(dt_xra_orig)

[1] "DAT_FINE_PERIO"     "COD_SOC"            "COD_NSG"           
 [4] "COD_SNDG"           "DAT_INI_SCF"        "NUM_GG_SCONF"      
 [7] "IMP_ACR_DEL"        "IMP_ACR_OP"         "IMP_UTIL"          
[10] "IMP_UTIL_TOT"       "IMP_UTIL_CUM"       "IMP_SCONFINO"      
[13] "IMP_SCONFINO_CUM"   "IMP_UTIL_CUM_SEGN"  "IMP_SCONF_CUM_SEGN"
[16] "COD_NDG_LEG"        "COD_ST_SEGN"        "NUM_GG_SCONF_OLD"  
[19] "COD_SETTORE"        "COD_DIP_GRP"        "COD_SEGM"          
[22] "DAT_SEGNAL"         "IMP_UTIL_FIRMA"     "FLG_CPGRP"         
[25] "FLG_LEG"            "FLG_PD_TECN_1"      "FLG_PD_TECN_2"     
[28] "FLG_PD_TECN_3"      "FLG_PD_TECN_4"      "FLG_QUINTO_CASO"   
[31] "FLG_SOSPENSIONE"    "FLG_AZZERAMENTO"    "FLG_ENTE_PUBBLICO" 
[34] "COD_NSG_PF"

In [499]:
table(dt_xra_orig[DAT_SEGNAL < "1950-01-01",]$DAT_SEGNAL)


1600-01-01 
   1770041 

In [500]:
colnames(dt_xra_orig)
old_cols <- colnames(dt_xra_orig)[grepl("OLD",colnames(dt_xra_orig))]
cum_cols <- colnames(dt_xra_orig)[grepl("CUM",colnames(dt_xra_orig))]
cum_cols

[1] "DAT_FINE_PERIO"     "COD_SOC"            "COD_NSG"           
 [4] "COD_SNDG"           "DAT_INI_SCF"        "NUM_GG_SCONF"      
 [7] "IMP_ACR_DEL"        "IMP_ACR_OP"         "IMP_UTIL"          
[10] "IMP_UTIL_TOT"       "IMP_UTIL_CUM"       "IMP_SCONFINO"      
[13] "IMP_SCONFINO_CUM"   "IMP_UTIL_CUM_SEGN"  "IMP_SCONF_CUM_SEGN"
[16] "COD_NDG_LEG"        "COD_ST_SEGN"        "NUM_GG_SCONF_OLD"  
[19] "COD_SETTORE"        "COD_DIP_GRP"        "COD_SEGM"          
[22] "DAT_SEGNAL"         "IMP_UTIL_FIRMA"     "FLG_CPGRP"         
[25] "FLG_LEG"            "FLG_PD_TECN_1"      "FLG_PD_TECN_2"     
[28] "FLG_PD_TECN_3"      "FLG_PD_TECN_4"      "FLG_QUINTO_CASO"   
[31] "FLG_SOSPENSIONE"    "FLG_AZZERAMENTO"    "FLG_ENTE_PUBBLICO" 
[34] "COD_NSG_PF"

[1] "IMP_UTIL_CUM"       "IMP_SCONFINO_CUM"   "IMP_UTIL_CUM_SEGN" 
[4] "IMP_SCONF_CUM_SEGN"

In [501]:
cols_xra <- setdiff(colnames(dt_xra_orig),
            c('FLG_CPGRP','FLG_LEG','FLG_PD_TECN_1','FLG_PD_TECN_2','FLG_PD_TECN_3',#'DAT_SEGNAL',
              'FLG_PD_TECN_4','FLG_QUINTO_CASO','FLG_SOSPENSIONE',
              'FLG_AZZERAMENTO','FLG_ENTE_PUBBLICO','COD_NSG_PF',old_cols))#,cum_cols,old_cols


In [502]:
dt_xra<- unique(dt_xra_orig[,..cols_xra])

In [503]:
sub_dati_xra <- dt_orig[,
                    c('NUM_PRATICA','PROGRESS_PRATICA','SNDG_CEDUTO','DATA_DELIBERA_O_ANNULL_FIDO',
                      'MacroSegmento_Attuale LC')]


In [504]:
sub_dati_xra[,data_xra := as.Date(DATA_DELIBERA_O_ANNULL_FIDO)]
data_tot_xra <-merge(
    sub_dati_xra,
    dt_xra,
    by.x=c("SNDG_CEDUTO"),
    by.y=c("COD_SNDG"),
    allow.cartesian = TRUE,
    all.x=TRUE
)

In [505]:
 xra_max<-data_tot_xra[DAT_FINE_PERIO<=data_xra,
                                            .(dat_xra_max=max(
                                                DAT_FINE_PERIO,na.rm=TRUE)),
                                               by=c("SNDG_CEDUTO",'DATA_DELIBERA_O_ANNULL_FIDO')]

In [506]:
data_merge_2_xra<-unique(merge(
   sub_dati_xra,
   xra_max,
   by.x=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
   by.y=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
   all.x=TRUE
))

In [507]:
master_xra <- unique(merge(
    data_merge_2_xra,
    dt_xra,
    by.x=c("SNDG_CEDUTO","dat_xra_max"),
    by.y=c("COD_SNDG","DAT_FINE_PERIO"),
    all.x=TRUE
))

In [508]:
rbindlist(lapply(unique(master_xra$`MacroSegmento_Attuale LC`), function(seg){
    missing_val <- sum(is.na(master_xra[`MacroSegmento_Attuale LC`==seg]$IMP_SCONFINO))
    all_val <- nrow(master_xra[`MacroSegmento_Attuale LC`==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,71637,90419,79.23
Sme Corporate,21882,31616,69.21
Altro,9835,10494,93.72
Corporate,4252,6855,62.03
Large corporate,1480,3078,48.08


In [509]:
dt_xra[COD_SNDG == '0000000000012113',]

DAT_FINE_PERIO,COD_SOC,COD_NSG,COD_SNDG,DAT_INI_SCF,NUM_GG_SCONF,IMP_ACR_DEL,IMP_ACR_OP,IMP_UTIL,IMP_UTIL_TOT,⋯,IMP_SCONFINO_CUM,IMP_UTIL_CUM_SEGN,IMP_SCONF_CUM_SEGN,COD_NDG_LEG,COD_ST_SEGN,COD_SETTORE,COD_DIP_GRP,COD_SEGM,DAT_SEGNAL,IMP_UTIL_FIRMA
<date>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
2020-03-03,,,0000000000012113,2020-03-03,1,13132,11182,-6215,-6215,⋯,-33,-6215,-33,4456293,,IB,,SRMI,1600-01-01,0


In [510]:
nrow(dt_orig[SNDG_CEDUTO =='NESSUN_NDG',])

[1] 5

In [511]:
master_xra[is.na(IMP_SCONFINO),]

SNDG_CEDUTO,dat_xra_max,DATA_DELIBERA_O_ANNULL_FIDO,NUM_PRATICA,PROGRESS_PRATICA,MacroSegmento_Attuale LC,data_xra,COD_SOC,COD_NSG,DAT_INI_SCF,⋯,IMP_SCONFINO_CUM,IMP_UTIL_CUM_SEGN,IMP_SCONF_CUM_SEGN,COD_NDG_LEG,COD_ST_SEGN,COD_SETTORE,COD_DIP_GRP,COD_SEGM,DAT_SEGNAL,IMP_UTIL_FIRMA
<chr>,<date>,<chr>,<dbl>,<dbl>,<chr>,<date>,<chr>,<chr>,<date>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
0000000000007435,NA,2017-12-13-23.12.13.369581,71213230934,19,Sme Retail,2017-12-13,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000008626,NA,2020-03-17-23.05.24.295914,317230521,1,Sme Retail,2020-03-17,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000009554,NA,2018-06-26-23.17.27.178500,80626230257,18,Sme Retail,2018-06-26,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000011301,NA,2018-08-07-23.26.20.879021,80807230438,1,Sme Retail,2018-08-07,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000012113,NA,2019-05-06-23.27.19.168761,90506230835,3,Sme Retail,2019-05-06,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000015807,NA,2017-12-13-00.53.21.585212,71212231459,1,Sme Retail,2017-12-13,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000017350,NA,2017-02-20-14.00.26.472007,70117223063,1,Sme Retail,2017-02-20,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000018094,NA,2018-09-21-23.56.52.478582,80921230561,1,Sme Retail,2018-09-21,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000021074,NA,2016-08-23-22.31.43.998670,60823223051,1,Sme Retail,2016-08-23,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [512]:
dt_xra_orig <- readRDS('../data/RDS/dt_xra.RDS')

In [513]:
colnames(dt_xra_orig)

[1] "DAT_FINE_PERIO"     "COD_SOC"            "COD_NSG"           
 [4] "COD_SNDG"           "DAT_INI_SCF"        "NUM_GG_SCONF"      
 [7] "IMP_ACR_DEL"        "IMP_ACR_OP"         "IMP_UTIL"          
[10] "IMP_UTIL_TOT"       "IMP_UTIL_CUM"       "IMP_SCONFINO"      
[13] "IMP_SCONFINO_CUM"   "IMP_UTIL_CUM_SEGN"  "IMP_SCONF_CUM_SEGN"
[16] "COD_NDG_LEG"        "COD_ST_SEGN"        "NUM_GG_SCONF_OLD"  
[19] "COD_SETTORE"        "COD_DIP_GRP"        "COD_SEGM"          
[22] "DAT_SEGNAL"         "IMP_UTIL_FIRMA"     "FLG_CPGRP"         
[25] "FLG_LEG"            "FLG_PD_TECN_1"      "FLG_PD_TECN_2"     
[28] "FLG_PD_TECN_3"      "FLG_PD_TECN_4"      "FLG_QUINTO_CASO"   
[31] "FLG_SOSPENSIONE"    "FLG_AZZERAMENTO"    "FLG_ENTE_PUBBLICO" 
[34] "COD_NSG_PF"

In [514]:
table(dt_xra_orig[DAT_SEGNAL < "1950-01-01",]$DAT_SEGNAL)


1600-01-01 
   1770041 

In [515]:
colnames(dt_xra_orig)
old_cols <- colnames(dt_xra_orig)[grepl("OLD",colnames(dt_xra_orig))]
cum_cols <- colnames(dt_xra_orig)[grepl("CUM",colnames(dt_xra_orig))]
cum_cols

[1] "DAT_FINE_PERIO"     "COD_SOC"            "COD_NSG"           
 [4] "COD_SNDG"           "DAT_INI_SCF"        "NUM_GG_SCONF"      
 [7] "IMP_ACR_DEL"        "IMP_ACR_OP"         "IMP_UTIL"          
[10] "IMP_UTIL_TOT"       "IMP_UTIL_CUM"       "IMP_SCONFINO"      
[13] "IMP_SCONFINO_CUM"   "IMP_UTIL_CUM_SEGN"  "IMP_SCONF_CUM_SEGN"
[16] "COD_NDG_LEG"        "COD_ST_SEGN"        "NUM_GG_SCONF_OLD"  
[19] "COD_SETTORE"        "COD_DIP_GRP"        "COD_SEGM"          
[22] "DAT_SEGNAL"         "IMP_UTIL_FIRMA"     "FLG_CPGRP"         
[25] "FLG_LEG"            "FLG_PD_TECN_1"      "FLG_PD_TECN_2"     
[28] "FLG_PD_TECN_3"      "FLG_PD_TECN_4"      "FLG_QUINTO_CASO"   
[31] "FLG_SOSPENSIONE"    "FLG_AZZERAMENTO"    "FLG_ENTE_PUBBLICO" 
[34] "COD_NSG_PF"

[1] "IMP_UTIL_CUM"       "IMP_SCONFINO_CUM"   "IMP_UTIL_CUM_SEGN" 
[4] "IMP_SCONF_CUM_SEGN"

In [516]:
cols_xra <- setdiff(colnames(dt_xra_orig),
            c('FLG_CPGRP','FLG_LEG','FLG_PD_TECN_1','FLG_PD_TECN_2','FLG_PD_TECN_3',#'DAT_SEGNAL',
              'FLG_PD_TECN_4','FLG_QUINTO_CASO','FLG_SOSPENSIONE',
              'FLG_AZZERAMENTO','FLG_ENTE_PUBBLICO','COD_NSG_PF',old_cols))#,cum_cols,old_cols


In [517]:
dt_xra<- unique(dt_xra_orig[,..cols_xra])

In [518]:
sub_dati_xra <- dt_orig[,
                    c('NUM_PRATICA','PROGRESS_PRATICA','SNDG_CEDUTO','DATA_DELIBERA_O_ANNULL_FIDO',
                      'MacroSegmento_Attuale LC')]


In [519]:
sub_dati_xra[,data_xra := as.Date(DATA_DELIBERA_O_ANNULL_FIDO)]
data_tot_xra <-merge(
    sub_dati_xra,
    dt_xra,
    by.x=c("SNDG_CEDUTO"),
    by.y=c("COD_SNDG"),
    allow.cartesian = TRUE,
    all.x=TRUE
)

In [520]:
 xra_max<-data_tot_xra[DAT_FINE_PERIO<=data_xra,
                                            .(dat_xra_max=max(
                                                DAT_FINE_PERIO,na.rm=TRUE)),
                                               by=c("SNDG_CEDUTO",'DATA_DELIBERA_O_ANNULL_FIDO')]

In [521]:
data_merge_2_xra<-unique(merge(
   sub_dati_xra,
   xra_max,
   by.x=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
   by.y=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
   all.x=TRUE
))

In [522]:
master_xra <- unique(merge(
    data_merge_2_xra,
    dt_xra,
    by.x=c("SNDG_CEDUTO","dat_xra_max"),
    by.y=c("COD_SNDG","DAT_FINE_PERIO"),
    all.x=TRUE
))

In [523]:
rbindlist(lapply(unique(master_xra$`MacroSegmento_Attuale LC`), function(seg){
    missing_val <- sum(is.na(master_xra[`MacroSegmento_Attuale LC`==seg]$IMP_SCONFINO))
    all_val <- nrow(master_xra[`MacroSegmento_Attuale LC`==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,71637,90419,79.23
Sme Corporate,21882,31616,69.21
Altro,9835,10494,93.72
Corporate,4252,6855,62.03
Large corporate,1480,3078,48.08


In [524]:
dt_xra[COD_SNDG == '0000000000012113',]

DAT_FINE_PERIO,COD_SOC,COD_NSG,COD_SNDG,DAT_INI_SCF,NUM_GG_SCONF,IMP_ACR_DEL,IMP_ACR_OP,IMP_UTIL,IMP_UTIL_TOT,⋯,IMP_SCONFINO_CUM,IMP_UTIL_CUM_SEGN,IMP_SCONF_CUM_SEGN,COD_NDG_LEG,COD_ST_SEGN,COD_SETTORE,COD_DIP_GRP,COD_SEGM,DAT_SEGNAL,IMP_UTIL_FIRMA
<date>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
2020-03-03,,,0000000000012113,2020-03-03,1,13132,11182,-6215,-6215,⋯,-33,-6215,-33,4456293,,IB,,SRMI,1600-01-01,0


In [525]:
nrow(dt_orig[SNDG_CEDUTO =='NESSUN_NDG',])

[1] 5

In [526]:
master_xra[is.na(IMP_SCONFINO),]

SNDG_CEDUTO,dat_xra_max,DATA_DELIBERA_O_ANNULL_FIDO,NUM_PRATICA,PROGRESS_PRATICA,MacroSegmento_Attuale LC,data_xra,COD_SOC,COD_NSG,DAT_INI_SCF,⋯,IMP_SCONFINO_CUM,IMP_UTIL_CUM_SEGN,IMP_SCONF_CUM_SEGN,COD_NDG_LEG,COD_ST_SEGN,COD_SETTORE,COD_DIP_GRP,COD_SEGM,DAT_SEGNAL,IMP_UTIL_FIRMA
<chr>,<date>,<chr>,<dbl>,<dbl>,<chr>,<date>,<chr>,<chr>,<date>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>
0000000000007435,NA,2017-12-13-23.12.13.369581,71213230934,19,Sme Retail,2017-12-13,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000008626,NA,2020-03-17-23.05.24.295914,317230521,1,Sme Retail,2020-03-17,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000009554,NA,2018-06-26-23.17.27.178500,80626230257,18,Sme Retail,2018-06-26,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000011301,NA,2018-08-07-23.26.20.879021,80807230438,1,Sme Retail,2018-08-07,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000012113,NA,2019-05-06-23.27.19.168761,90506230835,3,Sme Retail,2019-05-06,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000015807,NA,2017-12-13-00.53.21.585212,71212231459,1,Sme Retail,2017-12-13,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000017350,NA,2017-02-20-14.00.26.472007,70117223063,1,Sme Retail,2017-02-20,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000018094,NA,2018-09-21-23.56.52.478582,80921230561,1,Sme Retail,2018-09-21,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
0000000000021074,NA,2016-08-23-22.31.43.998670,60823223051,1,Sme Retail,2016-08-23,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [528]:
tab_DAT_SEGNAL <- dt_xra[,list(DAT_SEGNAL_DEF = max(DAT_SEGNAL),DAT_SEGNAL=DAT_SEGNAL), by= c("COD_SNDG","DAT_FINE_PERIO")]

dt_xra <- merge(dt_xra,tab_DAT_SEGNAL, all.x = TRUE, by= c("COD_SNDG","DAT_FINE_PERIO",'DAT_SEGNAL'))

dt_xra <- unique(dt_xra[DAT_SEGNAL_DEF == DAT_SEGNAL,])

In [529]:
dt_xra_red <- dt_xra[,c("COD_SNDG","DAT_FINE_PERIO",'IMP_UTIL_TOT','IMP_SCONFINO','NUM_GG_SCONF')]

In [530]:
sndg_dupl <- dt_xra[,list(n = .N), by=c('DAT_FINE_PERIO','COD_SNDG')][n>1]$COD_SNDG


#head(unique(sndg_dupl),40)
tab_xra_dup <- merge(dt_xra[,list(n = .N), by=c('DAT_FINE_PERIO','COD_SNDG')][n>1],
                     dt_xra,
                     by = c('DAT_FINE_PERIO','COD_SNDG'),
                     all.x = TRUE)
tab_xra_dup_nonseg <- merge(dt_xra[,list(n = .N), by=c('DAT_FINE_PERIO','COD_SNDG','COD_SEGM')][n>1],
                     dt_xra,
                     by = c('DAT_FINE_PERIO','COD_SNDG','COD_SEGM'),
                     all.x = TRUE)

In [531]:
tab_xra_dup_nonseg

DAT_FINE_PERIO,COD_SNDG,COD_SEGM,n,DAT_SEGNAL,COD_SOC,COD_NSG,DAT_INI_SCF,NUM_GG_SCONF,IMP_ACR_DEL,⋯,IMP_SCONFINO,IMP_SCONFINO_CUM,IMP_UTIL_CUM_SEGN,IMP_SCONF_CUM_SEGN,COD_NDG_LEG,COD_ST_SEGN,COD_SETTORE,COD_DIP_GRP,IMP_UTIL_FIRMA,DAT_SEGNAL_DEF
<date>,<chr>,<chr>,<int>,<date>,<chr>,<chr>,<date>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<date>
2019-11-05,0000000000011090,SR,2,2019-11-05,,,2019-11-04,2,27798,⋯,-1644,-1644,-29422,-1644,4102821,S,IB,,0,2019-11-05
2019-11-05,0000000000011090,SR,2,2019-11-05,,,2019-11-04,2,27798,⋯,-1624,-1694,-59870,-1694,4102821,S,IB,,0,2019-11-05
2019-11-05,0000000000011301,SR,2,1600-01-01,,,2019-10-16,21,13724,⋯,-39,-39,-13763,-39,10106857,,IB,,0,1600-01-01
2019-11-05,0000000000011301,SR,2,1600-01-01,,,2019-10-16,21,13724,⋯,-39,-4092,-292296,-4092,10106857,,IB,,0,1600-01-01
2019-11-05,0000000000094026,SRMI,2,1600-01-01,,,2019-11-04,2,8391,⋯,-6,-6,-8397,-6,86435,,IB,,0,1600-01-01
2019-11-05,0000000000094026,SRMI,2,1600-01-01,,,2019-11-04,2,8391,⋯,-6,-12,-17211,-12,86435,,IB,,0,1600-01-01
2019-11-05,0000000000105782,SR,2,1600-01-01,,,2019-10-31,6,142528,⋯,-35,-35,-142563,-35,55112339,,IB,,0,1600-01-01
2019-11-05,0000000000105782,SR,2,1600-01-01,,,2019-10-31,6,142528,⋯,-35,-14382,-869550,-14382,55112339,,IB,,0,1600-01-01
2019-11-05,0000000000210415,SR,2,1600-01-01,,,2019-10-30,7,14638,⋯,-44,-44,-14682,-44,66814282,,IB,,0,1600-01-01


In [532]:
100*nrow(dt_xra_red[,list(n = .N), by=c('DAT_FINE_PERIO','COD_SNDG')][n>1])/nrow(dt_xra_red)

[1] 6.218065

In [533]:
dt_xra_orig[,list(n = .N), by=c('DAT_FINE_PERIO','COD_SNDG')][n>1]

DAT_FINE_PERIO,COD_SNDG,n
<date>,<chr>,<int>
2021-04-01,0000000000008626,2
2021-04-02,0000000000008626,2
2021-04-06,0000000000008626,2
2021-04-07,0000000000008626,2
2021-04-08,0000000000008626,2
2021-04-09,0000000000008626,2
2020-04-03,0000000000009972,2
2020-04-06,0000000000009972,2
2020-04-07,0000000000009972,2


In [534]:
length(sndg_dupl)
nrow(tab_xra_dup)

[1] 271523

[1] 543046

In [535]:

100*(nrow(unique(dt_xra_orig[,c('DAT_FINE_PERIO','COD_SNDG','DAT_SEGNAL')])) - nrow(unique(dt_xra_orig[,c('DAT_FINE_PERIO','COD_SNDG')])))/nrow(dt_xra_orig)

100*(nrow(unique(dt_xra_orig[,c('DAT_FINE_PERIO','COD_SNDG','IMP_SCONFINO')])) - nrow(unique(dt_xra_orig[,c('DAT_FINE_PERIO','COD_SNDG')])))/nrow(dt_xra_orig)

100*(nrow(unique(dt_xra_orig[,c('DAT_FINE_PERIO','COD_SNDG','IMP_UTIL_TOT')])) - nrow(unique(dt_xra_orig[,c('DAT_FINE_PERIO','COD_SNDG')])))/nrow(dt_xra_orig)

100*(nrow(unique(dt_xra_orig[,c('DAT_FINE_PERIO','COD_SNDG','IMP_SCONFINO_CUM')])) - nrow(unique(dt_xra_orig[,c('DAT_FINE_PERIO','COD_SNDG')])))/nrow(dt_xra_orig)

[1] 14.61354

[1] 2.619466

[1] 0.8481252

[1] 18.70766

In [536]:
100*(nrow(unique(dt_xra_orig[,c('DAT_FINE_PERIO','COD_SNDG','NUM_GG_SCONF','IMP_SCONFINO')])) - nrow(unique(dt_xra_orig[,c('DAT_FINE_PERIO','COD_SNDG')])))/nrow(dt_xra_orig)

100*(nrow(unique(dt_xra_orig[,c('DAT_FINE_PERIO','COD_SNDG','DAT_INI_SCF')])) - nrow(unique(dt_xra_orig[,c('DAT_FINE_PERIO','COD_SNDG')])))/nrow(dt_xra_orig)


[1] 4.48752

[1] 2.862106

In [537]:
table(year(tab_xra_dup_nonseg$DAT_SEGNAL))
table(year(dt_xra$DAT_SEGNAL))


  1600   2019   2020   2021 
233488  40900 137510  34132 


   1600    2011    2012    2013    2014    2015    2016    2017    2018    2019 
1476844    1072     483    2771    5272   14028  234816  345322  506082  652143 
   2020    2021    2022 
 788492  332519    6836 

In [538]:
dt_xra[COD_SNDG == '0000000000348562' & DAT_FINE_PERIO == '2019-11-05',]

COD_SNDG,DAT_FINE_PERIO,DAT_SEGNAL,COD_SOC,COD_NSG,DAT_INI_SCF,NUM_GG_SCONF,IMP_ACR_DEL,IMP_ACR_OP,IMP_UTIL,⋯,IMP_SCONFINO_CUM,IMP_UTIL_CUM_SEGN,IMP_SCONF_CUM_SEGN,COD_NDG_LEG,COD_ST_SEGN,COD_SETTORE,COD_DIP_GRP,COD_SEGM,IMP_UTIL_FIRMA,DAT_SEGNAL_DEF
<chr>,<date>,<date>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<date>
0000000000348562,2019-11-05,1600-01-01,,,2019-11-04,2,70964,70964,-71384,⋯,-706,-71384,-706,6011793,,IB,,SR,0,1600-01-01
0000000000348562,2019-11-05,1600-01-01,,,2019-11-05,1,70964,70964,-71384,⋯,-420,-71384,-420,6011793,,IB,,SR,0,1600-01-01


In [539]:
table(tab_xra_dup_nonseg$DAT_FINE_PERIO)


2019-11-05 2019-11-06 2019-11-07 2019-11-08 2019-11-11 2019-11-12 2019-11-13 
      1550       1510       1450       1654       1644       1220       1362 
2019-11-14 2019-11-15 2019-11-18 2019-11-19 2019-11-20 2019-11-21 2019-11-22 
      1312       1416       1606       1156       1396       1326       1320 
2019-11-25 2019-11-26 2019-11-27 2019-11-28 2019-11-29 2019-12-02 2019-12-03 
      1180       1160       1236       1426       1460       1616       1764 
2019-12-04 2019-12-05 2019-12-06 2019-12-09 2019-12-10 2019-12-11 2019-12-12 
      1742       1638       1554       1636       1264       1274       1278 
2019-12-13 2019-12-16 2019-12-17 2019-12-18 2019-12-19 2019-12-20 2019-12-23 
      1336        960       1064       1024       1318        970        858 
2019-12-24 2019-12-27 2019-12-30 2019-12-31 2020-01-02 2020-01-03 2020-01-07 
       890        876        884       1116       2132       2040       1648 
2020-01-08 2020-01-09 2020-01-10 2020-01-13 2020-01-14 2020-01-

In [540]:
# write.xlsx(head(tab_xra_dup_nonseg,1000),
#            sheetName ='duplicati',
#            'duplicati_xra.xlsx',
#            row.names = FALSE,
#           append = FALSE)
# write.xlsx( setdiff(colnames(dt_xra_orig), cols_xra),
#            sheetName = "colonne_rimosse",
#             'duplicati_xra.xlsx',
#            row.names = FALSE,
#           append = TRUE)


## EWS TO-BE

In [541]:
dt_ews_tobe<-readRDS("../data/dati_ews/master_ews.RDS")



In [542]:
sub_dati<-dt_orig[,.(SNDG_CEDUTO,DATA_DELIBERA_O_ANNULL_FIDO,`MacroSegmento_Attuale LC`,NUM_PRATICA,PROGRESS_PRATICA)]

# sub_dati[,DATA_DELIBERA_O_ANNULL_FIDO_orig:=DATA_DELIBERA_O_ANNULL_FIDO]
# sub_dati[,DATA_DELIBERA_O_ANNULL_FIDO:=as.Date(DATA_DELIBERA_O_ANNULL_FIDO)]

In [543]:
#definisco anno rif. per i valori ews da considerare 
sub_dati[,DATA_RIF_EWS:=as.Date(ifelse(
    DATA_DELIBERA_O_ANNULL_FIDO<
        paste(
            sep="-",
            year(DATA_DELIBERA_O_ANNULL_FIDO),"06-30"
        ),
        paste(
            sep="-",
            year(DATA_DELIBERA_O_ANNULL_FIDO)-1,"12-31"
        ),
        paste(
            sep="-",
            year(DATA_DELIBERA_O_ANNULL_FIDO),"06-30"
        )
    ))                         
]

In [544]:
#attacco tutti i dati ews per ogni SDNG
data_tot<-merge(
    sub_dati,
    dt_ews_tobe,
    by.x=c("SNDG_CEDUTO"),
    by.y=c("SNDG_CEDUTO"),
    all.x=TRUE
)

In [545]:
#considero dati ews dell'ultima data disponibile minore alla data rif. calcolata da data delibera
ews_max<-data_tot[d_riferimento<=DATA_RIF_EWS,
                      .(d_riferimento_max=max(
                          d_riferimento,na.rm=TRUE
                      )),by=c("SNDG_CEDUTO",'DATA_DELIBERA_O_ANNULL_FIDO')]

ews_max[d_riferimento_max >DATA_DELIBERA_O_ANNULL_FIDO, ]

SNDG_CEDUTO,DATA_DELIBERA_O_ANNULL_FIDO,d_riferimento_max
<chr>,<chr>,<date>


In [546]:
nrow(unique(dt_orig))

[1] 140108

In [547]:
data_merge_2<-merge(
    sub_dati,
    ews_max,
    by.x=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    by.y=c("SNDG_CEDUTO","DATA_DELIBERA_O_ANNULL_FIDO"),
    all.x=TRUE
)

In [548]:
data_merge_filt <- data_merge_2[DATA_DELIBERA_O_ANNULL_FIDO >=min(dt_ews_tobe$d_riferimento), ]

In [549]:
##solo sui periodi comuni

rbindlist(lapply(unique(data_merge_filt$`MacroSegmento_Attuale LC`), function(seg){
    missing_val <- sum(is.na(data_merge_filt[`MacroSegmento_Attuale LC`==seg]$d_riferimento_max))
    all_val <- nrow(data_merge_filt[`MacroSegmento_Attuale LC`==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,45481,66341,68.56
Sme Corporate,5618,21170,26.54
Altro,5118,5713,89.59
Corporate,907,4899,18.51
Large corporate,255,2237,11.40


In [550]:


master_ews_tobe <- merge(
    data_merge_2,
    dt_ews_tobe,
    by.x=c("SNDG_CEDUTO","d_riferimento_max"),
    by.y=c("SNDG_CEDUTO","d_riferimento"),
    all.x=TRUE
)

In [551]:
table(master_ews_tobe[year(DATA_DELIBERA_O_ANNULL_FIDO) == '2020',pred], useNA = 'always')


 0.00010696730169002  0.00010763051250251 0.000110093867988326 
                   3                    1                    9 
0.000111644782009535   0.0001132499310188 0.000113764595880639 
                   8                    1                    1 
 0.00011412021558499  0.00011493841157062 0.000115935152280144 
                  10                    2                    1 
 0.00011623147292994 0.000117404655611608 0.000118808595289011 
                  63                   13                   16 
0.000120027587399818  0.00012143692583777 0.000121531571494415 
                  10                   81                    2 
0.000122061755973846 0.000123343517770991 0.000124301077448763 
                   1                    3                    5 
0.000124378260807134 0.000125555699924007 0.000127111736219376 
                   2                    1                    1 
0.000128804793348536 0.000128866464365274 0.000129243402625434 
                  18                   

In [552]:
cols_2keep_ews2be <- colnames(master_ews_tobe)[grepl('pred',colnames(master_ews_tobe))]

In [553]:
cols_2keep_ews2be

[1] "pred_xra"       "pred_aris"      "pred_afi"       "pred_cashflow" 
 [5] "pred_crsys"     "pred_cr0"       "pred_nopg"      "pred_regexp"   
 [9] "pred_cartacomm" "pred_prea"      "pred_bilancio"  "pred"

In [554]:
master_ews_nonsme <- master_ews_tobe[`MacroSegmento_Attuale LC` != "Sme Retail" & `MacroSegmento_Attuale LC` != "Altro",c('NUM_PRATICA','PROGRESS_PRATICA',cols_2keep_ews2be), with = FALSE]

In [555]:
#master_ews_nonsme
colnames(master_ews_nonsme)[grepl('pred',colnames(master_ews_nonsme))] <- paste0("EWS_",cols_2keep_ews2be)

In [556]:
saveRDS(master_ews_nonsme,'../data/RDS/master/non_sme/master_ews.RDS')

## EWS AS-IS

In [724]:
dt_ews_asis <- readRDS('../data/dt_ews_asis.RDS')

In [725]:
master_ews_asis <- merge(dt_orig[,c("NUM_PRATICA","PROGRESS_PRATICA")],
                        dt_ews_asis,
                        by = c("NUM_PRATICA","PROGRESS_PRATICA"),
                        all.x = TRUE)

In [726]:
master_ews_asis[Giorni_dist_Ews_Delibera < -30 & Giorni_dist_Ews_Delibera > -90,flag_missing := 1]

In [728]:
table(master_ews_asis$flag_missing,useNA = 'always')/nrow(master_ews_asis)


        1      <NA> 
0.1899035 0.8100965 

In [700]:
master_ews_asis[Giorni_dist_Ews_Delibera > -30 & Giorni_dist_Ews_Delibera < 90,flag_90gg := 1]
master_ews_asis[Giorni_dist_Ews_Delibera > -30 & Giorni_dist_Ews_Delibera < 90,flag_90gg := 1]
master_ews_asis[Giorni_dist_Ews_Delibera > -30,flag_30gg := 1]
master_ews_asis[Giorni_dist_Ews_Delibera > -10,flag_10gg := 1]

In [701]:
colnames(master_ews_asis)

[1] "NUM_PRATICA"                 "PROGRESS_PRATICA"           
 [3] "SNDG_CEDUTO"                 "ESITO_FINALE_VPC"           
 [5] "DATA_DELIBERA_O_ANNULL_FIDO" "Delibera_Numerica"          
 [7] "MacroSegmento_Attuale"       "Codice_Ews"                 
 [9] "Colore.Ews"                  "Data_Ews"                   
[11] "Giorni_dist_Ews_Delibera"    "flag_90gg"                  
[13] "flag_30gg"                   "flag_10gg"

In [702]:
table(master_ews_asis$Codice_Ews, useNA = 'always')
table(master_ews_asis$Colore.Ews, useNA = 'always')


  Ews non Valido Non presente ews              S01              S10 
             842            13871           108579             6964 
             S20              S30              S40              S50 
            5181             3396              213             1062 
            <NA> 
               0 


   Arancio Blu chiaro  Blu scuro      Rosso      Verde    Verdino       <NA> 
      5181        213       1062       3396     108579       6964      14713 

In [703]:
nrow(master_ews_asis[Giorni_dist_Ews_Delibera < -10,])
nrow(master_ews_asis)

[1] 92933

[1] 140108

In [704]:
table(master_ews_asis$Giorni_dist_Ews_Delibera, useNA = 'always')


-1971 -1965 -1963 -1959 -1953 -1952 -1938 -1937 -1936 -1935 -1929 -1926 -1924 
    1     1     1     1     2     1     1     2     1     1     1     1     1 
-1915 -1911 -1910 -1908 -1904 -1903 -1886 -1884 -1880 -1878 -1868 -1864 -1863 
    2     1     1     1     1     1     1     1     1     1     1     1     2 
-1855 -1839 -1836 -1834 -1830 -1827 -1825 -1823 -1822 -1818 -1814 -1812 -1808 
    1     1     1     1     1     1     1     1     1     1     1     1     2 
-1806 -1804 -1798 -1797 -1788 -1781 -1776 -1772 -1769 -1762 -1761 -1756 -1755 
    1     1     2     1     1     1     3     2     1     1     1     1     1 
-1754 -1749 -1745 -1731 -1723 -1721 -1718 -1704 -1700 -1698 -1693 -1692 -1687 
    1     1     1     1     1     1     1     1     1     2     2     1     2 
-1684 -1682 -1681 -1679 -1678 -1676 -1671 -1666 -1662 -1659 -1658 -1657 -1656 
    1     1     2     1     2     2     3     1     1     2     1     1     2 
-1655 -1651 -1647 -1646 -1640 -1634 -1631 -1629 -16

In [705]:
master_ews_asis[MacroSegmento_Attuale == 'Sme Retail' & Giorni_dist_Ews_Delibera < -60,]

NUM_PRATICA,PROGRESS_PRATICA,SNDG_CEDUTO,ESITO_FINALE_VPC,DATA_DELIBERA_O_ANNULL_FIDO,Delibera_Numerica,MacroSegmento_Attuale,Codice_Ews,Colore.Ews,Data_Ews,Giorni_dist_Ews_Delibera,flag_90gg,flag_30gg,flag_10gg
<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
102230029,2,0000000088606666,VERDE,2020-01-02-23.04.55.267865,43832,Sme Retail,S01,Verde,43921,-89,NA,NA,NA
102230035,1,0000000017605835,VERDE,2020-01-02-23.04.55.267865,43832,Sme Retail,S01,Verde,43921,-89,NA,NA,NA
107230030,2,0000000070119127,VERDE,2020-01-29-16.00.38.325521,43859,Sme Retail,S01,Verde,43921,-62,NA,NA,NA
108230028,1,0000000016528507,VERDE,2020-01-29-15.30.35.070281,43859,Sme Retail,S10,Verdino,43921,-62,NA,NA,NA
109230031,1,0000000082195366,GIALLO,2020-04-14-17.30.21.477410,43935,Sme Retail,S01,Verde,44012,-77,NA,NA,NA
109230041,1,0000000082517402,VERDE,2020-01-09-23.01.53.521036,43839,Sme Retail,S01,Verde,43921,-82,NA,NA,NA
109230056,1,0000000045963350,GIALLO,2020-02-13-17.00.32.708020,43874,Sme Retail,S01,Verde,44104,-230,NA,NA,NA
110230053,1,0000000025290054,VERDE,2020-01-10-23.02.44.200751,43840,Sme Retail,S01,Verde,43921,-81,NA,NA,NA
110230054,1,0000000090733103,VERDE,2020-01-10-23.02.44.200751,43840,Sme Retail,S01,Verde,43921,-81,NA,NA,NA


In [706]:
table(master_ews_asis[Giorni_dist_Ews_Delibera < -10,]$Giorni_dist_Ews_Delibera)


-1971 -1965 -1963 -1959 -1953 -1952 -1938 -1937 -1936 -1935 -1929 -1926 -1924 
    1     1     1     1     2     1     1     2     1     1     1     1     1 
-1915 -1911 -1910 -1908 -1904 -1903 -1886 -1884 -1880 -1878 -1868 -1864 -1863 
    2     1     1     1     1     1     1     1     1     1     1     1     2 
-1855 -1839 -1836 -1834 -1830 -1827 -1825 -1823 -1822 -1818 -1814 -1812 -1808 
    1     1     1     1     1     1     1     1     1     1     1     1     2 
-1806 -1804 -1798 -1797 -1788 -1781 -1776 -1772 -1769 -1762 -1761 -1756 -1755 
    1     1     2     1     1     1     3     2     1     1     1     1     1 
-1754 -1749 -1745 -1731 -1723 -1721 -1718 -1704 -1700 -1698 -1693 -1692 -1687 
    1     1     1     1     1     1     1     1     1     2     2     1     2 
-1684 -1682 -1681 -1679 -1678 -1676 -1671 -1666 -1662 -1659 -1658 -1657 -1656 
    1     1     2     1     2     2     3     1     1     2     1     1     2 
-1655 -1651 -1647 -1646 -1640 -1634 -1631 -1629 -16

In [707]:
rbindlist(lapply(unique(master_ews_asis$MacroSegmento_Attuale), function(seg){
    missing_val <- sum(is.na(master_ews_asis[MacroSegmento_Attuale==seg]$Colore.Ews))
    all_val <- nrow(master_ews_asis[MacroSegmento_Attuale==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,9749,88631,11.00
Sme Corporate,1453,31396,4.63
Corporate,411,9536,4.31
Altro,3100,10545,29.40


In [708]:
nrow(master_ews_asis[Giorni_dist_Ews_Delibera < -10, ])
nrow(master_ews_asis[Giorni_dist_Ews_Delibera < -30, ])

[1] 92933

[1] 79388

In [709]:

rbindlist(lapply(unique(master_ews_asis$MacroSegmento_Attuale), function(seg){
    missing_val <- sum(is.na(master_ews_asis[MacroSegmento_Attuale==seg]$flag_90gg))
    all_val <- nrow(master_ews_asis[MacroSegmento_Attuale==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,67851,88631,76.55
Sme Corporate,14885,31396,47.41
Corporate,3245,9536,34.03
Altro,9464,10545,89.75


In [710]:
table(master_ews_asis[,Giorni_dist_Ews_Delibera ], useNA = 'always')


-1971 -1965 -1963 -1959 -1953 -1952 -1938 -1937 -1936 -1935 -1929 -1926 -1924 
    1     1     1     1     2     1     1     2     1     1     1     1     1 
-1915 -1911 -1910 -1908 -1904 -1903 -1886 -1884 -1880 -1878 -1868 -1864 -1863 
    2     1     1     1     1     1     1     1     1     1     1     1     2 
-1855 -1839 -1836 -1834 -1830 -1827 -1825 -1823 -1822 -1818 -1814 -1812 -1808 
    1     1     1     1     1     1     1     1     1     1     1     1     2 
-1806 -1804 -1798 -1797 -1788 -1781 -1776 -1772 -1769 -1762 -1761 -1756 -1755 
    1     1     2     1     1     1     3     2     1     1     1     1     1 
-1754 -1749 -1745 -1731 -1723 -1721 -1718 -1704 -1700 -1698 -1693 -1692 -1687 
    1     1     1     1     1     1     1     1     1     2     2     1     2 
-1684 -1682 -1681 -1679 -1678 -1676 -1671 -1666 -1662 -1659 -1658 -1657 -1656 
    1     1     2     1     2     2     3     1     1     2     1     1     2 
-1655 -1651 -1647 -1646 -1640 -1634 -1631 -1629 -16

In [711]:
master_ews_asis[,EWS_ASIS := Colore.Ews]
master_ews_asis[is.na(flag_90gg)  ,EWS_ASIS := NA]

In [712]:
master_ews_asis[,censito := "NO"]
master_ews_asis[!is.na(Colore.Ews),censito := "SI"]

In [713]:
master_ews_asis_sme <- master_ews_asis[MacroSegmento_Attuale == "Sme Retail",
                                       c('NUM_PRATICA','PROGRESS_PRATICA','EWS_ASIS','censito','Giorni_dist_Ews_Delibera')]

# master_ews_asis_nonsme <- master_ews_asis[MacroSegmento_Attuale != "Sme Retail",
#                                        c('NUM_PRATICA','PROGRESS_PRATICA','EWS_ASIS')]

In [714]:
table(master_ews_asis_sme$EWS_ASIS)


   Arancio Blu chiaro  Blu scuro      Rosso      Verde    Verdino 
      1156         27         21        756      17712       1108 

In [720]:
master_ews_asis_sme[EWS_ASIS== 'Verde',EWS_ASIS := 0]
master_ews_asis_sme[EWS_ASIS== 'Verdino',EWS_ASIS := 1]
master_ews_asis_sme[EWS_ASIS== 'Arancio',EWS_ASIS := 2]
master_ews_asis_sme[EWS_ASIS== 'Rosso',EWS_ASIS := 3]
master_ews_asis_sme[EWS_ASIS== 'Blu chiaro',EWS_ASIS := 4]
master_ews_asis_sme[EWS_ASIS== 'Blu scuro',EWS_ASIS := 5]
master_ews_asis_sme[,EWS_ASIS := as.numeric(EWS_ASIS)]

In [721]:
str(master_ews_asis_sme$EWS_ASIS)

 num [1:88631] 0 NA 0 NA NA 0 0 0 0 0 ...


In [722]:
nrow(master_ews_asis_sme[is.na(EWS_ASIS)])

[1] 67851

In [723]:
saveRDS(master_ews_asis_sme,'../data/RDS/master/sme/master_ews.RDS')
#saveRDS(master_ews_asis_nonsme,'../data/RDS/master/non_sme/master_bil.RDS')

## FORBORNE

In [575]:
dt_fb <- readRDS('../data/RDS/dt_forborne.RDS')

In [576]:
sub_dati<-dt_orig[,.(SNDG_CEDUTO,DATA_DELIBERA_O_ANNULL_FIDO,`MacroSegmento_Attuale LC`,NUM_PRATICA,PROGRESS_PRATICA)]

sub_dati[year(DATA_DELIBERA_O_ANNULL_FIDO) != 2019,data_fb := as.Date(paste(
            sep="-",
            year(DATA_DELIBERA_O_ANNULL_FIDO)-1,"12-31"
        ))]
sub_dati[year(DATA_DELIBERA_O_ANNULL_FIDO) == 2019,data_fb := as.Date(paste(
            sep="-",
            year(DATA_DELIBERA_O_ANNULL_FIDO)-1,"06-30"
        ))]

master_fb <- unique(merge(sub_dati,
                   dt_fb,
                   by.x = c("SNDG_CEDUTO",'data_fb'),
                   by.y = c("COD_SNDG",'PDO_FRB'),
                   all.x =TRUE)
                   )

In [577]:
table(master_fb$data_fb)


2014-12-31 2015-12-31 2016-12-31 2017-12-31 2018-06-30 2019-12-31 2020-12-31 
        29      17091      22628      46535      34652      18817        356 

In [578]:
table(master_fb[year(DATA_DELIBERA_O_ANNULL_FIDO) %in% unique(year(dt_fb$PDO_FRB)),]$FORB_PRESENZA)


  1 
333 

In [579]:
master_fb[year(DATA_DELIBERA_O_ANNULL_FIDO) %in% unique(year(dt_fb$PDO_FRB)) &is.na(FORB_PRESENZA)  , FORB_PRESENZA := 0]

In [580]:
rbindlist(lapply(unique(master_fb$`MacroSegmento_Attuale LC`), function(seg){
    missing_val <- sum(is.na(master_fb[`MacroSegmento_Attuale LC`==seg]$FORB_PRESENZA))
    all_val <- nrow(master_fb[`MacroSegmento_Attuale LC`==seg])
    perc <- 100*missing_val/all_val
    return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
}))

segmento,totale_missing,totale_segmento,percentuale_missing
<chr>,<int>,<int>,<dbl>
Sme Retail,10724,88630,12.10
Sme Corporate,3746,31229,12.00
Altro,1929,10474,18.42
Corporate,524,6765,7.75
Large corporate,197,3010,6.54


In [581]:
master_fb_sme <- master_fb[`MacroSegmento_Attuale LC` == 'Sme Retail', 
                           .(NUM_PRATICA,PROGRESS_PRATICA,FORB_PRESENZA)]
master_fb_nonsme <- master_fb[`MacroSegmento_Attuale LC` != 'Sme Retail' & `MacroSegmento_Attuale LC` != "Altro", .(NUM_PRATICA,PROGRESS_PRATICA,FORB_PRESENZA) ]

saveRDS(master_fb_sme,'../data/RDS/master/sme/master_fb.RDS')
saveRDS(master_fb_nonsme,'../data/RDS/master/non_sme/master_fb.RDS')

In [582]:

table(master_fb[!is.na(FORB_PRESENZA),year(DATA_DELIBERA_O_ANNULL_FIDO)], useNA = 'always')


 2017  2018  2019  2020  2021  <NA> 
22628 46535 34652 18817   356     0 

In [583]:
table(master_fb_sme$FORB_PRESENZA)
table(master_fb_nonsme$FORB_PRESENZA)
table(master_fb$FORB_PRESENZA)


    0     1 
77776   130 


    0     1 
36337   200 


     0      1 
122655    333 

## CRA ANDAMENTALE

In [584]:
dt_cra_trend <- readRDS('../data/RDS/cra_trend.RDS')

In [585]:
master_cra_trend <- merge(dt_orig[,c("NUM_PRATICA","PROGRESS_PRATICA",'MacroSegmento_Attuale LC')],
                        dt_cra_trend,
                        by = c("NUM_PRATICA","PROGRESS_PRATICA"),
                        all.x = TRUE)

In [586]:
table(master_cra_trend$CRA_andamento, useNA = 'always')


         Cra Grigio nel trimestre Cra non presente ultimo trimestre 
                            10898                              5109 
                       Migliorato                  Non presente cra 
                             1808                              2160 
                       Peggiorato                           Stabile 
                             1876                             47298 
                         Volatile                              <NA> 
                              314                             70645 

In [587]:
#master_cra_trend[grepl('cra',tolower(CRA_andamento)), CRA_andamento := NA]

In [588]:
#master_cra_trend

master_cra_trend_sme <- master_cra_trend[`MacroSegmento_Attuale LC` == 'Sme Retail', 
                           .(NUM_PRATICA,PROGRESS_PRATICA,CRA_andamento)]
master_cra_trend_nonsme <- master_cra_trend[`MacroSegmento_Attuale LC` != 'Sme Retail' & `MacroSegmento_Attuale LC` != "Altro", .(NUM_PRATICA,PROGRESS_PRATICA,CRA_andamento) ]

saveRDS(master_cra_trend_sme,'../data/RDS/master/sme/master_cra_andamentale.RDS')
saveRDS(master_cra_trend_nonsme,'../data/RDS/master/non_sme/master_cra_andamentale.RDS')

In [589]:
table(master_cra_trend_sme$CRA_andamento,useNA = 'always')
table(master_cra_trend_nonsme$CRA_andamento,useNA = 'always')


         Cra Grigio nel trimestre Cra non presente ultimo trimestre 
                             8271                              3215 
                       Migliorato                  Non presente cra 
                              791                               840 
                       Peggiorato                           Stabile 
                              774                             17924 
                         Volatile                              <NA> 
                              160                             56655 


         Cra Grigio nel trimestre Cra non presente ultimo trimestre 
                             2614                              1879 
                       Migliorato                  Non presente cra 
                             1011                               760 
                       Peggiorato                           Stabile 
                             1097                             29209 
                         Volatile                              <NA> 
                              153                              4281 

## DATI DB DSI

In [590]:
source('model_functions.R')
source('cols_tokeep.R')

cols_anagrafica <- c(#'DATA_DELIBERA_O_ANNULL_FIDO',
"STATO_PRATICA",
"SNDG_CEDUTO",'ID_CEDENTE','d_riferimento','anno',"COD_SNDG")


cols_master <- setdiff(cols_tokeep,cols_anagrafica)


master_dsi<- dt_orig[,..cols_master]

master_dsi <- clean_data(master_dsi)

Warning message in eval(jsub, SDenv, parent.frame()):
“si è prodotto un NA per coercizione”
Warning message in `[.data.table`(master, , `:=`(input_di_fidi_cedente, NULL)):
“Column 'input_di_fidi_cedente' does not exist to remove”


In [591]:
colnames(master_dsi) <- paste0("DSI_",colnames(master_dsi))

In [592]:
setnames(master_dsi, c('DSI_num_pratica','DSI_progress_pratica','DSI_macrosegmento_attuale_lc',"DSI_target"),
         c('NUM_PRATICA','PROGRESS_PRATICA','macrosegmento_attuale_lc','target'))

In [593]:
sort(colnames(master_dsi))

[1] "DSI_autonomia_fido"                      
 [2] "DSI_causale_negativa_appl"               
 [3] "DSI_causale_negativa_fido"               
 [4] "DSI_col_appl_con_causale_negativa"       
 [5] "DSI_col_fido_con_causale_negativa"       
 [6] "DSI_col_presenza_pratica_pef"            
 [7] "DSI_delta_scadenza_rating"               
 [8] "DSI_diff_appr_appl"                      
 [9] "DSI_divisione_ceduto"                    
[10] "DSI_fallimenti_certi"                    
[11] "DSI_filiale_factoring_ceduto"            
[12] "DSI_flag_cr_accordato_utilizzato"        
[13] "DSI_flag_cr_presenza_sconfinamenti"      
[14] "DSI_flag_data_costituzione"              
[15] "DSI_flag_debiti_bancari_abreve_fatturato"
[16] "DSI_flag_fallimenti_dubbi"               
[17] "DSI_flag_fidi_scaduti"                   
[18] "DSI_flag_fido_ridotto"                   
[19] "DSI_flag_passivo_corrente_fatturato"     
[20] "DSI_flag_patrimonio_netto_neg"           
[21] "DSI_flag_pregiudizievoli_gravi"          
[22] "DSI_flag_presenza_sofferenze"            
[23] "DSI_flag_procedure_concorsuali"          
[24] "DSI_flag_scaduto_20_outstanding"         
[25] "DSI_imp_richiesto_lim_autonomia"         
[26] "DSI_importo_fido_richiesto"              
[27] "DSI_importo_richiesto_applicazione"      
[28] "DSI_modello_rating_t0"                   
[29] "DSI_num_fidi_scaduti"                    
[30] "DSI_parte_correlata"                     
[31] "DSI_polizza_diretta_39_"                 
[32] "DSI_polizza_indiretta_41_"               
[33] "DSI_presenza_fidi_ced"                   
[34] "DSI_procedure_imprese_collegate"         
[35] "DSI_protesti_certi"                      
[36] "DSI_rae_ceduto"                          
[37] "DSI_rapporti_in_osservazione"            
[38] "DSI_sab"                                 
[39] "DSI_settore_cedente"                     
[40] "DSI_settore_ceduto"                      
[41] "DSI_stato_societario"                    
[42] "DSI_tipo_pratica"                        
[43] "DSI_tipo_prodotto"                       
[44] "NUM_PRATICA"                             
[45] "PROGRESS_PRATICA"                        
[46] "macrosegmento_attuale_lc"                
[47] "target"

In [595]:
master_dsi_sme <- master_dsi[macrosegmento_attuale_lc == "Sme Retail",]
master_dsi_nonsme <- master_dsi[macrosegmento_attuale_lc != "Sme Retail"& macrosegmento_attuale_lc != "Altro",]

saveRDS(master_dsi_sme,'../data/RDS/master/sme/master_dsi.RDS')
saveRDS(master_dsi_nonsme,'../data/RDS/master/non_sme/master_dsi.RDS')

In [596]:
colnames(master_dsi_sme)

[1] "NUM_PRATICA"                             
 [2] "PROGRESS_PRATICA"                        
 [3] "DSI_tipo_pratica"                        
 [4] "DSI_filiale_factoring_ceduto"            
 [5] "DSI_settore_ceduto"                      
 [6] "DSI_divisione_ceduto"                    
 [7] "DSI_importo_richiesto_applicazione"      
 [8] "DSI_settore_cedente"                     
 [9] "DSI_tipo_prodotto"                       
[10] "DSI_rae_ceduto"                          
[11] "DSI_polizza_diretta_39_"                 
[12] "DSI_polizza_indiretta_41_"               
[13] "macrosegmento_attuale_lc"                
[14] "target"                                  
[15] "DSI_sab"                                 
[16] "DSI_col_presenza_pratica_pef"            
[17] "DSI_col_fido_con_causale_negativa"       
[18] "DSI_col_appl_con_causale_negativa"       
[19] "DSI_parte_correlata"                     
[20] "DSI_rapporti_in_osservazione"            
[21] "DSI_causale_negativa_fido"               
[22] "DSI_diff_appr_appl"                      
[23] "DSI_num_fidi_scaduti"                    
[24] "DSI_flag_fidi_scaduti"                   
[25] "DSI_autonomia_fido"                      
[26] "DSI_importo_fido_richiesto"              
[27] "DSI_flag_fido_ridotto"                   
[28] "DSI_causale_negativa_appl"               
[29] "DSI_flag_scaduto_20_outstanding"         
[30] "DSI_flag_presenza_sofferenze"            
[31] "DSI_flag_cr_accordato_utilizzato"        
[32] "DSI_flag_cr_presenza_sconfinamenti"      
[33] "DSI_flag_patrimonio_netto_neg"           
[34] "DSI_flag_debiti_bancari_abreve_fatturato"
[35] "DSI_flag_passivo_corrente_fatturato"     
[36] "DSI_stato_societario"                    
[37] "DSI_flag_data_costituzione"              
[38] "DSI_protesti_certi"                      
[39] "DSI_presenza_fidi_ced"                   
[40] "DSI_modello_rating_t0"                   
[41] "DSI_fallimenti_certi"                    
[42] "DSI_flag_fallimenti_dubbi"               
[43] "DSI_flag_pregiudizievoli_gravi"          
[44] "DSI_flag_procedure_concorsuali"          
[45] "DSI_procedure_imprese_collegate"         
[46] "DSI_delta_scadenza_rating"               
[47] "DSI_imp_richiesto_lim_autonomia"

## DATI COOPERATIVEE E NOPROFIT

In [597]:
dt_anag <- readRDS('../data/RDS/dt_anag.RDS')
dt_cartipo <- readRDS('../data/RDS/dt_cartipo.RDS')

In [598]:
sub_dati<-dt_orig[,.(SNDG_CEDUTO,DATA_DELIBERA_O_ANNULL_FIDO,`MacroSegmento_Attuale LC`,NUM_PRATICA,PROGRESS_PRATICA)]

In [599]:
head(dt_anag,1)

COD_NSG,COD_TP_CLI
<chr>,<chr>
0000000001177725,SCRL


In [600]:
master_tipo <- unique(merge(sub_dati,
                   dt_anag,
                   by.x = c("SNDG_CEDUTO"),
                   by.y = c("COD_NSG"),
                   all.x =TRUE)
                   )
master_tipo <- unique(merge(master_tipo,
                   dt_cartipo,
                   by.x = c("SNDG_CEDUTO"),
                   by.y = c("COD_SNDG"),
                   all.x =TRUE)
                   )

In [601]:
table(master_tipo$COD_CARTIPO, useNA = "always")
table(master_tipo$COD_TP_CLI, useNA = "always")


   115   <NA> 
   559 139549 


  ASNR  ASRIC    COM   CONS  COOSC   SCRL   <NA> 
    16      1      2      4      6     11 140068 

In [602]:
nrow(dt_anag[,.(COD_NSG)])
nrow(unique(dt_anag[,.(COD_NSG)]))
nrow(unique(dt_anag[,.(COD_NSG,COD_TP_CLI)]))

nrow(dt_cartipo[,.(COD_SNDG)])
nrow(unique(dt_cartipo[,.(COD_SNDG)]))
nrow(unique(dt_cartipo[,.(COD_SNDG,COD_CARTIPO)]))



[1] 34

[1] 34

[1] 34

[1] 418

[1] 418

[1] 418

In [603]:
cols_2keep_tipo <- c('NUM_PRATICA','PROGRESS_PRATICA','COD_TP_CLI','COD_CARTIPO')

In [604]:
nrow(master_bil_sme)
nrow(master_bil_nonsme)

nrow(master_ateco_sme)
nrow(master_ateco_nonsme)

[1] 88630

[1] 41004

[1] 88630

[1] 41004

In [605]:
master_tipo_sme <- master_tipo[`MacroSegmento_Attuale LC` == "Sme Retail",..cols_2keep_tipo]
master_tipo_nonsme <- master_tipo[`MacroSegmento_Attuale LC` != "Sme Retail" & `MacroSegmento_Attuale LC` != "Altro",..cols_2keep_tipo]

saveRDS(master_tipo_sme,'../data/RDS/master/sme/master_tipo.RDS')
saveRDS(master_tipo_nonsme,'../data/RDS/master/non_sme/master_tipo.RDS')

## CLIENTI / NON CLIENTI

In [1037]:
dt_clienti <- readRDS('../data/RDS/dt_clienti.RDS')

In [1038]:
colnames(dt_clienti)

[1] "NUM_PRATICA"                 "PROGRESS_PRATICA"           
 [3] "TIPO_PRATICA"                "STATO_PRATICA"              
 [5] "TIPO_PRODOTTO"               "SNDG_CEDUTO"                
 [7] "SNDG_CEDUTO_old"             "DATA_DELIBERA_O_ANNULL_FIDO"
 [9] "Data_delibera_Num"           "Mese_delibera"              
[11] "Anno_delibera"               "ESITO_FINALE_VPC"           
[13] "MacroSegmento_Attuale"       "Data_Pcr_Utilizzata"        
[15] "Classificazione_Dettagliata" "Classificazione_Cliente"

In [1039]:
sub_dati<-dt_orig[,.(`MacroSegmento_Attuale LC`,NUM_PRATICA,PROGRESS_PRATICA)]

In [1040]:
master_clienti <- unique(merge(sub_dati,
                   dt_clienti[,.(NUM_PRATICA,PROGRESS_PRATICA,Classificazione_Cliente)],
                   by= c('NUM_PRATICA','PROGRESS_PRATICA'),
                   all.x =TRUE)
                   )

In [1041]:
cols_2keep_clienti <- c('NUM_PRATICA','PROGRESS_PRATICA','Classificazione_Cliente')

In [736]:
master_clienti_sme <- master_clienti[`MacroSegmento_Attuale LC` == "Sme Retail",..cols_2keep_clienti]
master_clienti_nonsme <- master_clienti[`MacroSegmento_Attuale LC` != "Sme Retail" & `MacroSegmento_Attuale LC` != "Altro",..cols_2keep_clienti]

saveRDS(master_clienti_sme,'../data/RDS/master/sme/master_clienti.RDS')
saveRDS(master_clienti_nonsme,'../data/RDS/master/non_sme/master_clienti.RDS')

In [1044]:
master_clienti[,(perc =round(100*sum(Classificazione_Cliente=="NON CLIENTE")/.N,2)), by = c('MacroSegmento_Attuale LC')]

MacroSegmento_Attuale LC,V1
<chr>,<dbl>
Sme Retail,63.74
Sme Corporate,23.48
Large corporate,11.23
Corporate,15.62
Altro,80.65


In [1045]:
master_clienti[,(perc =round(100*sum(Classificazione_Cliente=="CLIENTE")/.N,2)), by = c('MacroSegmento_Attuale LC')]

MacroSegmento_Attuale LC,V1
<chr>,<dbl>
Sme Retail,36.26
Sme Corporate,76.52
Large corporate,88.77
Corporate,84.38
Altro,19.35


## ANALISI MISSING VARIE FONTI

In [606]:
nrow(master_rat_small[`MacroSegmento_Attuale LC`== "Sme Retail" & !is.na(RAT_rating_post_cal_Naff_finale),])/nrow(master_rat_small[`MacroSegmento_Attuale LC`== "Sme Retail" &!is.na(RAT_classe_rating),])

nrow(master_rat_small[`MacroSegmento_Attuale LC`== "Sme Retail" & !is.na(RAT_rating_finale_post_notch) & is.na(RAT_rating_post_cal_Naff_finale),])/nrow(master_rat_small[`MacroSegmento_Attuale LC`== "Sme Retail" &!is.na(RAT_classe_rating),])

nrow(master_rat_small[`MacroSegmento_Attuale LC`== "Sme Retail" & !is.na(RAT_classe_rating),])

[1] 0.8303049

[1] 0.1696951

[1] 36542

In [633]:
colnames(master_cr)

[1] "MacroSegmento_Attuale LC"    "NUM_PRATICA"                
 [3] "PROGRESS_PRATICA"            "STATO_PRATICA"              
 [5] "SNDG_CEDUTO"                 "DATA_DELIBERA_O_ANNULL_FIDO"
 [7] "DATA_ISTRUTTORIA"            "CR_Accordato_200_005"       
 [9] "CR_Utilizzato_200_005"       "CR_IMP_SCONF_200_005"       
[11] "CR_Accordato_200_018"        "CR_Utilizzato_200_018"      
[13] "CR_IMP_SCONF_200_018"        "CR_Accordato_400_005"       
[15] "CR_Utilizzato_400_005"       "CR_IMP_SCONF_400_005"       
[17] "CR_Accordato_400_018"        "CR_Utilizzato_400_018"      
[19] "CR_IMP_SCONF_400_018"        "CR_Accordato_600_000"       
[21] "CR_Utilizzato_600_000"       "CR_IMP_SCONF_600_000"       
[23] "CR_Accordato_800_000"        "CR_Utilizzato_800_000"      
[25] "CR_IMP_SCONF_800_000"        "CR_Accordato_1000_000"      
[27] "CR_Utilizzato_1000_000"      "CR_IMP_SCONF_1000_000"      
[29] "CR_Accordato_2200_000"       "CR_Utilizzato_2200_000"     
[31] "CR_IMP_SCONF_2200_000"       "CR_Accordato_2400_000"      
[33] "CR_Utilizzato_2400_000"      "CR_IMP_SCONF_2400_000"      
[35] "CR_Accordato_4900_003"       "CR_Utilizzato_4900_003"     
[37] "CR_IMP_SCONF_4900_003"       "CR_Accordato_4900_005"      
[39] "CR_Utilizzato_4900_005"      "CR_IMP_SCONF_4900_005"      
[41] "CR_Accordato_4900_018"       "CR_Utilizzato_4900_018"     
[43] "CR_IMP_SCONF_4900_018"       "CR_Accordato_4901_003"      
[45] "CR_Utilizzato_4901_003"      "CR_IMP_SCONF_4901_003"      
[47] "CR_Accordato_4901_005"       "CR_Utilizzato_4901_005"     
[49] "CR_IMP_SCONF_4901_005"       "CR_Accordato_4901_018"      
[51] "CR_Utilizzato_4901_018"      "CR_IMP_SCONF_4901_018"      
[53] "CR_Accordato_4902_003"       "CR_Utilizzato_4902_003"     
[55] "CR_IMP_SCONF_4902_003"       "CR_Accordato_4902_005"      
[57] "CR_Utilizzato_4902_005"      "CR_IMP_SCONF_4902_005"      
[59] "CR_Accordato_4902_018"       "CR_Utilizzato_4902_018"     
[61] "CR_IMP_SCONF_4902_018"       "accordato"                  
[63] "sconfino"                    "utilizzato"                 
[65] "accordato_revoca"            "sconfino_revoca"            
[67] "utilizzato_revoca"           "accordato_autoliq"          
[69] "sconfino_autoliq"            "utilizzato_autoliq"         
[71] "accordato_scadenza"          "sconfino_scadenza"          
[73] "utilizzato_scadenza"         "accordato_bt"               
[75] "sconfino_bt"                 "utilizzato_bt"              
[77] "accordato_mlt"               "sconfino_mlt"               
[79] "utilizzato_mlt"              "sconf_acc"                  
[81] "sconf_acc_bt"                "sconf_acc_mlt"              
[83] "uti_acc"                     "uti_acc_bt"                 
[85] "uti_acc_mlt"                 "uti_acc_revoca"             
[87] "uti_acc_autoliq"             "uti_acc_scadenza"

In [640]:
# master_varie_fonti <- merge(master_rat_small,
#                             master_cr[,c('PROGRESS_PRATICA','NUM_PRATICA','DATA_ISTRUTTORIA')],
#                             by = c('PROGRESS_PRATICA','NUM_PRATICA'))

# appo_master_bil <- master_bil_raw[,c('PROGRESS_PRATICA','NUM_PRATICA','DAT_ULTRICL')]
# setnames(appo_master_bil,'DAT_ULTRICL','BIL_DAT_ULTRICL')
# master_varie_fonti <- merge(master_varie_fonti,
#                            appo_master_bil,
#                            by = c('PROGRESS_PRATICA','NUM_PRATICA'))

# nrow(master_varie_fonti)

# appo_master_modfisc <-  master_mod_fisc[,c('PROGRESS_PRATICA','NUM_PRATICA','COD_MODELLO')]
# setnames(appo_master_modfisc,'COD_MODELLO','MODFISC_COD_MODELLO')
# master_varie_fonti <- merge(master_varie_fonti,
#                          appo_master_modfisc ,
#                            by = c('PROGRESS_PRATICA','NUM_PRATICA'))

# nrow(master_varie_fonti)


# appo_master_xra <-  master_xra_raw[,c('PROGRESS_PRATICA','NUM_PRATICA','IMP_SCONFINO')]
# setnames(appo_master_xra,'IMP_SCONFINO','XRA_IMP_SCONFINO')
# master_varie_fonti <- merge(master_varie_fonti,
#                          appo_master_xra ,
#                            by = c('PROGRESS_PRATICA','NUM_PRATICA'))


# nrow(master_varie_fonti)

# colnames(master_varie_fonti)

# cat("rating generale")
#  dt <- rbindlist(lapply(unique(master_varie_fonti$`MacroSegmento_Attuale LC`), function(seg){
#     missing_val <- sum(is.na(master_varie_fonti[`MacroSegmento_Attuale LC`==seg  ]$RAT_classe_rating) |
#                       master_varie_fonti[`MacroSegmento_Attuale LC`==seg  ]$RAT_classe_rating == "UR")
#     all_val <- nrow(master_varie_fonti[`MacroSegmento_Attuale LC`==seg ])
#     perc <- 100*missing_val/all_val
#     return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
# }))
# dt
# write.xlsx(dt,
#           'tabelle_perc_miss.xlsx',
#           row.names = FALSE,
#           append = FALSE,
#           sheetName = 'rating')

# master_varie_sme <- master_varie_fonti[`MacroSegmento_Attuale LC`== "Sme Retail",]
# master_varie_co <- master_varie_fonti[`MacroSegmento_Attuale LC`!= "Sme Retail",]

# table(master_varie_fonti[Rating_Inizio_Periodo == 'UR',`MacroSegmento_Attuale LC`])

# nrow(master_varie_sme[!is.na(RAT_rating_finale_post_notch) & is.na(BIL_DAT_ULTRICL)])
# nrow(master_varie_sme[is.na(RAT_rating_finale_post_notch) & !is.na(BIL_DAT_ULTRICL)])

# tab <-as.data.table(table((!is.na(master_varie_sme$RAT_rating_post_cal_Naff_finale) & master_varie_sme$RAT_rating_post_cal_Naff_finale != "UR")
#                            ,
#                           !is.na(master_varie_sme$CR_DAT_FINE_PERIO)))
# colnames(tab)<- c('banking','CR','N')

# write.xlsx(tab,
#           'tabelle_perc_miss.xlsx',
#           row.names = FALSE,
#           append = TRUE,
#           sheetName = 'banking_vs_CR')

# tab <-as.data.table(table((!is.na(master_varie_sme$RAT_rating_finale_post_notch) & master_varie_sme$RAT_rating_finale_post_notch != "UR"),
#                           !is.na(master_varie_sme$CR_DAT_FINE_PERIO)))
# colnames(tab)<- c('factoring','CR','N')

# write.xlsx(tab,
#           'tabelle_perc_miss.xlsx',
#           row.names = FALSE,
#           append = TRUE,
#           sheetName = 'factoring_vs_CR')

# tab <-as.data.table(table((!is.na(master_varie_sme$RAT_rating_post_cal_Naff_finale)& master_varie_sme$RAT_rating_post_cal_Naff_finale != "UR"),
#                           !is.na(master_varie_sme$BIL_DAT_ULTRICL)))
# colnames(tab)<- c('banking','BIL','N')

# write.xlsx(tab,
#           'tabelle_perc_miss.xlsx',
#           row.names = FALSE,
#           append = TRUE,
#           sheetName = 'banking_vs_BIL')

# tab <-as.data.table(table((!is.na(master_varie_sme$RAT_rating_finale_post_notch)& master_varie_sme$RAT_rating_finale_post_notch != "UR"),
#                           !is.na(master_varie_sme$BIL_DAT_ULTRICL)))
# colnames(tab)<- c('factoring','BIL','N')

# write.xlsx(tab,
#           'tabelle_perc_miss.xlsx',
#           row.names = FALSE,
#           append = TRUE,
#           sheetName = 'factoring_vs_BIL')

# tab <-as.data.table(table((!is.na(master_varie_sme$RAT_rating_post_cal_Naff_finale)& master_varie_sme$RAT_rating_post_cal_Naff_finale != "UR"),
#                           !is.na(master_varie_sme$MODFISC_COD_MODELLO)))
# colnames(tab)<- c('banking','mod_fisc','N')

# write.xlsx(tab,
#           'tabelle_perc_miss.xlsx',
#           row.names = FALSE,
#           append = TRUE,
#           sheetName = 'banking_vs_mod_fisc')

# tab <-as.data.table(table((!is.na(master_varie_sme$RAT_rating_finale_post_notch)& master_varie_sme$RAT_rating_finale_post_notch != "UR"),
#                           !is.na(master_varie_sme$MODFISC_COD_MODELLO)))
# colnames(tab)<- c('factoring','mod_fisc','N')

# write.xlsx(tab,
#           'tabelle_perc_miss.xlsx',
#           row.names = FALSE,
#           append = TRUE,
#           sheetName = 'factoring_vs_mod_fisc')

# nrow(master_varie_sme[!is.na(RAT_rating_post_cal_Naff_finale) & !is.na(MODFISC_COD_MODELLO)])

# dt_rating_bank[COD_SNDG == '0000000091526686',]
# dt_orig[SNDG_CEDUTO == '0000000091526686',c('SNDG_CEDUTO','DATA_DELIBERA_O_ANNULL_FIDO','Rating_Inizio_Periodo')]

# master_varie_sme[!is.na(RAT_rating_post_cal_Naff_finale) & is.na(Rating_Inizio_Periodo)]

# table(master_varie_sme[is.na(RAT_rating_post_cal_Naff_finale)  & is.na(RAT_rating_finale_post_notch),Rating_Inizio_Periodo],useNA = "always")

# # cat("rating banking valorizzato e CR valorizzata")
# # rbindlist(lapply(unique(master_varie_fonti$`MacroSegmento_Attuale LC`), function(seg){
# #     missing_val <- sum(is.na(master_varie_fonti[`MacroSegmento_Attuale LC`==seg & !is.na(RAT_rating_post_cal_Naff_finale)]$CR_DAT_FINE_PERIO))
# #     all_val <- nrow(master_varie_fonti[`MacroSegmento_Attuale LC`==seg & !is.na(RAT_rating_post_cal_Naff_finale)])
# #     perc <- 100*missing_val/all_val
# #     return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
# # }))

# 100*nrow(master_varie_fonti[!is.na(Rating_Inizio_Periodo)& Rating_Inizio_Periodo!= "UR" &  `MacroSegmento_Attuale LC` == "Sme Retail",])/nrow(master_varie_fonti[`MacroSegmento_Attuale LC` == "Sme Retail",])

# table(master_varie_sme[!is.na(RAT_rating_post_cal_Naff_finale) & RAT_rating_post_cal_Naff_finale!= "UR",Rating_Inizio_Periodo],useNA = 'always')

# nrow(master_varie_sme[!is.na(RAT_rating_post_cal_Naff_finale) & RAT_rating_post_cal_Naff_finale!= "UR",])

# table(master_varie_sme[is.na(Rating_Inizio_Periodo) | Rating_Inizio_Periodo == "UR" ,RAT_rating_finale_post_notch],useNA = 'always')

# tab1 <-as.data.table(table(
#     (!is.na(master_varie_sme$RAT_rating_post_cal_Naff_finale) & master_varie_sme$RAT_rating_post_cal_Naff_finale!= "UR"),
#     (!is.na(master_varie_sme$Rating_Inizio_Periodo) & master_varie_sme$Rating_Inizio_Periodo!= "UR"))
#                     )
# colnames(tab1)<- c('banking','rat_t0','N')

# tab2 <-as.data.table(table(
#     (!is.na(master_varie_sme$RAT_rating_finale_post_notch)& master_varie_sme$RAT_rating_finale_post_notch!= "UR"),
#     (!is.na(master_varie_sme$Rating_Inizio_Periodo) & master_varie_sme$Rating_Inizio_Periodo!= "UR"))
#                     )
# colnames(tab2)<- c('factoring','rat_t0','N')


# write.xlsx(tab1,
#           'tabelle_perc_miss.xlsx',
#           row.names = FALSE,
#           append = TRUE,
#           sheetName = 'banking_vs_rat_t0')
# write.xlsx(tab2,
#           'tabelle_perc_miss.xlsx',
#           row.names = FALSE,
#           append = TRUE,
#           sheetName = 'factoring_vs_rat_t0')

# master_sme_bank <- master_varie_sme[!is.na(RAT_rating_post_cal_Naff_finale) &  RAT_rating_post_cal_Naff_finale != "UR",]
# master_sme_fact <- master_varie_sme[!is.na(RAT_rating_finale_post_notch) &  RAT_rating_finale_post_notch != "UR",]

# tab1 <-as.data.table(table(!is.na(master_sme_bank$CR_DAT_FINE_PERIO),
#                           (!is.na(master_sme_bank$MODFISC_COD_MODELLO) |
#                           !is.na(master_sme_bank$BIL_DAT_ULTRICL))))
# colnames(tab1)<- c('CR','BIL_FISC','N')

# tab2 <-as.data.table(table(!is.na(master_sme_fact$CR_DAT_FINE_PERIO),
#                           (!is.na(master_sme_fact$MODFISC_COD_MODELLO) |
#                           !is.na(master_sme_fact$BIL_DAT_ULTRICL))))
# colnames(tab2)<- c('CR','BIL_FISC','N')


# write.xlsx(tab1,
#           'tabelle_perc_miss.xlsx',
#           row.names = FALSE,
#           append = TRUE,
#           sheetName = 'bank_val_CR_vs_BIL_FISC')
# write.xlsx(tab2,
#           'tabelle_perc_miss.xlsx',
#           row.names = FALSE,
#           append = TRUE,
#           sheetName = 'fact_val_CR_vs_BIL_FISC')



# rbindlist(lapply(unique(master_bil_mfisc$`MacroSegmento_Attuale LC`), function(seg){
#     missing_val <- sum(is.na(master_bil_mfisc[`MacroSegmento_Attuale LC`==seg]$COD_MODELLO) |is.na(master_bil_mfisc[`MacroSegmento_Attuale LC`==seg]$BIL_onerifin_ebitda) )
#     all_val <- nrow(master_bil_mfisc[`MacroSegmento_Attuale LC`==seg])
#     perc <- 100*missing_val/all_val
#     return(data.table(segmento = seg, totale_missing = missing_val, totale_segmento = all_val, percentuale_missing = round(perc,2)))
# }))

# master_bil_mfisc$BIL_onerifin_ebitda

# nrow()



# colnames(dt_orig)

## Analisi per business

In [641]:
## Lettura dati VPC (già salvati in RDS)
dt_completo <- readRDS('../data/DB_FINALE_INTEGRATO_v02.RDS')

In [642]:
#table(dt_completo$`Esito_final_giallo e Monocontrollo_giallo`)
#dt_completo[`Esito_final_giallo e Monocontrollo_giallo`=='Monocontrollo - LIMITI SOCIETARI',]

dt_lim_soc <- merge(dt_orig,
     dt_completo[`Esito_final_giallo e Monocontrollo_giallo`=='Monocontrollo - LIMITI SOCIETARI',
                c('NUM_PRATICA','PROGRESS_PRATICA','Esito_final_giallo e Monocontrollo_giallo')],
     by = c('NUM_PRATICA','PROGRESS_PRATICA'),
           all.x = TRUE)

In [643]:
#dt_orig[`Esito_final_giallo e Monocontrollo_giallo`=='Monocontrollo - LIMITI SOCIETARI' & ,]

In [644]:
colnames(dt_completo)[grepl('Esito',colnames(dt_completo))]

[1] "Esito - EXPORT FACILE- FISCALE C"                    
 [2] "Esito - CEDENTE REVOCATO"                            
 [3] "Esito - CEDENTE TRAVEL"                              
 [4] "Esito - SOGGETTO ESTERO"                             
 [5] "Esito - CODICE FISCALE/P. IVA NO"                    
 [6] "Esito - CODICE FISCALE GIA' PRES"                    
 [7] "Esito - CODICE FISCALE GIA'_0001"                    
 [8] "Esito - P.IVA GIA' PRESENTE NELL"                    
 [9] "Esito - CODICE CLIENTE GIA' PRES"                    
[10] "Esito - CODICE CLIENTE GIA'_0001"                    
[11] "Esito - CODICE FISCALE FORMALMEN"                    
[12] "Esito - PRESENZA SOGGETTO IN PIU"                    
[13] "Esito - TIPO OPERAZIONE RIDUZION"                    
[14] "Esito - IMPORTO RICHIESTO <= AD"                     
[15] "Esito - IMPORTO RICHIESTO A ZERO"                    
[16] "Esito - SOGGETTO EEPP"                               
[17] "Esito - ABI/CAB/CONTO OBBLIGATOR"                    
[18] "Esito - EXPORT FACILE-SIA CEDENT"                    
[19] "Esito - SOGGETTO CON FORMA GIURI"                    
[20] "Esito - FORMA GIURIDICA SENZA CC"                    
[21] "Esito - CONFIRMING - CHAMPION DE"                    
[22] "Esito - CONFIRMING - CODICE FISC"                    
[23] "Esito - CONFIRMING - CODICE_0001"                    
[24] "Esito - CONFIRMING - P.IVA GIA'"                     
[25] "Esito - CONFIRMING - CODICE FORN"                    
[26] "Esito - STATO SAB"                                   
[27] "Esito - RATING SCADUTO"                              
[28] "Esito - PARTE CORRELATA"                             
[29] "Esito - PRESENZA DI FIDI CEDENTE"                    
[30] "Esito - SOGGETTO CEDENTE"                            
[31] "Esito - RAPPORTI IN OSSERVAZIONE"                    
[32] "Esito - INCONGRUENZA TRA LOCALIT"                    
[33] "Esito - PRESENZA GARANZIA OPERAN"                    
[34] "Esito - PRESENZA PRATICA IN PEF5"                    
[35] "Esito - NDG SETTORIALE MANCANTE"                     
[36] "Esito - FIDO CON CAUSALE NEGATIV"                    
[37] "Esito - APPROVATO"                                   
[38] "Esito - FIDO SCADUTO"                                
[39] "Esito - AUTONOMIA FIDO"                              
[40] "Esito - LIMITI SOCIETARI"                            
[41] "Esito - LIMITI SOCIETARI - AS IS"                    
[42] "Esito - FIDO RIDOTTO"                                
[43] "Esito - AUTORIZZAZIONE NO CONTRO"                    
[44] "Esito - APPLICAZIONE CON CAUSALE"                    
[45] "Esito - APPLICAZIONI CON CAUSALE"                    
[46] "Esito - SCADUTO > 20%OUTSTANDING"                    
[47] "Esito - CR: PRESENZA SOFFERENZE"                     
[48] "Esito - CR: UTILIZZATO > ACCORDA"                    
[49] "Esito - CR: PRESENZA SCONFINAMEN"                    
[50] "Esito - BILANCI :PATRIMONIO NETT"                    
[51] "Esito - BILANCI :DEBITI BANCARI"                     
[52] "Esito - BILANCI :PASSIVO CORRENT"                    
[53] "Esito - STATO SOCIETARIO"                            
[54] "Esito - DATA COSTITUZIONE"                           
[55] "Esito - NUMERO INTERROGAZIONI"                       
[56] "Esito - PROTESTI CERTI"                              
[57] "Esito - PROTESTI DUBBI"                              
[58] "Esito - FALLIMENTI CERTI"                            
[59] "Esito - FALLIMENTI DUBBI"                            
[60] "Esito - PREGIUDIZIEVOLI GRAVI"                       
[61] "Esito - PROCEDURE CONCORSUALI"                       
[62] "Esito - IMPRESE COLLEGATE: PROCE"                    
[63] "Esito - ATECO BKIT OBBLIGATORIO"                     
[64] "Esito - CODICE CLIENTE GIA'_0002"                    
[65] "Esito - CEDENTE APPARTIENE A GRU"                    
[66] "Esito - TIMEOUT SCADUTO1"                            
[67] "Esito - NESSUNA POSIZIONE INCROC"  

In [645]:
table( dt_completo[TIPO_PRATICA == 2,`IMPORTO FIDO`],useNA = 'always')


       0     1500     2000     3000     5000     7000    10000    13000 
      36        1        1        2        3        1        2        1 
   15000    20000    21000    30000    35000    50000    55000    60000 
       1        5        1        1        5        6        1        1 
   68000    70000    80000    85000    91000    1e+05   138000   145000 
       1        2        2        1        1        7        1        1 
  150000   220000   250000    3e+05   360000    5e+05   620000   750000 
       2        1        1        4        1        1        1        1 
   9e+05    1e+06  1610000  2150000    3e+06    4e+06    5e+06  7200000 
       1        1        1        1        1        2        2        1 
   8e+06  9900000  1.2e+07    2e+07 25164000 1.28e+08     <NA> 
       1        1        1        1        1        1    28690 

In [646]:
nrow(dt_completo)

[1] 252939

In [647]:
sub_dati <- dt_completo[TIPO_PRATICA == 3,]

In [648]:
year(sub_dati$DATA_DELIBERA_O_ANNULL_FIDO[1])

[1] 2020

In [649]:
sub_dati[year(DATA_DELIBERA_O_ANNULL_FIDO) == '2020' & ESITO_FINALE_VPC == "ROSSO",
         #& `Esito_final_giallo e Monocontrollo_giallo`!='Monocontrollo - LIMITI SOCIETARI',
         .(num_pratiche = .N, somma_fidi = sum(as.numeric(NUMERO_TOT_FIDI),na.rm = TRUE),
          imp_tot_fidi = sum(as.numeric(`IMPORTO FIDO`),na.rm = TRUE)), by = c('Colore - LIMITI SOCIETARI')]

Colore - LIMITI SOCIETARI,num_pratiche,somma_fidi,imp_tot_fidi
<chr>,<int>,<dbl>,<dbl>
VERDE,1873,1920,12770900
ROSSO,1509,1550,213000
GIALLO,522,578,64806000
ND,20,20,291650
NA,7,9,0


In [650]:
sub_dati[year(DATA_DELIBERA_O_ANNULL_FIDO) == '2020' & `Colore - LIMITI SOCIETARI` == "GIALLO",
         #& `Esito_final_giallo e Monocontrollo_giallo`!='Monocontrollo - LIMITI SOCIETARI',
         .(num_pratiche = .N, somma_fidi = sum(as.numeric(NUMERO_TOT_FIDI),na.rm = TRUE),
          imp_tot_fidi = sum(as.numeric(`IMPORTO FIDO`),na.rm = TRUE)), by = c('ESITO_FINALE_VPC')][order(ESITO_FINALE_VPC)]

Warning message in `[.data.table`(sub_dati, year(DATA_DELIBERA_O_ANNULL_FIDO) == :
“si è prodotto un NA per coercizione”


ESITO_FINALE_VPC,num_pratiche,somma_fidi,imp_tot_fidi
<chr>,<int>,<dbl>,<dbl>
GIALLO,4532,5178,4609220715
ROSSO,522,578,64806000


In [651]:
sub_dati[year(DATA_DELIBERA_O_ANNULL_FIDO) == '2020',
         .(num_pratiche = .N, somma_fidi = sum(as.numeric(NUMERO_TOT_FIDI),na.rm = TRUE),
          imp_tot_fidi = sum(as.numeric(`IMPORTO FIDO`),na.rm = TRUE)), by = c('ESITO_FINALE_VPC','Colore - LIMITI SOCIETARI')][order(ESITO_FINALE_VPC)]

Warning message in `[.data.table`(sub_dati, year(DATA_DELIBERA_O_ANNULL_FIDO) == :
“si è prodotto un NA per coercizione”
Warning message in `[.data.table`(sub_dati, year(DATA_DELIBERA_O_ANNULL_FIDO) == :
“si è prodotto un NA per coercizione”
Warning message in `[.data.table`(sub_dati, year(DATA_DELIBERA_O_ANNULL_FIDO) == :
“si è prodotto un NA per coercizione”


ESITO_FINALE_VPC,Colore - LIMITI SOCIETARI,num_pratiche,somma_fidi,imp_tot_fidi
<chr>,<chr>,<int>,<dbl>,<dbl>
GIALLO,NA,1529,1517,475828750
GIALLO,GIALLO,4532,5178,4609220715
GIALLO,VERDE,2782,3113,106849691
GIALLO,ND,9,11,25480800
GIALLO,ROSSO,17,17,3150755
ROSSO,VERDE,1873,1920,12770900
ROSSO,ROSSO,1509,1550,213000
ROSSO,GIALLO,522,578,64806000
ROSSO,ND,20,20,291650


In [652]:
sub_dati[year(DATA_DELIBERA_O_ANNULL_FIDO) == '2020' & `Esito_final_giallo e Monocontrollo_giallo`=='Monocontrollo - LIMITI SOCIETARI',
         .(num_pratiche = .N, somma_fidi = sum(as.numeric(NUMERO_TOT_FIDI),na.rm = TRUE),
          imp_tot_fidi = sum(as.numeric(`IMPORTO FIDO`),na.rm = TRUE)), by = c('ESITO_FINALE_VPC')]

Warning message in `[.data.table`(sub_dati, year(DATA_DELIBERA_O_ANNULL_FIDO) == :
“si è prodotto un NA per coercizione”


ESITO_FINALE_VPC,num_pratiche,somma_fidi,imp_tot_fidi
<chr>,<int>,<dbl>,<dbl>
GIALLO,2380,2403,440927464


In [653]:
sub_dati[year(DATA_DELIBERA_O_ANNULL_FIDO) == '2020' & ESITO_FINALE_VPC == "GIALLO" & `Colore - LIMITI SOCIETARI` == "GIALLO",]

NUM_PRATICA,PROGRESS_PRATICA,STATO_PRATICA,CHIAVE_DB,ABI,ABI_PEF5,FILIALE_PEF5,ANNO_PEF5,PROGRESSIVO_PEF5,TIPO_PRATICA,⋯,M3-M4,M1-M2,I5-I6,I4,I3,I1-I2,NUOVI_VERDI,NUOVO_ESITO_FINALE,NUOVI_VERDI_SENZA_SOGLIA,NUOVO_ESITO_SENZA SOGLIA
<dbl>,<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>
102230036,1,11,1899-12-31 20:01:11,1025,1025,68255,2020,143,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO
102230038,1,11,1899-12-31 22:01:11,NA,NA,NA,NA,NA,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO
103230022,1,10,1899-12-31 22:01:10,1025,1025,75000,2020,315,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO
103230024,1,10,1899-12-31 00:01:10,1025,1025,7717,2020,127,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO
103230025,1,10,1899-12-31 01:01:10,1025,1025,66281,2020,102,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO
103230027,1,10,1899-12-31 03:01:10,1025,1025,75000,2020,491,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO
103230028,1,10,1899-12-31 04:01:10,1025,1025,75000,2020,117,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO
103230030,2,10,1899-12-31 06:02:10,1025,1025,66220,2020,461,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO
103230031,1,10,1899-12-31 07:01:10,1025,1025,7717,2020,521,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO


In [654]:
nrow(dt_orig)

[1] 140108

In [655]:
sub_dati_perimetro <- dt_orig[TIPO_PRATICA == 3 ,]

In [656]:
#sub_dati_perimetro

sub_dati_perimetro[year(DATA_DELIBERA_O_ANNULL_FIDO) == '2020' & `Esito_final_giallo e Monocontrollo_giallo`=='Monocontrollo - LIMITI SOCIETARI',
                   .(num_pratiche = .N, somma_fidi = sum(as.numeric(NUMERO_TOT_FIDI),na.rm = TRUE),
                     imp_tot_fidi = sum(as.numeric(`IMPORTO FIDO`),na.rm = TRUE),
                     td = round(100*sum(target == "DEFAULT")/.N,2)), by = 'ESITO_FINALE_VPC']

ESITO_FINALE_VPC,num_pratiche,somma_fidi,imp_tot_fidi,td
<chr>,<int>,<dbl>,<dbl>,<dbl>
GIALLO,968,985,181119250,0.41


In [657]:
100*nrow(dt_orig[year(DATA_DELIBERA_O_ANNULL_FIDO) == '2020' & target == "DEFAULT",])/nrow(dt_orig[year(DATA_DELIBERA_O_ANNULL_FIDO) == '2020'  ,])

[1] 1.466759

In [658]:
sub_dati_perimetro[year(DATA_DELIBERA_O_ANNULL_FIDO) == '2020' & target == "DEFAULT" & ESITO_FINALE_VPC == 'GIALLO' ,]

NUM_PRATICA,PROGRESS_PRATICA,STATO_PRATICA,CHIAVE_DB,ABI,ABI_PEF5,FILIALE_PEF5,ANNO_PEF5,PROGRESSIVO_PEF5,TIPO_PRATICA,⋯,M1-M2,I5-I6,I4,I3,I1-I2,NUOVI_VERDI,NUOVO_ESITO_FINALE,NUOVI_VERDI_SENZA_SOGLIA,NUOVO_ESITO_SENZA SOGLIA,target
<dbl>,<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
117230060,4,10,1899-12-31 20:04:10,1025,1025,75000,2020,215,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO,DEFAULT
203230139,1,10,1899-12-31 11:01:11,1025,1025,55001,2020,24,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO,DEFAULT
211230089,1,10,1899-12-31 17:01:11,1025,1025,66540,2020,98,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO,DEFAULT
221230062,1,10,1899-12-31 06:01:10,1025,1025,7717,2020,2302,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO,DEFAULT
221230075,1,10,1899-12-31 19:01:10,1025,1025,68140,2020,306,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO,DEFAULT
303230242,1,10,1899-12-31 10:01:10,1025,1025,7717,2020,1713,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO,DEFAULT
310230311,1,10,1899-12-31 23:01:11,1025,1025,7717,2020,3037,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO,DEFAULT
310230539,1,10,1899-12-31 11:01:11,1025,1025,66271,2020,386,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO,DEFAULT
506230136,1,10,1899-12-31 08:01:11,1025,1025,7717,2020,3178,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO,DEFAULT


In [659]:
cols_june <- c('NUM_PRATICA','PROGRESS_PRATICA','SNDG_CEDUTO','RAGIO_SOCIALE_CEDUTO',
               'CODICE_FISCALE_CEDUTO',"IMPORTO RICHIESTO APPLICAZIONE",'ESITO_FINALE_VPC','IMPORTO CONCESSO APPLICAZIONE')

In [660]:
dt_jun_2021 <- dt_completo[year(DATA_DELIBERA_O_ANNULL_FIDO) == '2021' & month(DATA_DELIBERA_O_ANNULL_FIDO) ==  '6',..cols_june]

In [661]:
colnames(dt_jun_2021)

[1] "NUM_PRATICA"                    "PROGRESS_PRATICA"              
[3] "SNDG_CEDUTO"                    "RAGIO_SOCIALE_CEDUTO"          
[5] "CODICE_FISCALE_CEDUTO"          "IMPORTO RICHIESTO APPLICAZIONE"
[7] "ESITO_FINALE_VPC"               "IMPORTO CONCESSO APPLICAZIONE"

In [662]:
write.xlsx(dt_jun_2021,
           'estrazione_2021_06.xlsx',
           sheetName = 'campione',
           row.names = FALSE)

In [663]:

round(100*nrow(dt_orig[`MacroSegmento_Attuale LC` == "Sme Retail" & target == "DEFAULT",])/nrow(dt_orig[`MacroSegmento_Attuale LC` == "Sme Retail",]),2)

[1] 3.76

In [664]:
round(100*nrow(dt_orig[`MacroSegmento_Attuale LC` != "Sme Retail" & target == "DEFAULT",])/nrow(dt_orig[`MacroSegmento_Attuale LC` != "Sme Retail",]),2)

[1] 1.74

In [665]:
colnames(dt_completo)[grepl('ating',colnames(dt_completo))]

[1] "Presenza_Rating"                "Rating_Inizio_Periodo"         
 [3] "Rating_periodo_adj"             "Rating_Fine_periodo_adj"       
 [5] "Media_rating_Per_Ante"          "Rating_Medio_Post_Stipula"     
 [7] "Rating_Attuale"                 "Rating_Iniz_periodo_Ricondotto"
 [9] "MEAN_of_Rating_periodo_adj"     "Rating_Fine_periodo_Ricondotto"
[11] "Rating_Attuale_Ricondotto"      "Primo_rating_Primula"

In [666]:
dt_completo['Flag_utilizzato=0' == 1, ]$Rating_Inizio_Periodo

character(0)

In [667]:
table(dt_completo[`Flag_utilizzato=0` == "SI",Rating_Fine_periodo_adj ],useNA = 'always')


    D    I1  I1.B  I1.C  I1.D  I1.E  I1.F    I2    I3    I4    I5    I6    M1 
  458    76     1     2    15     2     2   314   457   909  1053  1411  2396 
   M2    M3    M4    R1    R2    R3    R4    R5  RT01  RT02  RT03  RT04  RT05 
  968  1231   838   633   355   288   240  5469     2     2     2     2     2 
 RT06  RT07  RT08  RT09  RT10  RT12  RT14    UR  <NA> 
    4     8     9     6     2     2     1  5821 75791 

In [668]:
table(dt_completo$`Flag_utilizzato=0`)


    NO     SI 
154167  98772 

In [670]:
# dt_completo[,flag_default := FALSE]
# dt_completo[Rating_Inizio_Periodo != "D"  & (Rating_periodo_adj  > 15 | Rating_Fine_periodo_Ricondotto == "D",flag_default := TRUE)

In [671]:
rbindlist(lapply(unique(dt_completo$`MacroSegmento_Attuale LC`), function(seg){

    perc <- nrow(
        dt_completo[`Flag_utilizzato=0` =='SI' & Rating_Inizio_Periodo != "D"  & (Rating_periodo_adj  > 15 | Rating_Fine_periodo_Ricondotto == "D") &`MacroSegmento_Attuale LC`==seg,])/
        nrow(dt_completo[`Flag_utilizzato=0` =='SI' & Rating_Inizio_Periodo != "D" &`MacroSegmento_Attuale LC`==seg,])
    return(data.table(macroseg = seg, td = round(100*perc,2)))
}))

macroseg,td
<chr>,<dbl>
Corporate,0.12
Altro,1.24
Sme Retail,0.37
Sme Corporate,0.40
Large corporate,0.00


In [672]:
dt_completo[`Flag_utilizzato=0` =='SI' & Rating_Fine_periodo_adj == "D", ]

NUM_PRATICA,PROGRESS_PRATICA,STATO_PRATICA,CHIAVE_DB,ABI,ABI_PEF5,FILIALE_PEF5,ANNO_PEF5,PROGRESSIVO_PEF5,TIPO_PRATICA,⋯,M3-M4,M1-M2,I5-I6,I4,I3,I1-I2,NUOVI_VERDI,NUOVO_ESITO_FINALE,NUOVI_VERDI_SENZA_SOGLIA,NUOVO_ESITO_SENZA SOGLIA
<dbl>,<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>
116231743,3,10,1899-12-31 07:03:10,NA,NA,NA,NA,NA,2,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,ROSSO,0,ROSSO
120230076,1,10,1899-12-31 12:01:10,NA,NA,NA,NA,NA,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,ROSSO,0,ROSSO
124230037,1,10,1899-12-31 13:01:10,NA,NA,NA,NA,NA,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,ROSSO,0,ROSSO
127230035,1,10,1899-12-31 11:01:10,1025,1025,3139,2020,43,3,⋯,n.a.,300000,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO
127230040,1,10,1899-12-31 16:01:10,NA,NA,NA,NA,NA,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,ROSSO,0,ROSSO
203230181,1,10,1899-12-31 05:01:11,NA,NA,NA,NA,NA,2,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,ROSSO,0,ROSSO
203230321,4,10,1899-12-31 01:04:10,NA,NA,NA,NA,NA,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,ROSSO,0,ROSSO
210230070,1,10,1899-12-31 06:01:10,NA,NA,NA,NA,NA,2,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,ROSSO,0,ROSSO
210230149,1,10,1899-12-31 13:01:10,NA,NA,NA,NA,NA,2,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,ROSSO,0,ROSSO


In [673]:
table(dt_completo[`Flag_utilizzato=0` =='SI',`Input - STATO SAB`],useNA = "always")


    2     3     5     7     8     A     B  <NA> 
37569   696     1   722   141    10    10 59623 

In [674]:
dt_completo

NUM_PRATICA,PROGRESS_PRATICA,STATO_PRATICA,CHIAVE_DB,ABI,ABI_PEF5,FILIALE_PEF5,ANNO_PEF5,PROGRESSIVO_PEF5,TIPO_PRATICA,⋯,M3-M4,M1-M2,I5-I6,I4,I3,I1-I2,NUOVI_VERDI,NUOVO_ESITO_FINALE,NUOVI_VERDI_SENZA_SOGLIA,NUOVO_ESITO_SENZA SOGLIA
<dbl>,<dbl>,<dbl>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>
102163026,1,11,1899-12-31 18:01:11,NA,NA,NA,NA,NA,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO
102163027,1,10,1899-12-31 19:01:10,1025,1025,75000,2020,1442,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO
102163028,1,11,1899-12-31 20:01:11,NA,NA,NA,NA,NA,2,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,GIALLO,0,GIALLO
102230026,1,10,1899-12-31 10:01:10,NA,NA,NA,NA,NA,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,ROSSO,0,ROSSO
102230027,1,10,1899-12-31 11:01:10,NA,NA,NA,NA,NA,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,ROSSO,0,ROSSO
102230028,1,10,1899-12-31 12:01:10,NA,NA,NA,NA,NA,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,VERDE,0,VERDE
102230029,1,10,1899-12-31 13:01:10,NA,NA,NA,NA,NA,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,ROSSO,0,ROSSO
102230029,2,10,1899-12-31 13:02:10,NA,NA,NA,NA,NA,3,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,VERDE,0,VERDE
102230030,1,10,1899-12-31 14:01:10,NA,NA,NA,NA,NA,2,⋯,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.,0,VERDE,0,VERDE


## NON CENSITI 

In [675]:
colnames(master_rating)

[1] "NUM_PRATICA"                     "PROGRESS_PRATICA"               
 [3] "SNDG_CEDUTO"                     "d_fineperiodo_max"              
 [5] "DATA_DELIBERA_O_ANNULL_FIDO"     "MacroSegmento_Attuale LC"       
 [7] "Rating_Inizio_Periodo"           "COD_COLORE_CLASSE_T0"           
 [9] "RAT_FORMA_GIURIDICA"             "RAT_AREA_GEO"                   
[11] "RAT_FAMIGLIA_ANAG"               "RAT_flag_esclusivo_cf"          
[13] "RAT_SIZE"                        "RAT_FLAG_HR"                    
[15] "RAT_DEFAULT"                     "RAT_DEFAULT_S100"               
[17] "RAT_SamplingWeight"              "RAT_score_cr0"                  
[19] "RAT_score_crsys"                 "RAT_score_anag"                 
[21] "RAT_score_bil_integ"             "RAT_score_bil_integ_w"          
[23] "RAT_score_anag_w"                "RAT_score_crsys_w"              
[25] "RAT_score_cr0_w"                 "RAT_score_integ"                
[27] "RAT_FLAG_DENOTCHING"             "RAT_rating_pdc"                 
[29] "RAT_rating_finale"               "RAT_offset"                     
[31] "RAT_prob"                        "RAT_score_integr_offset"        
[33] "RAT_prob_offset"                 "RAT_rating_finale_post_notch"   
[35] "RAT_pd_finale_post_notch"        "RAT_pd_post_lim"                
[37] "RAT_rating_post_cal_Naff_finale" "RAT_classe_rating"              
[39] "RAT_delta_t"                     "RAT_classe_rating_filt"         
[41] "RAT_andamento"                   "CRA_ADJ"                        
[43] "COD_TP_CLI"                      "FORMA_GIURIDICA"

In [676]:
dt_test <- merge(master_rating[,.(NUM_PRATICA,PROGRESS_PRATICA,RAT_rating_finale_post_notch,RAT_rating_post_cal_Naff_finale)],
      master_ews_asis,
      by = c('NUM_PRATICA','PROGRESS_PRATICA'))

In [677]:
colnames(dt_test)

[1] "NUM_PRATICA"                     "PROGRESS_PRATICA"               
 [3] "RAT_rating_finale_post_notch"    "RAT_rating_post_cal_Naff_finale"
 [5] "SNDG_CEDUTO"                     "ESITO_FINALE_VPC"               
 [7] "DATA_DELIBERA_O_ANNULL_FIDO"     "Delibera_Numerica"              
 [9] "MacroSegmento_Attuale"           "Codice_Ews"                     
[11] "Colore.Ews"                      "Data_Ews"                       
[13] "Giorni_dist_Ews_Delibera"        "flag_90gg"                      
[15] "flag_30gg"                       "flag_10gg"                      
[17] "EWS_ASIS"                        "censito"

In [678]:
dt_test[,only_factoring := FALSE]
dt_test[is.na(RAT_rating_post_cal_Naff_finale) & !is.na(RAT_rating_finale_post_notch),only_factoring := TRUE]

In [679]:
table(dt_test$censito)/nrow(dt_test)


       NO        SI 
0.1050118 0.8949882 

In [680]:
table(dt_test$only_factoring)/nrow(dt_test)


     FALSE       TRUE 
0.94310817 0.05689183 

In [687]:
colnames(master_dsi)

[1] "NUM_PRATICA"                             
 [2] "PROGRESS_PRATICA"                        
 [3] "DSI_tipo_pratica"                        
 [4] "DSI_filiale_factoring_ceduto"            
 [5] "DSI_settore_ceduto"                      
 [6] "DSI_divisione_ceduto"                    
 [7] "DSI_importo_richiesto_applicazione"      
 [8] "DSI_settore_cedente"                     
 [9] "DSI_tipo_prodotto"                       
[10] "DSI_rae_ceduto"                          
[11] "DSI_polizza_diretta_39_"                 
[12] "DSI_polizza_indiretta_41_"               
[13] "macrosegmento_attuale_lc"                
[14] "target"                                  
[15] "DSI_sab"                                 
[16] "DSI_col_presenza_pratica_pef"            
[17] "DSI_col_fido_con_causale_negativa"       
[18] "DSI_col_appl_con_causale_negativa"       
[19] "DSI_parte_correlata"                     
[20] "DSI_rapporti_in_osservazione"            
[21] "DSI_causale_negativa_fido"               
[22] "DSI_diff_appr_appl"                      
[23] "DSI_num_fidi_scaduti"                    
[24] "DSI_flag_fidi_scaduti"                   
[25] "DSI_autonomia_fido"                      
[26] "DSI_importo_fido_richiesto"              
[27] "DSI_flag_fido_ridotto"                   
[28] "DSI_causale_negativa_appl"               
[29] "DSI_flag_scaduto_20_outstanding"         
[30] "DSI_flag_presenza_sofferenze"            
[31] "DSI_flag_cr_accordato_utilizzato"        
[32] "DSI_flag_cr_presenza_sconfinamenti"      
[33] "DSI_flag_patrimonio_netto_neg"           
[34] "DSI_flag_debiti_bancari_abreve_fatturato"
[35] "DSI_flag_passivo_corrente_fatturato"     
[36] "DSI_stato_societario"                    
[37] "DSI_flag_data_costituzione"              
[38] "DSI_protesti_certi"                      
[39] "DSI_presenza_fidi_ced"                   
[40] "DSI_modello_rating_t0"                   
[41] "DSI_fallimenti_certi"                    
[42] "DSI_flag_fallimenti_dubbi"               
[43] "DSI_flag_pregiudizievoli_gravi"          
[44] "DSI_flag_procedure_concorsuali"          
[45] "DSI_procedure_imprese_collegate"         
[46] "DSI_delta_scadenza_rating"               
[47] "DSI_imp_richiesto_lim_autonomia"

In [681]:
dt_test <- merge(dt_test,
                dt_orig[,.(NUM_PRATICA,PROGRESS_PRATICA,target,`MacroSegmento_Attuale LC`)],
                by = c('NUM_PRATICA','PROGRESS_PRATICA'))

In [688]:
dt_test <- merge(master_dsi[,.(NUM_PRATICA,PROGRESS_PRATICA,DSI_flag_fidi_scaduti)],
                 dt_test,
                 by.x = c('NUM_PRATICA','PROGRESS_PRATICA'),
                 by.y = c('NUM_PRATICA','PROGRESS_PRATICA')
                )

In [689]:
nrow(dt_test[`MacroSegmento_Attuale LC` == 'Sme Retail' & is.na(EWS_ASIS),])

[1] 67850

In [691]:
table(dt_test[`MacroSegmento_Attuale LC` == 'Sme Retail' & is.na(EWS_ASIS),DSI_flag_fidi_scaduti])


      FALSE        TRUE non censito 
      18674        5645       43531 

In [692]:
100*table(dt_test[censito == 'NO',target])/nrow(dt_test[censito == 'NO',])


  DEFAULT        OK 
 4.975192 95.024808 

In [693]:
100*table(dt_test[censito == 'SI',target])/nrow(dt_test[censito == 'SI',])


  DEFAULT        OK 
 2.789585 97.210415 

In [694]:
colnames(dt_test)

[1] "NUM_PRATICA"                     "PROGRESS_PRATICA"               
 [3] "DSI_flag_fidi_scaduti"           "RAT_rating_finale_post_notch"   
 [5] "RAT_rating_post_cal_Naff_finale" "SNDG_CEDUTO"                    
 [7] "ESITO_FINALE_VPC"                "DATA_DELIBERA_O_ANNULL_FIDO"    
 [9] "Delibera_Numerica"               "MacroSegmento_Attuale"          
[11] "Codice_Ews"                      "Colore.Ews"                     
[13] "Data_Ews"                        "Giorni_dist_Ews_Delibera"       
[15] "flag_90gg"                       "flag_30gg"                      
[17] "flag_10gg"                       "EWS_ASIS"                       
[19] "censito"                         "only_factoring"                 
[21] "target"                          "MacroSegmento_Attuale LC"

In [695]:
dt_noncensiti <- dt_test[censito == 'NO' & only_factoring == TRUE,]

In [697]:
saveRDS(dt_noncensiti[,.(NUM_PRATICA,PROGRESS_PRATICA)],'../data/RDS/dt_test_noncensiti.RDS')